In [2]:
# ================================
# TCC TABELAS 2026 — DIAGNÓSTICO INICIAL
# Passo 2.1 | NÃO gera tabelas
# ================================

from google.colab import drive
import os
import pandas as pd

print("🔹 Montando Google Drive...")
drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/TCC TABELAS 2026"

INPUT_PATH  = os.path.join(BASE_PATH, "01_INPUT")
OUTPUT_PATH = os.path.join(BASE_PATH, "02_OUTPUT")
CONFIG_PATH = os.path.join(BASE_PATH, "03_CONFIG")

print("\n🔹 Verificando estrutura de pastas...")
for p in [INPUT_PATH, OUTPUT_PATH, CONFIG_PATH]:
    print(f"{p} ->", "OK" if os.path.exists(p) else "❌ NÃO ENCONTRADO")

print("\n🔹 Verificando arquivos de INPUT...")
required_inputs = {
    "Scouts_Reorganizado.xlsx": INPUT_PATH,
    "DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv": INPUT_PATH,
    "RODADAS_2026_CONFRONTOS.txt": INPUT_PATH,
}

for f, path in required_inputs.items():
    full = os.path.join(path, f)
    print(f"{f} ->", "OK" if os.path.exists(full) else "❌ NÃO ENCONTRADO")

print("\n🔹 Verificando arquivo PILOTO...")
piloto_path = os.path.join(CONFIG_PATH, "PILOTO_GERACAO_TABELAS.csv")
print("PILOTO_GERACAO_TABELAS.csv ->", "OK" if os.path.exists(piloto_path) else "❌ NÃO ENCONTRADO")

print("\n🔹 Lendo PILOTO_GERACAO_TABELAS.csv...")
piloto = pd.read_csv(piloto_path)

print("\n📌 Conteúdo do PILOTO:")
display(piloto)

# Normalizações defensivas
piloto.columns = [c.strip().upper() for c in piloto.columns]
piloto = piloto.applymap(lambda x: x.strip().upper() if isinstance(x, str) else x)

print("\n🔹 Validações conceituais:")

# FILTRO
filtro = piloto.loc[piloto["CHAVE"]=="FILTRO", "VALOR"].values[0]
if filtro not in ["TODAS", "POR_MANDO"]:
    print(f"❌ FILTRO inválido: {filtro}")
else:
    print(f"✅ FILTRO = {filtro}")

# N_JOGOS
n_jogos = int(piloto.loc[piloto["CHAVE"]=="N_JOGOS", "VALOR"].values[0])
rodada  = int(piloto.loc[piloto["CHAVE"]=="RODADA_ATUAL", "VALOR"].values[0])

if n_jogos >= rodada:
    print(f"❌ N_JOGOS ({n_jogos}) >= RODADA_ATUAL ({rodada}) → filtro impossível")
else:
    print(f"✅ N_JOGOS = {n_jogos} | RODADA_ATUAL = {rodada}")

print("\n✅ DIAGNÓSTICO FINALIZADO COM SUCESSO")
print("👉 Nenhuma tabela foi gerada ainda.")



🔹 Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🔹 Verificando estrutura de pastas...
/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT -> OK
/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT -> OK
/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG -> OK

🔹 Verificando arquivos de INPUT...
Scouts_Reorganizado.xlsx -> OK
DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv -> OK
RODADAS_2026_CONFRONTOS.txt -> OK

🔹 Verificando arquivo PILOTO...
PILOTO_GERACAO_TABELAS.csv -> OK

🔹 Lendo PILOTO_GERACAO_TABELAS.csv...

📌 Conteúdo do PILOTO:


,CHAVE,VALOR
0,RODADA_REFERENCIA,39
1,N_JOGOS,5
2,FILTRO,POR_MANDO
3,GERAR_TABELAS,SIM



🔹 Validações conceituais:
✅ FILTRO = POR_MANDO


/tmp/ipython-input-2352787526.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  piloto = piloto.applymap(lambda x: x.strip().upper() if isinstance(x, str) else x)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import os
import pandas as pd

BASE_PATH = "/content/drive/MyDrive/TCC TABELAS 2026"
CONFIG_PATH = os.path.join(BASE_PATH, "03_CONFIG")
piloto_path = os.path.join(CONFIG_PATH, "PILOTO_GERACAO_TABELAS.csv")

# Lê SEM cabeçalho e força 2 colunas: CHAVE / VALOR
piloto = pd.read_csv(piloto_path, header=None, names=["CHAVE", "VALOR"])

# Normaliza texto
piloto["CHAVE"] = piloto["CHAVE"].astype(str).str.strip().str.upper()
piloto["VALOR"] = piloto["VALOR"].astype(str).str.strip().str.upper()

print("✅ PILOTO lido com sucesso. Conteúdo:")
display(piloto)

def get(chave):
    return piloto.loc[piloto["CHAVE"]==chave, "VALOR"].values[0]

filtro = get("FILTRO")
rodada = int(get("RODADA_ATUAL"))
n_jogos = int(get("N_JOGOS"))
temporada = get("TEMPORADA")
gerar = get("GERAR_TABELAS")

print("\n🔹 Validações:")
print("TEMPORADA:", temporada)
print("RODADA_ATUAL:", rodada)
print("N_JOGOS:", n_jogos)
print("FILTRO:", filtro)
print("GERAR_TABELAS:", gerar)

if filtro not in ["TODAS", "POR_MANDO"]:
    raise ValueError(f"❌ FILTRO inválido: {filtro} (use TODAS ou POR_MANDO)")

# Regra: N_JOGOS precisa ser < RODADA_ATUAL
if n_jogos >= rodada:
    raise ValueError(f"❌ N_JOGOS ({n_jogos}) >= RODADA_ATUAL ({rodada}) → impossível")

print("\n✅ PILOTO OK. Próximo passo: motor Python (geração).")


In [ ]:
import os, re, json, time
import pandas as pd

BASE = "/content/drive/MyDrive/TCC TABELAS 2026"
P_INPUT  = os.path.join(BASE, "01_INPUT")
P_OUTPUT = os.path.join(BASE, "02_OUTPUT")
P_CONFIG = os.path.join(BASE, "03_CONFIG")

# Arquivos (nomes EXATOS que você mostrou no Drive)
F_SCOUTS   = os.path.join(P_INPUT, "Scouts_Reorganizado.xlsx")
F_DIVISAO  = os.path.join(P_INPUT, "DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv")
F_RODADAS  = os.path.join(P_INPUT, "RODADAS_2026_CONFRONTOS.txt")
F_PILOTO   = os.path.join(P_CONFIG, "PILOTO_GERACAO_TABELAS.csv")

def assert_exists(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ Não achei: {path}")
    print("✅", os.path.basename(path), "-> OK")

print("🔹 Checando arquivos...")
for f in [F_SCOUTS, F_DIVISAO, F_RODADAS, F_PILOTO]:
    assert_exists(f)

print("\n🔹 Lendo PILOTO...")
piloto = pd.read_csv(F_PILOTO, header=None, names=["CHAVE","VALOR"])
piloto["CHAVE"] = piloto["CHAVE"].astype(str).str.strip().str.upper()
piloto["VALOR"] = piloto["VALOR"].astype(str).str.strip().str.upper()

def getv(key, default=None):
    s = piloto.loc[piloto["CHAVE"]==key, "VALOR"]
    return s.values[0] if len(s) else default

temporada = int(getv("TEMPORADA"))
rodada    = int(getv("RODADA_ATUAL"))
n_jogos   = int(getv("N_JOGOS"))
filtro    = getv("FILTRO")
gerar     = getv("GERAR_TABELAS")

if filtro not in ["TODAS", "POR_MANDO"]:
    raise ValueError(f"❌ FILTRO inválido: {filtro} (use TODAS ou POR_MANDO)")
if n_jogos >= rodada:
    raise ValueError(f"❌ N_JOGOS ({n_jogos}) >= RODADA_ATUAL ({rodada}) -> impossível")
if gerar != "SIM":
    raise ValueError("❌ GERAR_TABELAS != SIM (o motor foi bloqueado pelo piloto)")

print(f"✅ PILOTO: temporada={temporada} rodada={rodada} n_jogos={n_jogos} filtro={filtro} gerar={gerar}")

# ---- Ler confrontos da rodada (parser flexível) ----
print("\n🔹 Lendo confrontos...")
txt = open(F_RODADAS, "r", encoding="utf-8", errors="ignore").read().splitlines()

# Tentativa 1: linhas que contenham a rodada e 'x' ou '×'
round_lines = []
for line in txt:
    if re.search(rf"\b{rodada}\b", line) and ((" x " in line.lower()) or ("×" in line)):
        round_lines.append(line.strip())

# Fallback: bloco por "RODADA {rodada}"
if not round_lines:
    start_idx = None
    for i,l in enumerate(txt):
        if re.search(rf"rodada\s*{rodada}\b", l.lower()):
            start_idx = i
            break
    if start_idx is not None:
        for l in txt[start_idx:start_idx+60]:
            if (" x " in l.lower()) or ("×" in l):
                round_lines.append(l.strip())

def parse_match(line):
    # remove prefixos comuns
    s = re.sub(r"^\s*(rodada\s*\d+\s*[:\-]?\s*)", "", line, flags=re.I).strip()
    s = s.replace("×", " x ")
    parts = re.split(r"\s+x\s+", s, flags=re.I)
    if len(parts) != 2:
        return None
    home = parts[0].strip().upper()
    away = parts[1].strip().upper()
    # limpa múltiplos espaços
    home = re.sub(r"\s+", " ", home)
    away = re.sub(r"\s+", " ", away)
    return home, away

fixtures = []
for l in round_lines:
    m = parse_match(l)
    if m:
        fixtures.append(m)

if not fixtures:
    raise ValueError("❌ Não consegui extrair os confrontos da rodada do TXT. Me mande o TXT (ou um print do formato).")

print(f"✅ Confrontos encontrados: {len(fixtures)}")
# print(fixtures)  # descomente se quiser ver

# ---- Ler times disponíveis no Scouts_Reorganizado (só para checagem anti-bug) ----
print("\n🔹 Checando times existentes no Scouts_Reorganizado.xlsx...")
xls = pd.ExcelFile(F_SCOUTS)
# tenta achar uma aba com coluna Time/Clube
candidate_sheets = xls.sheet_names
team_col = None
team_series = None

for sh in candidate_sheets:
    df0 = pd.read_excel(F_SCOUTS, sheet_name=sh, nrows=200)
    cols = [c.strip().upper() for c in df0.columns.astype(str)]
    if "TIME" in cols:
        team_col = df0.columns[cols.index("TIME")]
        team_series = df0[team_col].dropna().astype(str).str.strip().str.upper()
        break
    if "CLUBE" in cols:
        team_col = df0.columns[cols.index("CLUBE")]
        team_series = df0[team_col].dropna().astype(str).str.strip().str.upper()
        break

teams_in_scouts = set(team_series.unique()) if team_series is not None else set()

missing_teams = sorted({t for fx in fixtures for t in fx if t not in teams_in_scouts}) if teams_in_scouts else []
if teams_in_scouts:
    print(f"✅ Times detectados na base: {len(teams_in_scouts)}")
    if missing_teams:
        print("⚠️ Times presentes no TXT mas AUSENTES no Scouts_Reorganizado (isso é esperado com base 2025):")
        for t in missing_teams:
            print(" -", t)
else:
    print("⚠️ Não consegui detectar a coluna TIME/CLUBE automaticamente. Sem checagem anti-bug por nome (sem travar).")

# ---- Schemas (colunas) - ESQUELETO por posição (sem cálculo ainda) ----
schemas = {
    "GOLEIROS":  ["HOME","AWAY"],  # v0: só confrontos; cálculo entra depois
    "ZAGUEIROS": ["HOME","AWAY"],
    "LATERAIS":  ["HOME","AWAY"],
    "MEIAS":     ["HOME","AWAY"],
    "ATACANTES": ["HOME","AWAY"],
}

# ---- Criar pasta de output com timestamp ----
run_id = time.strftime("%Y%m%d_%H%M%S")
out_dir = os.path.join(P_OUTPUT, f"RUN_{temporada}_R{rodada:02d}_{run_id}")
os.makedirs(out_dir, exist_ok=True)

# ---- Gerar CSVs ----
rows = [{"HOME":h, "AWAY":a} for (h,a) in fixtures]
for pos, cols in schemas.items():
    df = pd.DataFrame(rows)[cols]
    out_csv = os.path.join(out_dir, f"TABELA_{pos}_R{rodada:02d}.csv")
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print("✅ Gerado:", os.path.basename(out_csv))

# ---- Metadata ----
meta = {
    "temporada": temporada,
    "rodada_atual": rodada,
    "n_jogos": n_jogos,
    "filtro": filtro,
    "inputs": {
        "scouts_reorganizado": os.path.basename(F_SCOUTS),
        "divisao_meias_volantes": os.path.basename(F_DIVISAO),
        "rodadas_confrontos": os.path.basename(F_RODADAS),
        "piloto": os.path.basename(F_PILOTO),
    },
    "confrontos": [{"home":h, "away":a} for (h,a) in fixtures],
    "missing_teams_vs_base_2025": missing_teams,
    "status": "OK_ESQUELETO_GERADO"
}
with open(os.path.join(out_dir, "metadata.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("\n✅ Motor v0 (esqueleto) finalizado.")
print("📁 Saída em:", out_dir)


In [ ]:
import os, re, json, time
import pandas as pd

BASE = "/content/drive/MyDrive/TCC TABELAS 2026"
P_INPUT  = os.path.join(BASE, "01_INPUT")
P_OUTPUT = os.path.join(BASE, "02_OUTPUT")
P_CONFIG = os.path.join(BASE, "03_CONFIG")

F_SCOUTS   = os.path.join(P_INPUT, "Scouts_Reorganizado.xlsx")
F_DIVISAO  = os.path.join(P_INPUT, "DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv")
F_RODADAS  = os.path.join(P_INPUT, "RODADAS_2026_CONFRONTOS.txt")
F_PILOTO   = os.path.join(P_CONFIG, "PILOTO_GERACAO_TABELAS.csv")

# --- PILOTO ---
piloto = pd.read_csv(F_PILOTO, header=None, names=["CHAVE","VALOR"])
piloto["CHAVE"] = piloto["CHAVE"].astype(str).str.strip().str.upper()
piloto["VALOR"] = piloto["VALOR"].astype(str).str.strip().str.upper()

def getv(key, default=None):
    s = piloto.loc[piloto["CHAVE"]==key, "VALOR"]
    return s.values[0] if len(s) else default

temporada = int(getv("TEMPORADA"))
rodada    = int(getv("RODADA_ATUAL"))
n_jogos   = int(getv("N_JOGOS"))
filtro    = getv("FILTRO")
gerar     = getv("GERAR_TABELAS")

if filtro not in ["TODAS", "POR_MANDO"]:
    raise ValueError(f"❌ FILTRO inválido: {filtro} (use TODAS ou POR_MANDO)")
if n_jogos >= rodada:
    raise ValueError(f"❌ N_JOGOS ({n_jogos}) >= RODADA_ATUAL ({rodada}) -> impossível")
if gerar != "SIM":
    raise ValueError("❌ GERAR_TABELAS != SIM")

print(f"✅ PILOTO ok: temporada={temporada} rodada={rodada} n_jogos={n_jogos} filtro={filtro}")

# --- PARSER ROBUSTO (por bloco RODADA N) ---
lines = open(F_RODADAS, "r", encoding="utf-8", errors="ignore").read().splitlines()

def find_block(lines, rodada):
    start = None
    end = None
    # acha o começo: "RODADA 1" (flexível)
    pat_start = re.compile(rf"^\s*rodada\s*{rodada}\b", re.I)
    pat_any_round = re.compile(r"^\s*rodada\s*\d+\b", re.I)

    for i, l in enumerate(lines):
        if pat_start.search(l):
            start = i
            break
    if start is None:
        return None, None

    # fim: próxima "RODADA <número>" depois do start
    for j in range(start + 1, len(lines)):
        if pat_any_round.search(lines[j]):
            end = j
            break

    if end is None:
        end = len(lines)
    return start, end

def parse_match(line):
    s = line.strip()
    s = s.replace("×", " x ")
    # remove prefixos tipo "1 -", "JOGO:", etc
    s = re.sub(r"^\s*[\-\•\*\d\)\(]+[\s\-:]+", "", s)
    parts = re.split(r"\s+x\s+", s, flags=re.I)
    if len(parts) != 2:
        return None
    home = re.sub(r"\s+", " ", parts[0].strip()).upper()
    away = re.sub(r"\s+", " ", parts[1].strip()).upper()
    return home, away

start, end = find_block(lines, rodada)
if start is None:
    raise ValueError("❌ Não encontrei o cabeçalho 'RODADA X' no TXT.")

block = lines[start:end]
fixtures = []
for l in block:
    if (" x " in l.lower()) or ("×" in l):
        m = parse_match(l)
        if m:
            fixtures.append(m)

# remove duplicados preservando ordem
seen = set()
uniq = []
for h,a in fixtures:
    key = (h,a)
    if key not in seen:
        seen.add(key)
        uniq.append((h,a))
fixtures = uniq

print(f"✅ Confrontos da rodada {rodada}: {len(fixtures)}")
if len(fixtures) != 10:
    print("⚠️ Atenção: esperado 10 confrontos (Série A). Vou listar o que foi extraído:")
    for h,a in fixtures:
        print(" -", h, "x", a)

# --- Checagem anti-bug (times faltando na base 2025) ---
xls = pd.ExcelFile(F_SCOUTS)
team_series = None
for sh in xls.sheet_names:
    df0 = pd.read_excel(F_SCOUTS, sheet_name=sh, nrows=300)
    cols = [c.strip().upper() for c in df0.columns.astype(str)]
    if "TIME" in cols:
        team_series = df0[df0.columns[cols.index("TIME")]].dropna().astype(str).str.strip().str.upper()
        break
    if "CLUBE" in cols:
        team_series = df0[df0.columns[cols.index("CLUBE")]].dropna().astype(str).str.strip().str.upper()
        break

teams_in_scouts = set(team_series.unique()) if team_series is not None else set()
missing_teams = sorted({t for fx in fixtures for t in fx if t not in teams_in_scouts}) if teams_in_scouts else []

if missing_teams:
    print("⚠️ Times presentes no TXT mas AUSENTES no Scouts_Reorganizado (base 2025):")
    for t in missing_teams:
        print(" -", t)

# --- Gerar outputs (esqueleto) ---
run_id = time.strftime("%Y%m%d_%H%M%S")
out_dir = os.path.join(P_OUTPUT, f"RUN_{temporada}_R{rodada:02d}_{run_id}_FIX")
os.makedirs(out_dir, exist_ok=True)

rows = [{"HOME":h, "AWAY":a} for (h,a) in fixtures]
for pos in ["GOLEIROS","ZAGUEIROS","LATERAIS","MEIAS","ATACANTES"]:
    df = pd.DataFrame(rows)[["HOME","AWAY"]]
    out_csv = os.path.join(out_dir, f"TABELA_{pos}_R{rodada:02d}.csv")
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print("✅ Gerado:", os.path.basename(out_csv))

meta = {
    "temporada": temporada,
    "rodada_atual": rodada,
    "n_jogos": n_jogos,
    "filtro": filtro,
    "confrontos_extraidos": len(fixtures),
    "missing_teams_vs_base_2025": missing_teams,
    "status": "OK_ESQUELETO_GERADO_FIX"
}
with open(os.path.join(out_dir, "metadata.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("\n✅ Motor v0 (FIX) finalizado.")
print("📁 Saída em:", out_dir)


In [ ]:
from pathlib import Path
import pandas as pd

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

print("✅ Pastas:")
for p in [INP, OUT, CFG]:
    print(p, "->", "OK" if p.exists() else "❌ NÃO ENCONTRADA")

print("\n✅ Arquivos esperados:")
files = {
    "Scouts_Reorganizado.xlsx": INP/"Scouts_Reorganizado.xlsx",
    "DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv": INP/"DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv",
    "RODADAS_2026_CONFRONTOS.txt": INP/"RODADAS_2026_CONFRONTOS.txt",
    "PILOTO_GERACAO_TABELAS.csv": CFG/"PILOTO_GERACAO_TABELAS.csv",
}
for k,v in files.items():
    print(k, "->", "OK" if v.exists() else "❌ FALTANDO")

# --- Ler PILOTO ---
piloto = pd.read_csv(files["PILOTO_GERACAO_TABELAS.csv"])
# normaliza nomes e valores (aceita POR MANDO e POR_MANDO)
piloto.columns = [c.strip().upper() for c in piloto.columns]
piloto = piloto.applymap(lambda x: x.strip().upper() if isinstance(x,str) else x)

required_cols = {"CHAVE","VALOR"}
if not required_cols.issubset(set(piloto.columns)):
    raise ValueError(f"PILOTO inválido. Precisa ter colunas {required_cols}. Veio: {list(piloto.columns)}")

def get_p(ch):
    return piloto.loc[piloto["CHAVE"]==ch, "VALOR"].values[0]

RODADA_ATUAL = int(get_p("RODADA_ATUAL"))
N_JOGOS      = int(get_p("N_JOGOS"))
FILTRO       = get_p("FILTRO").replace(" ", "_")  # POR MANDO -> POR_MANDO
GERAR        = get_p("GERAR_TABELAS")

if FILTRO not in ["TODAS","POR_MANDO"]:
    raise ValueError(f"FILTRO inválido: {FILTRO}. Use TODAS ou POR_MANDO.")

if GERAR != "SIM":
    raise ValueError("GERAR_TABELAS está diferente de SIM. Ajuste no PILOTO.")

print("\n📌 PILOTO OK:")
print("RODADA_ATUAL:", RODADA_ATUAL)
print("N_JOGOS:", N_JOGOS)
print("FILTRO:", FILTRO)
print("GERAR_TABELAS:", GERAR)

# --- Ler confrontos ---
txt = files["RODADAS_2026_CONFRONTOS.txt"].read_text(encoding="utf-8", errors="ignore").splitlines()
rows = []
for line in txt:
    line = line.strip()
    if not line or line.startswith("#"):
        continue
    parts = [p.strip() for p in line.split(";")]
    if len(parts) < 3:
        continue
    try:
        rodada = int(parts[0])
    except:
        continue
    mandante = parts[1]
    visitante = parts[2]
    rows.append((rodada, mandante, visitante))

df_conf = pd.DataFrame(rows, columns=["rodada","mandante","visitante"])
df_r = df_conf[df_conf["rodada"] == RODADA_ATUAL].copy()

print("\n✅ Confrontos da rodada", RODADA_ATUAL, ":", len(df_r))
display(df_r.head(10))

# --- Export teste (para provar pipeline) ---
OUT.mkdir(parents=True, exist_ok=True)
out_file = OUT / f"TESTE_CONFRONTOS_RODADA_{RODADA_ATUAL:02d}.csv"
df_r.to_csv(out_file, index=False, encoding="utf-8-sig")
print("\n✅ Salvo:", out_file)


In [ ]:
from pathlib import Path
import pandas as pd

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

piloto_path = CFG/"PILOTO_GERACAO_TABELAS.csv"

def load_piloto_robusto(path):
    # 1) tenta ler com cabeçalho normal
    df = pd.read_csv(path)
    cols = [c.strip().upper() for c in df.columns]
    df.columns = cols

    # Caso A: já veio CHAVE/VALOR
    if set(["CHAVE","VALOR"]).issubset(set(df.columns)):
        df = df[["CHAVE","VALOR"]].copy()

    # Caso B: veio como ['TEMPORADA','2026'] (primeira linha virou header)
    # então relê SEM header, e usa as 2 colunas como CHAVE/VALOR
    elif len(df.columns) == 2 and "CHAVE" not in df.columns and "VALOR" not in df.columns:
        df = pd.read_csv(path, header=None)
        df = df.iloc[:, :2].copy()
        df.columns = ["CHAVE","VALOR"]

    else:
        raise ValueError(f"PILOTO inválido. Colunas encontradas: {list(df.columns)}")

    # normalização
    df["CHAVE"] = df["CHAVE"].astype(str).str.strip().str.upper()
    df["VALOR"] = df["VALOR"].astype(str).str.strip().str.upper()
    df["VALOR"] = df["VALOR"].str.replace(r"\s+", " ", regex=True)

    return df

piloto = load_piloto_robusto(piloto_path)

def get_p(ch):
    v = piloto.loc[piloto["CHAVE"]==ch, "VALOR"]
    if len(v)==0:
        raise ValueError(f"Chave não encontrada no PILOTO: {ch}")
    return v.values[0]

RODADA_ATUAL = int(get_p("RODADA_ATUAL"))
N_JOGOS      = int(get_p("N_JOGOS"))
FILTRO       = get_p("FILTRO").replace(" ", "_")  # POR MANDO -> POR_MANDO
GERAR        = get_p("GERAR_TABELAS")

if FILTRO not in ["TODAS","POR_MANDO"]:
    raise ValueError(f"FILTRO inválido: {FILTRO}. Use TODAS ou POR_MANDO.")
if GERAR != "SIM":
    raise ValueError("GERAR_TABELAS precisa ser SIM.")

print("✅ PILOTO LIDO COM SUCESSO:")
display(piloto)

print("\n📌 PARAMETROS:")
print("RODADA_ATUAL:", RODADA_ATUAL)
print("N_JOGOS:", N_JOGOS)
print("FILTRO:", FILTRO)
print("GERAR_TABELAS:", GERAR)

# ---- confrontos ----
conf_path = INP/"RODADAS_2026_CONFRONTOS.txt"
txt = conf_path.read_text(encoding="utf-8", errors="ignore").splitlines()

rows = []
for line in txt:
    line = line.strip()
    if not line or line.startswith("#"):
        continue
    parts = [p.strip() for p in line.split(";")]
    if len(parts) < 3:
        continue
    try:
        rodada = int(parts[0])
    except:
        continue
    rows.append((rodada, parts[1], parts[2]))

df_conf = pd.DataFrame(rows, columns=["rodada","mandante","visitante"])
df_r = df_conf[df_conf["rodada"] == RODADA_ATUAL].copy()

print("\n✅ Confrontos da rodada", RODADA_ATUAL, ":", len(df_r))
display(df_r)

OUT.mkdir(parents=True, exist_ok=True)
out_file = OUT / f"TESTE_CONFRONTOS_RODADA_{RODADA_ATUAL:02d}.csv"
df_r.to_csv(out_file, index=False, encoding="utf-8-sig")
print("\n✅ Salvo:", out_file)


In [ ]:
import re
import pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"

path_txt = INP/"RODADAS_2026_CONFRONTOS.txt"

lines = path_txt.read_text(encoding="utf-8", errors="ignore").splitlines()

rodada_atual = None
rows = []

# padrões:
re_rodada = re.compile(r"^\s*Rodada\s+(\d+)\b", re.IGNORECASE)
re_jogo   = re.compile(r"^\s*(.+?)\s*[xX]\s*(.+?)\s*$")  # "Time x Time" (com espaços variáveis)

for raw in lines:
    line = raw.strip()
    if not line:
        continue

    m = re_rodada.match(line)
    if m:
        rodada_atual = int(m.group(1))
        continue

    m2 = re_jogo.match(line)
    if m2 and rodada_atual is not None:
        mandante = m2.group(1).strip()
        visitante = m2.group(2).strip()
        rows.append({"rodada": rodada_atual, "mandante": mandante, "visitante": visitante})

df = pd.DataFrame(rows)

print("✅ Total de jogos extraídos:", len(df))
print("✅ Rodadas detectadas:", df["rodada"].nunique() if not df.empty else 0)

# checagem: em pontos corridos, normalmente 10 jogos por rodada
if not df.empty:
    print("\n📌 Jogos por rodada (primeiras 10 rodadas listadas):")
    print(df.groupby("rodada").size().head(10))

# salva um CSV normalizado para conferência rápida
out_csv = OUT/"CONFRONTOS_2026_NORMALIZADO.csv"
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("\n💾 CSV normalizado salvo em:", out_csv)

df.head(20)


In [ ]:
import pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"

# 1) Lê o PILOTO (já está funcionando com CHAVE/VALOR)
piloto_path = BASE/"03_CONFIG"/"PILOTO_GERACAO_TABELAS.csv"
piloto_df = pd.read_csv(piloto_path)
def get_val(chave):
    return piloto_df.loc[piloto_df["CHAVE"].str.upper()==chave, "VALOR"].values[0]
RODADA_ATUAL = int(get_val("RODADA_ATUAL"))

# 2) Lê confrontos normalizados (CSV)
conf_path = INP/"CONFRONTOS_2026_NORMALIZADO.csv"
df_conf = pd.read_csv(conf_path)

# garante tipos
df_conf["rodada"] = pd.to_numeric(df_conf["rodada"], errors="coerce").astype("Int64")

# 3) Filtra a rodada atual
df_r = df_conf[df_conf["rodada"] == RODADA_ATUAL].copy()

print("✅ Rodada do piloto:", RODADA_ATUAL)
print("✅ Confrontos encontrados:", len(df_r))
display(df_r)

# 4) Salva teste
out_test = OUT/f"TESTE_CONFRONTOS_RODADA_{RODADA_ATUAL:02d}.csv"
df_r.to_csv(out_test, index=False, encoding="utf-8-sig")
print("\n💾 Salvo:", out_test)


In [ ]:
import pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
PILOTO_PATH = BASE/"03_CONFIG"/"PILOTO_GERACAO_TABELAS.csv"

print("📌 Lendo PILOTO em:", PILOTO_PATH)

# Leitura blindada (detecta separador automaticamente)
piloto_df = pd.read_csv(PILOTO_PATH, sep=None, engine="python")

print("✅ Colunas detectadas:", list(piloto_df.columns))
display(piloto_df.head(10))

# Normaliza para o padrão CHAVE/VALOR caso venha “torto”
if "CHAVE" not in piloto_df.columns or "VALOR" not in piloto_df.columns:
    # Caso 1: CSV veio com 2 colunas mas sem header correto (ex.: TEMPORADA | 2026)
    if piloto_df.shape[1] == 2:
        piloto_df.columns = ["CHAVE", "VALOR"]
    else:
        raise ValueError(f"PILOTO inválido. Esperado 2 colunas. Veio {piloto_df.shape[1]} colunas: {list(piloto_df.columns)}")

# Limpeza
piloto_df["CHAVE"] = piloto_df["CHAVE"].astype(str).str.strip().str.upper()
piloto_df["VALOR"] = piloto_df["VALOR"].astype(str).str.strip().str.upper()

print("\n✅ PILOTO NORMALIZADO:")
display(piloto_df)


In [ ]:
import pandas as pd
from pathlib import Path

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
PILOTO_PATH = BASE/"03_CONFIG"/"PILOTO_GERACAO_TABELAS.csv"

df = pd.read_csv(PILOTO_PATH, sep=None, engine="python")
print("Colunas originais:", list(df.columns))

# Caso especial: colunas vieram como ["TEMPORADA", "2026"]
col0 = str(df.columns[0]).strip().upper()
col1 = str(df.columns[1]).strip()

if col0 == "TEMPORADA" and col1.isdigit():
    temporada = col1  # vem do cabeçalho
    # monta linhas CHAVE/VALOR
    piloto_ok = pd.DataFrame({
        "CHAVE": ["TEMPORADA"] + df.iloc[:,0].astype(str).tolist(),
        "VALOR": [temporada]   + df.iloc[:,1].astype(str).tolist()
    })
else:
    # caso normal: tenta mapear para CHAVE/VALOR
    if df.shape[1] == 2:
        piloto_ok = df.copy()
        piloto_ok.columns = ["CHAVE", "VALOR"]
    else:
        raise ValueError(f"PILOTO inválido: esperado 2 colunas, veio {df.shape[1]}")

# limpeza
piloto_ok["CHAVE"] = piloto_ok["CHAVE"].astype(str).str.strip().str.upper()
piloto_ok["VALOR"] = piloto_ok["VALOR"].astype(str).str.strip().str.upper()

print("\n✅ PILOTO FINAL (CHAVE/VALOR):")
display(piloto_ok)

# salva corrigido por cima (garante que o motor não quebra mais)
piloto_ok.to_csv(PILOTO_PATH, index=False, encoding="utf-8-sig")
print(f"\n💾 Salvo corrigido em: {PILOTO_PATH}")


In [ ]:
# ============================
# MOTOR v1 (MEIAS) - DATA FIRST
# ============================
import pandas as pd
import numpy as np
import re
from pathlib import Path
from datetime import datetime

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

PILOTO_PATH = CFG/"PILOTO_GERACAO_TABELAS.csv"
DIV_PATH    = INP/"DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"
SCOUTS_XLSX  = INP/"Scouts_Reorganizado.xlsx"
TXT_CONFR    = INP/"RODADAS_2026_CONFRONTOS.txt"
CSV_CONFR_N  = OUT/"CONFRONTOS_2026_NORMALIZADO.csv"

def norm_txt(s):
    if pd.isna(s):
        return ""
    s = str(s).strip().upper()
    s = re.sub(r"\s+", " ", s)
    return s

def read_piloto(path: Path):
    df = pd.read_csv(path, encoding="utf-8-sig")
    # Blindagem: se vier em formato "duas colunas" sem header (caso extremo)
    if df.shape[1] == 2 and ("CHAVE" not in df.columns or "VALOR" not in df.columns):
        df.columns = ["CHAVE","VALOR"]
    # Se vier em formato esquisito com TEMPORADA/2026 como header
    if df.shape[1] == 2 and list(df.columns) == ["TEMPORADA","2026"]:
        df = df.rename(columns={"TEMPORADA":"CHAVE","2026":"VALOR"})
    # Se vier correto mas com BOM na coluna
    df.columns = [c.replace("\ufeff","") for c in df.columns]
    if not set(["CHAVE","VALOR"]).issubset(df.columns):
        raise ValueError(f"PILOTO inválido. Precisa ter colunas CHAVE/VALOR. Veio: {list(df.columns)}")

    df["CHAVE"] = df["CHAVE"].astype(str).map(norm_txt)
    df["VALOR"] = df["VALOR"].astype(str).map(norm_txt)

    def getv(k, default=None):
        v = df.loc[df["CHAVE"]==k, "VALOR"]
        return v.values[0] if len(v) else default

    out = {
        "TEMPORADA": int(getv("TEMPORADA", "2026")),
        "RODADA_ATUAL": int(getv("RODADA_ATUAL", "1")),
        "N_JOGOS": int(getv("N_JOGOS", "0")),
        "FILTRO": getv("FILTRO", "TODAS"),
        "GERAR_TABELAS": getv("GERAR_TABELAS", "SIM"),
    }

    if out["FILTRO"] not in ["TODAS","POR_MANDO"]:
        raise ValueError(f"FILTRO inválido: {out['FILTRO']} (use TODAS ou POR_MANDO)")

    # Regra de início de campeonato: N_JOGOS pode ser 0 na rodada 1.
    # Para rodada >= 2: N_JOGOS deve ser >=1 e < rodada_atual (se você quiser manter sua regra rígida)
    if out["RODADA_ATUAL"] >= 2:
        if out["N_JOGOS"] <= 0:
            raise ValueError("N_JOGOS precisa ser >= 1 a partir da rodada 2.")
        if out["N_JOGOS"] >= out["RODADA_ATUAL"]:
            raise ValueError(f"N_JOGOS ({out['N_JOGOS']}) >= RODADA_ATUAL ({out['RODADA_ATUAL']}) -> inválido")

    return out

def parse_confrontos_txt(txt_path: Path):
    """
    Lê o txt no formato:
    Rodada X (dd/mm, ... ou dd/mm, ...)
      TimeA x TimeB
    Retorna df com [rodada, mandante, visitante, data_ref]
    data_ref = primeira data listada na rodada (usada como data-corte)
    """
    lines = txt_path.read_text(encoding="utf-8", errors="ignore").splitlines()
    rodada = None
    data_ref = None
    rows = []

    for ln in lines:
        ln = ln.strip()
        if not ln:
            continue

        m = re.match(r"^Rodada\s+(\d+)\s*\((.+)\)\s*$", ln, flags=re.IGNORECASE)
        if m:
            rodada = int(m.group(1))
            # pega a primeira data dd/mm do parêntese
            dm = re.search(r"(\d{2}/\d{2})", m.group(2))
            if dm:
                data_ref = dm.group(1) + f"/{datetime.now().year}"
                # Ano não está no txt; como você está trabalhando 2026, força 2026:
                data_ref = dm.group(1) + "/2026"
                data_ref = datetime.strptime(data_ref, "%d/%m/%Y").date()
            else:
                data_ref = None
            continue

        # confronto: "Time A x Time B"
        if " x " in ln.lower() or " X " in ln:
            parts = re.split(r"\s+[xX]\s+", ln)
            if len(parts) == 2 and rodada is not None:
                mand = norm_txt(parts[0])
                vis  = norm_txt(parts[1])
                rows.append({"rodada": rodada, "mandante": mand, "visitante": vis, "data_ref": data_ref})

    df = pd.DataFrame(rows)
    return df

def load_confrontos():
    if CSV_CONFR_N.exists():
        df = pd.read_csv(CSV_CONFR_N, encoding="utf-8-sig")
        df.columns = [norm_txt(c) for c in df.columns]
        # espera RODADA/MANDANTE/VISITANTE
        # se estiver em lowercase:
        colmap = {}
        for c in df.columns:
            if c == "RODADA": colmap[c] = "rodada"
            if c == "MANDANTE": colmap[c] = "mandante"
            if c == "VISITANTE": colmap[c] = "visitante"
        df = df.rename(columns=colmap)
        if not set(["rodada","mandante","visitante"]).issubset(df.columns):
            # fallback: tenta ler do txt
            df = parse_confrontos_txt(TXT_CONFR)
    else:
        df = parse_confrontos_txt(TXT_CONFR)

    # garante normalização
    df["mandante"] = df["mandante"].map(norm_txt)
    df["visitante"] = df["visitante"].map(norm_txt)
    return df

def detect_date_col(df):
    # procura colunas prováveis de data
    for cand in ["DATA","DATE","DIA","DT"]:
        if cand in [norm_txt(c) for c in df.columns]:
            # acha o nome original
            for c in df.columns:
                if norm_txt(c) == cand:
                    return c
    # fallback: procura "DATA" dentro do nome
    for c in df.columns:
        if "DATA" in norm_txt(c):
            return c
    raise ValueError("Não encontrei coluna de DATA na aba 'Por jogo'.")

def build_match_id(df, date_col, time_col, adv_col, mando_col):
    # cria mandante/visitante por linha usando MANDO
    mando = df[mando_col].map(norm_txt)
    time  = df[time_col].map(norm_txt)
    adv   = df[adv_col].map(norm_txt)

    mandante = np.where(mando=="CASA", time, adv)
    visitante = np.where(mando=="CASA", adv, time)

    df["MANDANTE_FIX"] = mandante
    df["VISITANTE_FIX"] = visitante

    # data normalizada
    df["DATA_FIX"] = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True)
    if df["DATA_FIX"].isna().all():
        raise ValueError("A coluna de DATA existe, mas não consegui converter para datetime. Verifique o formato.")

    df["MATCH_ID"] = df["DATA_FIX"].dt.strftime("%Y-%m-%d") + "|" + df["MANDANTE_FIX"] + "|" + df["VISITANTE_FIX"]
    return df

def safe_num(df, col):
    if col not in df.columns:
        return pd.Series(0, index=df.index)
    return pd.to_numeric(df[col], errors="coerce").fillna(0)

# --------------------------
# 1) Carregar parâmetros
# --------------------------
p = read_piloto(PILOTO_PATH)
RODADA_ATUAL = p["RODADA_ATUAL"]
N_JOGOS      = p["N_JOGOS"]
FILTRO       = p["FILTRO"]

print("✅ PILOTO:", p)

# --------------------------
# 2) Confrontos e data-corte
# --------------------------
df_confr = load_confrontos()

df_r = df_confr[df_confr["rodada"]==RODADA_ATUAL].copy()
if df_r.empty:
    raise ValueError(f"Não encontrei confrontos para a rodada {RODADA_ATUAL}.")

# data-corte da rodada (primeira data do txt). Se vier None, não corta por data.
data_ref = df_r["data_ref"].dropna().iloc[0] if "data_ref" in df_r.columns and df_r["data_ref"].notna().any() else None
print("✅ Confrontos na rodada:", len(df_r))
print("✅ Data-corte (rodada):", data_ref)

# --------------------------
# 3) Ler POR JOGO
# --------------------------
df_pj = pd.read_excel(SCOUTS_XLSX, sheet_name="Por jogo")

# detectar colunas essenciais
cols_norm = {norm_txt(c): c for c in df_pj.columns}
def must(col_upper):
    if col_upper not in cols_norm:
        raise ValueError(f"Não achei a coluna '{col_upper}' na aba 'Por jogo'. Colunas detectadas: {list(df_pj.columns)[:20]} ...")
    return cols_norm[col_upper]

TIME_COL  = must("TIME")
ADV_COL   = must("ADVERSÁRIO") if "ADVERSÁRIO" in cols_norm else must("ADVERSARIO")
MANDO_COL = must("MANDO")
NOME_COL  = must("NOME")

DATE_COL = detect_date_col(df_pj)

df_pj = build_match_id(df_pj, DATE_COL, TIME_COL, ADV_COL, MANDO_COL)

# --------------------------
# 4) Ler divisão MEIA/VOLANTE
# --------------------------
df_div = pd.read_csv(DIV_PATH, encoding="utf-8-sig")
div_cols = {norm_txt(c): c for c in df_div.columns}

# esperado: TIME / JOGADOR / GRUPO (MEIA|VOLANTE)
TIME_DIV = div_cols.get("TIME")
JOG_DIV  = div_cols.get("JOGADOR")
GRPGRP_DIV  = div_cols.get("POS_GRUPO") or div_cols.get("GRUPO")


if not all([TIME_DIV, JOG_DIV, GRP_DIV]):
    raise ValueError(f"DIVISÃO precisa ter colunas TIME, JOGADOR, GRUPO. Veio: {list(df_div.columns)}")

df_div["TIME_FIX"] = df_div[TIME_DIV].map(norm_txt)
df_div["NOME_FIX"] = df_div[JOG_DIV].map(norm_txt)
df_div["GRUPO_FIX"] = df_div[GRP_DIV].map(norm_txt)

# merge para marcar grupo
df_pj["TIME_FIX"] = df_pj[TIME_COL].map(norm_txt)
df_pj["NOME_FIX"] = df_pj[NOME_COL].map(norm_txt)

df_pj = df_pj.merge(df_div[["TIME_FIX","NOME_FIX","GRUPO_FIX"]], on=["TIME_FIX","NOME_FIX"], how="left")

# filtra só MEIAS
df_meias = df_pj[df_pj["GRUPO_FIX"]=="MEIA"].copy()

print("✅ Linhas (MEIAS) na base Por jogo:", len(df_meias))

# --------------------------
# 5) Definir métricas base (MEIAS)
# --------------------------
# AF: por enquanto 0 (será implementado via aba Scouts na próxima etapa)
df_meias["AF"] = 0

# PG: G + A (se existirem)
G = safe_num(df_meias, cols_norm.get("G","G"))
A = safe_num(df_meias, cols_norm.get("A","A"))
df_meias["PG"] = G + A

# CHUTES: FS + FF + FT (se existirem)
FS = safe_num(df_meias, cols_norm.get("FS","FS"))
FF = safe_num(df_meias, cols_norm.get("FF","FF"))
FT = safe_num(df_meias, cols_norm.get("FT","FT"))
df_meias["CHUTES"] = FS + FF + FT

# MÉDIA BÁSICA: coluna geralmente "BÁSICA" ou "BASICA"
bas_col = None
for c in df_meias.columns:
    if norm_txt(c) in ["BASICA","BÁSICA","BÁSICA "]:
        bas_col = c
        break
df_meias["BASICA"] = safe_num(df_meias, bas_col) if bas_col else 0

# --------------------------
# 6) Agregar por match_id/time (conquistados)
# --------------------------
team_match = (
    df_meias
    .groupby(["MATCH_ID","TIME_FIX"], as_index=False)
    .agg(
        DATA=("DATA_FIX","first"),
        MANDO=("MANDO","first"),
        AF=("AF","sum"),
        CHUTES=("CHUTES","sum"),
        PG=("PG","sum"),
        BASICA_MEDIA=("BASICA","mean"),
    )
)

# derivar CASA/FORA normalizado
team_match["MANDO_FIX"] = team_match["MANDO"].map(norm_txt)

# cedidos por time no match = conquistados do adversário no mesmo MATCH_ID
# cria tabela "swap" por MATCH_ID
swap = team_match[["MATCH_ID","TIME_FIX","AF","CHUTES","PG","BASICA_MEDIA"]].copy()
swap = swap.rename(columns={
    "TIME_FIX":"OPONENTE_FIX",
    "AF":"AF_OP",
    "CHUTES":"CHUTES_OP",
    "PG":"PG_OP",
    "BASICA_MEDIA":"BASICA_MEDIA_OP"
})

# descobrir oponente por match_id:
# usando o MATCH_ID: data|mandante|visitante
def get_opponent(match_id, team):
    parts = match_id.split("|")
    mand = parts[1]
    vis  = parts[2]
    if team == mand: return vis
    if team == vis:  return mand
    return None

team_match["OPONENTE_FIX"] = team_match.apply(lambda r: get_opponent(r["MATCH_ID"], r["TIME_FIX"]), axis=1)

team_match = team_match.merge(swap, on=["MATCH_ID","OPONENTE_FIX"], how="left")

# CEDIDOS = métricas do adversário (AF_OP, CHUTES_OP, PG_OP, BASICA_MEDIA_OP)
team_match["AF_CED"] = team_match["AF_OP"].fillna(0)
team_match["CHUTES_CED"] = team_match["CHUTES_OP"].fillna(0)
team_match["PG_CED"] = team_match["PG_OP"].fillna(0)
team_match["BASICA_MEDIA_CED"] = team_match["BASICA_MEDIA_OP"].fillna(0)

# --------------------------
# 7) Função: janela N_JOGOS por DATA e (opcional) mando
# --------------------------
def window_metrics(df_team, cutoff_date, n, modo, mando_req=None):
    df = df_team.copy()
    if cutoff_date is not None:
        df = df[df["DATA"].dt.date < cutoff_date]

    if modo == "POR_MANDO":
        df = df[df["MANDO_FIX"] == mando_req]

    df = df.sort_values("DATA")
    if n > 0:
        df = df.tail(n)

    out = {
        "N_ENCONTRADO": int(df["MATCH_ID"].nunique()),
        "AF": float(df["AF"].sum()) if len(df) else 0.0,
        "CHUTES": float(df["CHUTES"].sum()) if len(df) else 0.0,
        "PG": float(df["PG"].sum()) if len(df) else 0.0,
        "BASICA_MEDIA": float(df["BASICA_MEDIA"].mean()) if len(df) else 0.0,

        "AF_CED": float(df["AF_CED"].sum()) if len(df) else 0.0,
        "CHUTES_CED": float(df["CHUTES_CED"].sum()) if len(df) else 0.0,
        "PG_CED": float(df["PG_CED"].sum()) if len(df) else 0.0,
        "BASICA_MEDIA_CED": float(df["BASICA_MEDIA_CED"].mean()) if len(df) else 0.0,
    }
    return out

# index por time
by_team = {t: g.copy() for t, g in team_match.groupby("TIME_FIX")}

# --------------------------
# 8) Montar tabela final por confronto
# --------------------------
rows = []
for _, r in df_r.iterrows():
    casa = r["mandante"]
    fora = r["visitante"]

    # mando do confronto
    mando_casa = "CASA"
    mando_fora = "FORA"

    df_casa = by_team.get(casa, pd.DataFrame(columns=team_match.columns))
    df_fora = by_team.get(fora, pd.DataFrame(columns=team_match.columns))

    met_casa = window_metrics(df_casa, data_ref, N_JOGOS, FILTRO, mando_req=mando_casa)
    met_fora = window_metrics(df_fora, data_ref, N_JOGOS, FILTRO, mando_req=mando_fora)

    rows.append({
        "RODADA_REF": RODADA_ATUAL,
        "CASA": casa,
        "FORA": fora,
        "FILTRO": FILTRO,
        "N_JOGOS_SOL": N_JOGOS,
        "DATA_CORTE": str(data_ref) if data_ref else "",

        # CASA (COC)
        "CASA_COC_AF": met_casa["AF"],
        "CASA_COC_CHUTES": met_casa["CHUTES"],
        "CASA_COC_PG": met_casa["PG"],
        "CASA_COC_BASICA_MEDIA": met_casa["BASICA_MEDIA"],

        # CASA (CDF) = cedidos pelo CASA (isto é, conquistados do adversário nos jogos do CASA)
        "CASA_CDF_AF": met_casa["AF_CED"],
        "CASA_CDF_CHUTES": met_casa["CHUTES_CED"],
        "CASA_CDF_PG": met_casa["PG_CED"],
        "CASA_CDF_BASICA_MEDIA": met_casa["BASICA_MEDIA_CED"],

        # FORA (COC)
        "FORA_COC_AF": met_fora["AF"],
        "FORA_COC_CHUTES": met_fora["CHUTES"],
        "FORA_COC_PG": met_fora["PG"],
        "FORA_COC_BASICA_MEDIA": met_fora["BASICA_MEDIA"],

        # FORA (CDF)
        "FORA_CDF_AF": met_fora["AF_CED"],
        "FORA_CDF_CHUTES": met_fora["CHUTES_CED"],
        "FORA_CDF_PG": met_fora["PG_CED"],
        "FORA_CDF_BASICA_MEDIA": met_fora["BASICA_MEDIA_CED"],
    })

df_out = pd.DataFrame(rows)

# --------------------------
# 9) Export
# --------------------------
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = OUT/f"RUN_2026_RO{RODADA_ATUAL:02d}_{ts}"
run_dir.mkdir(parents=True, exist_ok=True)

out_file = run_dir/f"TABELA_MEIAS_R{RODADA_ATUAL:02d}.csv"
df_out.to_csv(out_file, index=False, encoding="utf-8-sig")

print("✅ Gerado:", out_file)
display(df_out.head(10))


In [ ]:
# ============================
# TCC - MOTOR v0 (PIPELINE)
# Um bloco único, para rodar do zero
# ============================

import os, re
from pathlib import Path
from datetime import datetime
import pandas as pd

# 1) MONTAR DRIVE
from google.colab import drive
drive.mount("/content/drive")

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE / "01_INPUT"
OUT  = BASE / "02_OUTPUT"
CFG  = BASE / "03_CONFIG"

print("\n🔹 Verificando estrutura de pastas...")
for p in [INP, OUT, CFG]:
    print(f"{p} -> {'OK' if p.exists() else '❌ NÃO ENCONTRADA'}")
    if not p.exists():
        raise FileNotFoundError(f"Pasta ausente: {p}")

# 2) CHECAR ARQUIVOS
SCOUTS_XLSX = INP / "Scouts_Reorganizado.xlsx"
DIVISAO_CSV = INP / "DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"
CONFR_TXT   = INP / "RODADAS_2026_CONFRONTOS.txt"
PILOTO_CSV  = CFG / "PILOTO_GERACAO_TABELAS.csv"

print("\n🔹 Verificando arquivos de INPUT...")
for f in [SCOUTS_XLSX, DIVISAO_CSV, CONFR_TXT, PILOTO_CSV]:
    print(f"{f.name} -> {'OK' if f.exists() else '❌ NÃO ENCONTRADO'}")
    if not f.exists():
        raise FileNotFoundError(f"Arquivo ausente: {f}")

# 3) FUNÇÕES UTILITÁRIAS
def norm_txt(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    s = s.replace("\u00a0", " ")  # nbsp
    s = re.sub(r"\s+", " ", s)
    return s.strip()

def norm_team(s: str) -> str:
    s = norm_txt(s).upper()
    # não mexe em acento aqui; apenas normaliza espaços e caixa
    return s

def read_piloto(path: Path) -> dict:
    """
    Lê PILOTO_GERACAO_TABELAS.csv aceitando 2 formatos:
    A) colunas CHAVE/VALOR
    B) 2 colunas genéricas (ex.: TEMPORADA,2026) sem cabeçalho correto
    Também remove BOM (\\ufeff) caso exista.
    """
    # tenta ler com utf-8-sig para remover BOM
    df = pd.read_csv(path, header=0, encoding="utf-8-sig")

    # caso venha como colunas tipo ['TEMPORADA','2026'] (sem CHAVE/VALOR)
    if df.shape[1] == 2 and set(df.columns) != {"CHAVE", "VALOR"}:
        df.columns = ["CHAVE", "VALOR"]
    else:
        # normaliza nomes de colunas (remove BOM em CHAVE, etc.)
        cols = [c.replace("\ufeff", "").strip().upper() for c in df.columns]
        df.columns = cols

        # se ainda não for CHAVE/VALOR mas tiver 2 colunas, força
        if df.shape[1] == 2 and set(df.columns) != {"CHAVE", "VALOR"}:
            df.columns = ["CHAVE", "VALOR"]

    # valida
    if set(df.columns) != {"CHAVE", "VALOR"}:
        raise ValueError(f"PILOTO inválido. Precisa ter colunas CHAVE/VALOR. Veio: {list(df.columns)}")

    # normaliza conteúdo
    df["CHAVE"] = df["CHAVE"].astype(str).map(lambda x: norm_txt(x).upper())
    df["VALOR"] = df["VALOR"].astype(str).map(lambda x: norm_txt(x).upper())

    # transforma em dict
    d = {row["CHAVE"]: row["VALOR"] for _, row in df.iterrows()}

    # defaults e conversões
    temporada = int(d.get("TEMPORADA", "2026"))
    rodada    = int(d.get("RODADA_ATUAL", "1"))
    n_jogos   = int(d.get("N_JOGOS", "0"))
    filtro    = d.get("FILTRO", "TODAS").upper()
    gerar     = d.get("GERAR_TABELAS", "SIM").upper()

    if filtro not in ["TODAS", "POR_MANDO"]:
        raise ValueError(f"❌ FILTRO inválido: {filtro}. Use TODAS ou POR_MANDO.")

    # regra: N_JOGOS precisa ser < RODADA_ATUAL (para R1, N_JOGOS=0 é o correto)
    if n_jogos >= rodada:
        raise ValueError(f"❌ N_JOGOS ({n_jogos}) >= RODADA_ATUAL ({rodada}) → impossível no início. Ajuste o PILOTO.")

    if gerar not in ["SIM", "NAO", "NÃO"]:
        raise ValueError(f"❌ GERAR_TABELAS inválido: {gerar}. Use SIM ou NAO.")

    return {
        "TEMPORADA": temporada,
        "RODADA_ATUAL": rodada,
        "N_JOGOS": n_jogos,
        "FILTRO": filtro,
        "GERAR_TABELAS": "SIM" if gerar == "SIM" else "NAO",
    }

def parse_confrontos_txt(txt_path: Path) -> pd.DataFrame:
    """
    Parse do seu TXT no formato:
    Rodada 1 (...)
    Time A x Time B
    ...
    Retorna DF com colunas: rodada, mandante, visitante
    """
    lines = txt_path.read_text(encoding="utf-8").splitlines()
    rodada_atual = None
    rows = []

    # aceita "x" e "X", com espaços variados
    jogo_re = re.compile(r"^\s*(.+?)\s*[xX]\s*(.+?)\s*$")
    rodada_re = re.compile(r"^\s*Rodada\s+(\d+)\b", re.IGNORECASE)

    for ln in lines:
        ln = ln.strip()
        if not ln:
            continue

        mrod = rodada_re.search(ln)
        if mrod:
            rodada_atual = int(mrod.group(1))
            continue

        mj = jogo_re.match(ln)
        if mj and rodada_atual is not None:
            mand = norm_txt(mj.group(1))
            vis  = norm_txt(mj.group(2))
            rows.append({
                "rodada": rodada_atual,
                "mandante": mand,
                "visitante": vis
            })

    df = pd.DataFrame(rows)
    if df.empty:
        raise ValueError("Não consegui extrair jogos do TXT. Confirme se o padrão 'Time x Time' está intacto.")
    return df

def load_divisao(path: Path) -> pd.DataFrame:
    """
    Aceita colunas:
      TIME, JOGADOR, POS_GRUPO  (seu padrão atual)
    ou:
      TIME, JOGADOR, GRUPO
    Normaliza para TIME/JOGADOR/GRUPO.
    """
    df = pd.read_csv(path, encoding="utf-8")
    df.columns = [c.replace("\ufeff","").strip().upper() for c in df.columns]

    if "POS_GRUPO" in df.columns and "GRUPO" not in df.columns:
        df = df.rename(columns={"POS_GRUPO": "GRUPO"})

    required = {"TIME","JOGADOR","GRUPO"}
    if not required.issubset(set(df.columns)):
        raise ValueError(f"DIVISÃO precisa ter colunas TIME, JOGADOR, GRUPO (ou POS_GRUPO). Veio: {list(df.columns)}")

    df["TIME"]   = df["TIME"].astype(str).map(norm_team)
    df["JOGADOR"] = df["JOGADOR"].astype(str).map(lambda x: norm_txt(x).upper())
    df["GRUPO"]  = df["GRUPO"].astype(str).map(lambda x: norm_txt(x).upper())

    # mantém apenas MEIA e VOLANTE (se tiver ruído)
    df = df[df["GRUPO"].isin(["MEIA","VOLANTE"])].copy()
    return df

# 4) EXECUÇÃO
print("\n🔹 Lendo PILOTO...")
piloto = read_piloto(PILOTO_CSV)
print("✅ PILOTO:", piloto)

print("\n🔹 Lendo confrontos TXT e normalizando...")
df_con = parse_confrontos_txt(CONFR_TXT)
df_con["mandante"]  = df_con["mandante"].map(norm_team)
df_con["visitante"] = df_con["visitante"].map(norm_team)

print(f"✅ Total de jogos extraídos: {len(df_con)}")
print(f"✅ Rodadas detectadas: {df_con['rodada'].nunique()}")
print("✅ Jogos por rodada (primeiras 10):")
print(df_con.groupby("rodada").size().head(10))

# salva confrontos normalizado
con_out = OUT / "CONFRONTOS_2026_NORMALIZADO.csv"
df_con.to_csv(con_out, index=False, encoding="utf-8-sig")
print(f"\n📌 CSV normalizado salvo em: {con_out}")

print("\n🔹 Lendo divisão meias/volantes...")
df_div = load_divisao(DIVISAO_CSV)
print("✅ DIVISÃO OK. Linhas:", len(df_div), "| Colunas:", list(df_div.columns))

# 5) MOTOR SKELETON: gera arquivos por posição (infra ok)
if piloto["GERAR_TABELAS"] != "SIM":
    print("\n🟡 GERAR_TABELAS = NAO. Encerrando sem gerar outputs.")
    raise SystemExit

RODADA_ATUAL = piloto["RODADA_ATUAL"]
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = OUT / f"RUN_2026_R{RODADA_ATUAL:02d}_{ts}"
show_run = str(run_dir)
run_dir.mkdir(parents=True, exist_ok=True)

# confrontos da rodada atual (para 2026)
df_r = df_con[df_con["rodada"] == RODADA_ATUAL].copy()
print(f"\n✅ Confrontos encontrados na rodada {RODADA_ATUAL}: {len(df_r)}")

# salva uma cópia de confrontos da rodada dentro da RUN
(df_r.to_csv(run_dir / f"CONFRONTOS_R{RODADA_ATUAL:02d}.csv", index=False, encoding="utf-8-sig"))

# outputs skeleton por posição (por enquanto, só estrutura auditável)
pos_files = {
    "GOLEIROS":  run_dir / f"TABELA_GOLEIROS_R{RODADA_ATUAL:02d}.csv",
    "ZAGUEIROS": run_dir / f"TABELA_ZAGUEIROS_R{RODADA_ATUAL:02d}.csv",
    "LATERAIS":  run_dir / f"TABELA_LATERAIS_R{RODADA_ATUAL:02d}.csv",
    "MEIAS":     run_dir / f"TABELA_MEIAS_R{RODADA_ATUAL:02d}.csv",
    "ATACANTES": run_dir / f"TABELA_ATACANTES_R{RODADA_ATUAL:02d}.csv",
}

# Cria um “esqueleto” com metadados + lista de confrontos (ainda sem cálculo)
meta_rows = []
for _, row in df_r.iterrows():
    meta_rows.append({
        "RODADA_REF": RODADA_ATUAL,
        "FILTRO": piloto["FILTRO"],
        "N_JOGOS": piloto["N_JOGOS"],
        "MANDANTE": row["mandante"],
        "VISITANTE": row["visitante"],
    })

df_meta = pd.DataFrame(meta_rows)

for k, fp in pos_files.items():
    df_meta.to_csv(fp, index=False, encoding="utf-8-sig")
    print("✅ Gerado:", fp)

print("\n✅ FINALIZADO.")
print("📌 Saída em:", show_run)


In [ ]:
# =========================
# MOTOR v1 (ROBUSTO) - STARTER
# Cole esta célula inteira e rode (▶)
# =========================

import os, re
import pandas as pd
from pathlib import Path
from datetime import datetime

# ---------- 0) Drive ----------
from google.colab import drive
drive.mount("/content/drive", force_remount=False)

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

print("\n✅ Verificando estrutura de pastas...")
for p in [INP, OUT, CFG]:
    print(str(p), "->", "OK" if p.exists() else "❌ NÃO EXISTE")

# ---------- 1) Helpers ----------
def norm_txt(x):
    if pd.isna(x): return ""
    x = str(x).strip()
    x = re.sub(r"\s+", " ", x)
    return x

def norm_time(x):
    # padronização leve; sem inventar nomes
    x = norm_txt(x).upper()
    x = x.replace("ATHLETICO-PR", "ATHLETICO-PR")
    return x

def load_piloto_robusto(path_csv):
    """
    Aceita:
    - CSV com colunas CHAVE/VALOR (com ou sem BOM)
    - CSV "torto" vindo de download do Google Sheets
    """
    df = pd.read_csv(path_csv, dtype=str, encoding="utf-8-sig")
    df.columns = [c.strip() for c in df.columns]

    # Caso comum: CHAVE/VALOR (às vezes vem com BOM no nome)
    cols = [c.upper().replace("\ufeff","") for c in df.columns]
    df.columns = cols

    if set(["CHAVE","VALOR"]).issubset(set(df.columns)):
        piloto = df[["CHAVE","VALOR"]].copy()
    else:
        # Caso alternativo: 2 colunas sem header correto (ex: TEMPORADA,2026)
        # Se o df tem 2 colunas, transforma em CHAVE/VALOR
        if df.shape[1] == 2:
            piloto = df.copy()
            piloto.columns = ["CHAVE","VALOR"]
        else:
            raise ValueError(f"PILOTO inválido. Esperado CHAVE/VALOR ou 2 colunas. Veio: {df.columns.tolist()}")

    piloto["CHAVE"] = piloto["CHAVE"].astype(str).str.strip().str.upper()
    piloto["VALOR"] = piloto["VALOR"].astype(str).str.strip().str.upper()

    d = {r["CHAVE"]: r["VALOR"] for _, r in piloto.iterrows()}
    # validações mínimas
    for k in ["TEMPORADA","RODADA_ATUAL","N_JOGOS","FILTRO","GERAR_TABELAS"]:
        if k not in d: raise ValueError(f"PILOTO faltando chave: {k}")
    if d["FILTRO"] not in ["TODAS","POR_MANDO"]:
        raise ValueError("FILTRO deve ser TODAS ou POR_MANDO")
    # rodada/n_jogos numéricos
    d["RODADA_ATUAL"] = int(float(d["RODADA_ATUAL"]))
    d["N_JOGOS"]      = int(float(d["N_JOGOS"]))
    return d, piloto

def parse_confrontos_txt(path_txt):
    """
    Lê RODADAS_2026_CONFRONTOS.txt no formato:
    Rodada X (...)
    Time A x Time B
    ...
    """
    txt = Path(path_txt).read_text(encoding="utf-8", errors="ignore").splitlines()
    rodada_atual = None
    rows = []
    for line in txt:
        line = line.strip()
        if not line:
            continue
        m = re.match(r"^Rodada\s+(\d+)", line, flags=re.I)
        if m:
            rodada_atual = int(m.group(1))
            continue
        if " x " in line and rodada_atual is not None:
            mand, vis = [norm_txt(p) for p in line.split(" x ", 1)]
            rows.append({"rodada": rodada_atual, "mandante": mand, "visitante": vis})
    df = pd.DataFrame(rows)
    df["mandante"]  = df["mandante"].map(norm_time)
    df["visitante"] = df["visitante"].map(norm_time)
    return df

def load_divisao(path_csv):
    df = pd.read_csv(path_csv, dtype=str, encoding="utf-8-sig")
    df.columns = [c.strip().upper().replace("\ufeff","") for c in df.columns]

    # Aceita GRUPO ou POS_GRUPO
    if "GRUPO" not in df.columns and "POS_GRUPO" in df.columns:
        df = df.rename(columns={"POS_GRUPO":"GRUPO"})

    needed = {"TIME","JOGADOR","GRUPO"}
    if not needed.issubset(set(df.columns)):
        raise ValueError(f"DIVISÃO precisa ter colunas {needed}. Veio: {df.columns.tolist()}")

    df["TIME"]    = df["TIME"].map(norm_time)
    df["JOGADOR"] = df["JOGADOR"].map(lambda x: norm_txt(x).upper())
    df["GRUPO"]   = df["GRUPO"].map(lambda x: norm_txt(x).upper())
    return df

# ---------- 2) Checagem de arquivos ----------
print("\n✅ Verificando arquivos de INPUT...")
scouts_path = INP/"Scouts_Reorganizado.xlsx"
div_path    = INP/"DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"
con_path    = INP/"RODADAS_2026_CONFRONTOS.txt"
piloto_path = CFG/"PILOTO_GERACAO_TABELAS.csv"

for f in [scouts_path, div_path, con_path, piloto_path]:
    print(f.name, "->", "OK" if f.exists() else "❌ NÃO EXISTE")

# ---------- 3) PILOTO ----------
print("\n🔹 Lendo PILOTO...")
PILOTO, piloto_df = load_piloto_robusto(piloto_path)
print("✅ PILOTO:", PILOTO)

RODADA_ATUAL = PILOTO["RODADA_ATUAL"]
N_JOGOS      = PILOTO["N_JOGOS"]
FILTRO       = PILOTO["FILTRO"]
GERAR        = PILOTO["GERAR_TABELAS"]

# ---------- 4) Confrontos ----------
print("\n🔹 Lendo confrontos TXT e normalizando...")
df_con = parse_confrontos_txt(con_path)

# validações do arquivo
print("✅ Total de jogos extraídos:", len(df_con))
print("✅ Rodadas detectadas:", df_con["rodada"].nunique())
print("✅ Jogos por rodada (primeiras 10):")
print(df_con.groupby("rodada").size().head(10))

# salva normalizado
con_norm_path = OUT/"CONFRONTOS_2026_NORMALIZADO.csv"
df_con.to_csv(con_norm_path, index=False, encoding="utf-8-sig")
print("📌 Confrontos normalizados salvos em:", con_norm_path)

df_r = df_con[df_con["rodada"] == RODADA_ATUAL].copy()
print(f"\n✅ Confrontos encontrados na rodada {RODADA_ATUAL}: {len(df_r)}")

# ---------- 5) Scouts (Por jogo) ----------
print("\n🔹 Lendo Scouts_Reorganizado.xlsx (aba 'Por jogo')...")
df_pj = pd.read_excel(scouts_path, sheet_name="Por jogo")
df_pj.columns = [str(c).strip() for c in df_pj.columns]

# normalizações mínimas (sem inferir rodada)
for col in ["Time","Adversário","Mando","Nome","Posreal"]:
    if col in df_pj.columns:
        if col in ["Time","Adversário"]:
            df_pj[col] = df_pj[col].map(norm_time)
        else:
            df_pj[col] = df_pj[col].map(lambda x: norm_txt(x).upper())

# data
if "Data" in df_pj.columns:
    df_pj["Data"] = pd.to_datetime(df_pj["Data"], dayfirst=True, errors="coerce")

# ---------- 6) Divisão meias/volantes ----------
print("\n🔹 Lendo divisão meias/volantes...")
df_div = load_divisao(div_path)
print("✅ DIVISÃO OK. Linhas:", len(df_div), "| Colunas:", df_div.columns.tolist())

# ---------- 7) Auditoria de times (2025 x 2026) ----------
times_base = set(df_pj["Time"].dropna().unique()) if "Time" in df_pj.columns else set()
times_txt  = set(df_con["mandante"].unique()) | set(df_con["visitante"].unique())

ausentes = sorted(list(times_txt - times_base))
print("\n🔎 Times detectados na base (Scouts):", len(times_base))
print("🔎 Times no TXT 2026:", len(times_txt))
if ausentes:
    print("⚠️ Times presentes no TXT 2026 mas AUSENTES no Scouts (esperado usando base 2025):")
    for t in ausentes:
        print(" -", t)
else:
    print("✅ Nenhum time ausente (perfeito).")

# ---------- 8) Gerar tabelas (skeleton seguro) ----------
if GERAR != "SIM":
    print("\n⏭️ GERAR_TABELAS != SIM. Encerrando.")
    raise SystemExit()

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = OUT / f"RUN_2026_R{RODADA_ATUAL:02d}_{ts}"
run_dir.mkdir(parents=True, exist_ok=True)

def tabela_vazia(nome):
    out = pd.DataFrame([{
        "RODADA_REF": RODADA_ATUAL,
        "FILTRO": FILTRO,
        "N_JOGOS_SOLICITADO": N_JOGOS,
        "OBS": "Base 2025 x Confrontos 2026: tabelas-skeleton (sem cálculo de mérito ainda)."
    }])
    out_file = run_dir / f"{nome}_R{RODADA_ATUAL:02d}.csv"
    out.to_csv(out_file, index=False, encoding="utf-8-sig")
    print("✅ Gerado:", out_file)

# por enquanto: skeleton (estrutura OK). Próximo passo será cálculo real.
tabela_vazia("TABELA_GOLEIROS")
tabela_vazia("TABELA_ZAGUEIROS")
tabela_vazia("TABELA_LATERAIS")
tabela_vazia("TABELA_MEIAS")
tabela_vazia("TABELA_ATACANTES")

print("\n✅ FINALIZADO.")
print("📌 Saída em:", run_dir)


In [ ]:
# ==========================
# TESTE COMPLETO - MEIAS (2025)
# Por DATA_CORTE + N_JOGOS + TODAS/POR_MANDO + auditoria
# ==========================

import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np

# --------- Paths ----------
BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

PILOTO_PATH = CFG/"PILOTO_GERACAO_TABELAS.csv"
SCOUTS_PATH = INP/"Scouts_Reorganizado.xlsx"
DIVISAO_PATH = INP/"DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"

# --------- Helpers ----------
def norm_txt(x):
    if pd.isna(x):
        return ""
    return str(x).strip().upper()

def load_piloto(p):
    df = pd.read_csv(p, dtype=str, encoding="utf-8-sig")
    # blindagem BOM (ufeff)
    df.columns = [c.replace("\ufeff","").strip().upper() for c in df.columns]

    # Se veio no formato "duas colunas" erradas, tenta detectar (mas seu já está ok)
    if set(df.columns) >= {"CHAVE","VALOR"}:
        df["CHAVE"] = df["CHAVE"].map(norm_txt)
        df["VALOR"] = df["VALOR"].astype(str).str.strip()
        d = dict(zip(df["CHAVE"], df["VALOR"]))
        return d, df

    raise ValueError(f"PILOTO inválido. Precisa ter colunas CHAVE e VALOR. Veio: {list(df.columns)}")

def pick_col(df_cols, candidates):
    # retorna a primeira coluna existente na lista candidates (case-insensitive)
    upper_map = {c.upper(): c for c in df_cols}
    for cand in candidates:
        if cand.upper() in upper_map:
            return upper_map[cand.upper()]
    return None

# --------- Mount Drive (se necessário) ----------
try:
    from google.colab import drive
    if not Path("/content/drive").exists():
        drive.mount("/content/drive")
except Exception:
    pass

# --------- Checks ----------
for p in [INP, OUT, CFG]:
    assert p.exists(), f"Pasta ausente: {p}"

for f in [PILOTO_PATH, SCOUTS_PATH, DIVISAO_PATH]:
    assert f.exists(), f"Arquivo ausente: {f}"

print("✅ Estrutura OK")

# --------- Load PILOTO ----------
piloto, df_piloto = load_piloto(PILOTO_PATH)

DATA_CORTE = piloto.get("DATA_CORTE", "").strip()
N_TESTE = int(piloto.get("N_JOGOS_TESTE", "5"))
TIME_AUD = norm_txt(piloto.get("TIME_AUDITORIA", "FLAMENGO"))

if not DATA_CORTE:
    raise ValueError("❌ Falta DATA_CORTE no PILOTO. Ex.: 2025-08-10")

try:
    data_corte_dt = pd.to_datetime(DATA_CORTE, format="%Y-%m-%d")
except Exception:
    raise ValueError("❌ DATA_CORTE inválida. Use formato YYYY-MM-DD (ex.: 2025-08-10)")

print(f"✅ PILOTO: DATA_CORTE={DATA_CORTE} | N_JOGOS_TESTE={N_TESTE} | TIME_AUDITORIA={TIME_AUD}")

# --------- Load DIVISÃO ----------
df_div = pd.read_csv(DIVISAO_PATH, dtype=str, encoding="utf-8-sig")
df_div.columns = [c.replace("\ufeff","").strip().upper() for c in df_div.columns]

# contrato aceito: TIME, JOGADOR, GRUPO
required = {"TIME","JOGADOR","GRUPO"}
if not required.issubset(set(df_div.columns)):
    raise ValueError(f"❌ DIVISÃO precisa ter colunas TIME, JOGADOR, GRUPO. Veio: {list(df_div.columns)}")

df_div["TIME_FIX"] = df_div["TIME"].map(norm_txt)
df_div["JOG_FIX"]  = df_div["JOGADOR"].map(norm_txt)
df_div["GRUPO"]    = df_div["GRUPO"].map(norm_txt)

# pega só MEIA (você já normalizou para MEIA/VOLANTE)
df_div_meia = df_div[df_div["GRUPO"]=="MEIA"].copy()
print(f"✅ DIVISÃO carregada: {len(df_div)} linhas | MEIAS={len(df_div_meia)}")

# --------- Load SCOUTS (aba Por jogo) ----------
xls = pd.ExcelFile(SCOUTS_PATH)
sheet_name = None
for s in xls.sheet_names:
    if norm_txt(s) in ["POR JOGO", "POR_JOGO", "PORJOGO"]:
        sheet_name = s
        break
if sheet_name is None:
    # tenta achar parcialmente
    for s in xls.sheet_names:
        if "JOGO" in norm_txt(s):
            sheet_name = s
            break
if sheet_name is None:
    raise ValueError(f"❌ Não achei a aba 'Por jogo'. Abas disponíveis: {xls.sheet_names}")

df = pd.read_excel(SCOUTS_PATH, sheet_name=sheet_name)
df.columns = [str(c).strip() for c in df.columns]

# localizar colunas essenciais na base
COL_TIME = pick_col(df.columns, ["TIME", "CLUBE"])
COL_NOME = pick_col(df.columns, ["NOME", "JOGADOR"])
COL_MANDO = pick_col(df.columns, ["MANDO"])
COL_DATA = pick_col(df.columns, ["DATA", "DATE"])

if not all([COL_TIME, COL_NOME, COL_MANDO, COL_DATA]):
    raise ValueError(f"❌ Base Por jogo precisa ter TIME/NOME/MANDO/DATA. Detectei: TIME={COL_TIME}, NOME={COL_NOME}, MANDO={COL_MANDO}, DATA={COL_DATA}")

# normalizações
df["TIME_FIX"] = df[COL_TIME].map(norm_txt)
df["JOG_FIX"]  = df[COL_NOME].map(norm_txt)
df["MANDO_FIX"] = df[COL_MANDO].map(norm_txt)
df["DATA_FIX"] = pd.to_datetime(df[COL_DATA], errors="coerce", dayfirst=True)

df = df[df["DATA_FIX"].notna()].copy()

# recorte temporal (antes da data_corte)
df = df[df["DATA_FIX"] < data_corte_dt].copy()

# tenta padronizar mando para CASA/FORA
# (se vier como "C"/"F" ou "CASA"/"FORA", normaliza)
def norm_mando(x):
    x = norm_txt(x)
    if x in ["C", "CASA", "HOME"]: return "CASA"
    if x in ["F", "FORA", "AWAY"]: return "FORA"
    return x
df["MANDO_FIX"] = df["MANDO_FIX"].map(norm_mando)

print(f"✅ Scouts carregado ({sheet_name}): linhas antes da DATA_CORTE = {len(df)}")

# --------- Colunas numéricas relevantes (mínimo para MEIAS) ----------
# Vamos procurar colunas por nome:
# AF (assistência para finalização) pode estar como "AF" ou "A.F." etc.
# Chutes: você pode querer "FF" e/ou "FS". Vamos exportar ambos se existirem.
# PG: "PG"
# Média Básica: "BÁSICA" (ou BASICA) / "Básica"
# Pontos: "PTS" ou "PONTOS" se existir
cand_cols = {
    "AF": ["AF"],
    "PG": ["PG"],
    "FF": ["FF"],
    "FS": ["FS"],
    "BASICA": ["BÁSICA", "BASICA", "BÁSICA ", "BASICA "],
    "PTS": ["PTS", "PONTOS", "PONTUAÇÃO", "PONTUACAO"]
}

col_map = {}
for k, cands in cand_cols.items():
    col_map[k] = pick_col(df.columns, cands)

# converte numéricas (se existirem)
for k, c in col_map.items():
    if c:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

print("✅ Colunas detectadas:", {k:v for k,v in col_map.items() if v})

# --------- Join para filtrar apenas jogadores MEIA ----------
# chave (TIME_FIX + JOG_FIX)
df_meia = df.merge(
    df_div_meia[["TIME_FIX","JOG_FIX","GRUPO"]],
    on=["TIME_FIX","JOG_FIX"],
    how="inner"
)

print(f"✅ Linhas após filtrar MEIAS pela divisão: {len(df_meia)}")

# --------- Função: últimos N jogos do time (TODAS ou POR_MANDO) ----------
def last_n_games(df_base, time_fix, n, modo, mando=None):
    dft = df_base[df_base["TIME_FIX"]==time_fix].copy()
    if modo == "POR_MANDO":
        if mando not in ["CASA","FORA"]:
            return dft.iloc[0:0].copy()
        dft = dft[dft["MANDO_FIX"]==mando].copy()
    dft = dft.sort_values("DATA_FIX")
    # últimos N jogos (por data)
    # Observação: aqui cada linha é jogador-jogo. Precisamos pegar os últimos N jogos do TIME,
    # então pegamos as últimas N datas únicas e mantemos todas as linhas dessas datas.
    datas = sorted(dft["DATA_FIX"].unique())
    datas_sel = datas[-n:] if n > 0 else []
    dft = dft[dft["DATA_FIX"].isin(datas_sel)].copy()
    return dft

# --------- Agregação MEIAS por time ----------
def agg_metrics(df_slice):
    out = {}
    # soma
    if col_map["AF"]: out["AF"] = float(df_slice[col_map["AF"]].sum())
    else: out["AF"] = np.nan
    # chutes: exporta FF e FS se existirem
    if col_map["FF"]: out["CHUTES_FF"] = float(df_slice[col_map["FF"]].sum())
    else: out["CHUTES_FF"] = np.nan
    if col_map["FS"]: out["CHUTES_FS"] = float(df_slice[col_map["FS"]].sum())
    else: out["CHUTES_FS"] = np.nan

    if col_map["PG"]: out["PG"] = float(df_slice[col_map["PG"]].sum())
    else: out["PG"] = np.nan

    if col_map["BASICA"]: out["MEDIA_BASICA"] = float(df_slice[col_map["BASICA"]].mean()) if len(df_slice) else 0.0
    else: out["MEDIA_BASICA"] = np.nan

    if col_map["PTS"]: out["PTS_TOTAL"] = float(df_slice[col_map["PTS"]].sum())
    else: out["PTS_TOTAL"] = np.nan

    # jogos efetivos (datas únicas)
    out["JOGOS_ENCONTRADOS"] = int(df_slice["DATA_FIX"].nunique())
    return out

times = sorted(df_meia["TIME_FIX"].unique())

# --------- Rodar 3 cenários: TODAS, CASA, FORA ----------
rows = []

for t in times:
    # TODAS
    sl = last_n_games(df_meia, t, N_TESTE, "TODAS")
    m = agg_metrics(sl)
    rows.append({
        "TIME": t,
        "MODO": "TODAS",
        "MANDO": "TODAS",
        "N_JOGOS_SOLICITADO": N_TESTE,
        **m
    })
    # CASA
    sl = last_n_games(df_meia, t, N_TESTE, "POR_MANDO", "CASA")
    m = agg_metrics(sl)
    rows.append({
        "TIME": t,
        "MODO": "POR_MANDO",
        "MANDO": "CASA",
        "N_JOGOS_SOLICITADO": N_TESTE,
        **m
    })
    # FORA
    sl = last_n_games(df_meia, t, N_TESTE, "POR_MANDO", "FORA")
    m = agg_metrics(sl)
    rows.append({
        "TIME": t,
        "MODO": "POR_MANDO",
        "MANDO": "FORA",
        "N_JOGOS_SOLICITADO": N_TESTE,
        **m
    })

df_out = pd.DataFrame(rows)

# --------- Auditoria detalhada de 1 time ----------
aud = []

for modo, mando in [("TODAS","TODAS"), ("POR_MANDO","CASA"), ("POR_MANDO","FORA")]:
    if modo == "TODAS":
        sl = last_n_games(df_meia, TIME_AUD, N_TESTE, "TODAS")
    else:
        sl = last_n_games(df_meia, TIME_AUD, N_TESTE, "POR_MANDO", mando)

    # listar datas únicas selecionadas e adversários
    # (adversário pode existir ou não; tentamos detectar)
    COL_ADV = pick_col(df.columns, ["ADVERSÁRIO", "ADVERSARIO", "OPONENTE"])
    if COL_ADV:
        sl["ADV_FIX"] = sl[COL_ADV].map(norm_txt)
        adv_list = ", ".join(sorted(sl["ADV_FIX"].unique()))
    else:
        adv_list = ""

    datas = ", ".join([d.strftime("%Y-%m-%d") for d in sorted(sl["DATA_FIX"].unique())])
    met = agg_metrics(sl)

    aud.append({
        "TIME": TIME_AUD,
        "CENARIO": f"{modo}-{mando}",
        "DATAS_USADAS": datas,
        "ADVERSARIOS": adv_list,
        **met
    })

df_aud = pd.DataFrame(aud)

# --------- Export versionado ----------
ts = datetime.now().strftime("%Y%m%d_%H%M%S")

# usa exatamente as chaves do PILOTO
run_dir = OUT / f"TESTE_MEIAS_2025_{DATA_CORTE}_N{N_JOGOS}_{ts}"
run_dir.mkdir(parents=True, exist_ok=True)

out_main = run_dir / "MEIAS_METRICAS_POR_TIME.csv"
out_aud  = run_dir / f"AUDITORIA_{TIME_AUDITORIA}.csv"

display(df_aud)
display(df_out.head(10))


In [ ]:
# ==========================
# TESTE COMPLETO - MEIAS (2025) - CÉLULA ÚNICA BLINDADA
# DATA_CORTE + N_JOGOS_TESTE + (TODAS / POR_MANDO CASA/FORA) + AUDITORIA
# ==========================

import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np

# --------- Paths ----------
BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

PILOTO_PATH = CFG/"PILOTO_GERACAO_TABELAS.csv"
SCOUTS_PATH = INP/"Scouts_Reorganizado.xlsx"
DIVISAO_PATH = INP/"DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"

# --------- Mount Drive ----------
from google.colab import drive
if not Path("/content/drive").exists():
    drive.mount("/content/drive")
elif not Path("/content/drive/MyDrive").exists():
    drive.mount("/content/drive", force_remount=True)

# --------- Helpers ----------
def norm_txt(x):
    if pd.isna(x):
        return ""
    return str(x).strip().upper()

def strip_bom(s: str) -> str:
    return s.replace("\ufeff","").strip()

def pick_col(cols, candidates):
    up = {strip_bom(str(c)).upper(): str(c) for c in cols}
    for cand in candidates:
        k = strip_bom(str(cand)).upper()
        if k in up:
            return up[k]
    return None

def load_piloto_any(p):
    df = pd.read_csv(p, dtype=str, encoding="utf-8-sig")
    df.columns = [strip_bom(c).upper() for c in df.columns]

    # Caso 1: já veio certo
    if set(df.columns) >= {"CHAVE","VALOR"}:
        df["CHAVE"] = df["CHAVE"].map(norm_txt)
        df["VALOR"] = df["VALOR"].astype(str).str.strip()
        d = dict(zip(df["CHAVE"], df["VALOR"]))
        return d, df[["CHAVE","VALOR"]].copy()

    # Caso 2: veio “tabela” do Sheets com 2 colunas, ex: A=DATA_CORTE, B=2025-08-10
    if len(df.columns) == 2:
        c0, c1 = df.columns[0], df.columns[1]
        # Se primeira coluna parece ser "DATA_CORTE" etc
        df2 = df[[c0, c1]].copy()
        df2.columns = ["CHAVE","VALOR"]
        df2["CHAVE"] = df2["CHAVE"].map(norm_txt)
        df2["VALOR"] = df2["VALOR"].astype(str).str.strip()
        d = dict(zip(df2["CHAVE"], df2["VALOR"]))
        return d, df2

    raise ValueError(f"❌ PILOTO inválido. Precisa ser CHAVE/VALOR ou 2 colunas. Veio: {list(df.columns)}")

def load_divisao_any(p):
    df = pd.read_csv(p, dtype=str, encoding="utf-8-sig")
    df.columns = [strip_bom(c).upper() for c in df.columns]

    col_time = pick_col(df.columns, ["TIME","CLUBE"])
    col_jog  = pick_col(df.columns, ["JOGADOR","NOME"])
    col_grp  = pick_col(df.columns, ["GRUPO","POS_GRUPO","POSICAO_GRUPO","GRUPO_POS","GRUPO_POSICAO"])

    if not all([col_time, col_jog, col_grp]):
        raise ValueError(f"❌ DIVISÃO precisa ter TIME, JOGADOR e GRUPO (ou POS_GRUPO). Veio: {list(df.columns)}")

    out = df[[col_time, col_jog, col_grp]].copy()
    out.columns = ["TIME","JOGADOR","GRUPO"]
    out["TIME_FIX"] = out["TIME"].map(norm_txt)
    out["JOG_FIX"]  = out["JOGADOR"].map(norm_txt)
    out["GRUPO"]    = out["GRUPO"].map(norm_txt)

    # Normaliza possíveis valores
    out["GRUPO"] = out["GRUPO"].replace({
        "MEIAS":"MEIA",
        "VOLANTES":"VOLANTE",
        "VOL":"VOLANTE",
        "MEI":"MEIA",
    })

    return out

def detect_por_jogo_sheet(xls: pd.ExcelFile):
    # tenta match exato
    for s in xls.sheet_names:
        if norm_txt(s) in ["POR JOGO","POR_JOGO","PORJOGO"]:
            return s
    # tenta parcial
    for s in xls.sheet_names:
        if "JOGO" in norm_txt(s):
            return s
    return None

def norm_mando(x):
    x = norm_txt(x)
    if x in ["C","CASA","HOME","MANDANTE"]: return "CASA"
    if x in ["F","FORA","AWAY","VISITANTE"]: return "FORA"
    return x

# --------- Checks ----------
for p in [INP, OUT, CFG]:
    if not p.exists():
        raise ValueError(f"❌ Pasta ausente: {p}")

for f in [PILOTO_PATH, SCOUTS_PATH, DIVISAO_PATH]:
    if not f.exists():
        raise ValueError(f"❌ Arquivo ausente: {f}")

print("✅ Estrutura OK")
print("📌 PILOTO:", PILOTO_PATH)
print("📌 SCOUTS:", SCOUTS_PATH)
print("📌 DIVISÃO:", DIVISAO_PATH)

# --------- Load PILOTO (blindado) ----------
piloto, df_piloto = load_piloto_any(PILOTO_PATH)

DATA_CORTE = piloto.get("DATA_CORTE","").strip()
N_JOGOS_TESTE = int(piloto.get("N_JOGOS_TESTE", piloto.get("N_JOGOS","5")).strip() if piloto.get("N_JOGOS_TESTE") or piloto.get("N_JOGOS") else "5")
TIME_AUDITORIA = norm_txt(piloto.get("TIME_AUDITORIA","FLAMENGO"))
FILTRO = norm_txt(piloto.get("FILTRO","TODAS"))  # pode existir, mas o teste sempre roda os 3 cenários

if not DATA_CORTE:
    raise ValueError("❌ Falta DATA_CORTE no PILOTO. Ex.: 2025-08-10")

try:
    data_corte_dt = pd.to_datetime(DATA_CORTE, format="%Y-%m-%d")
except Exception:
    raise ValueError("❌ DATA_CORTE inválida. Use formato YYYY-MM-DD (ex.: 2025-08-10)")

print(f"✅ PILOTO OK | DATA_CORTE={DATA_CORTE} | N_JOGOS_TESTE={N_JOGOS_TESTE} | TIME_AUDITORIA={TIME_AUDITORIA} | FILTRO={FILTRO}")
display(df_piloto)

# --------- Load DIVISÃO (blindado) ----------
df_div = load_divisao_any(DIVISAO_PATH)
df_div_meia = df_div[df_div["GRUPO"]=="MEIA"].copy()

print(f"✅ DIVISÃO OK | Linhas={len(df_div)} | MEIAS={len(df_div_meia)} | Colunas={list(df_div.columns)}")

# --------- Load SCOUTS (aba Por jogo) ----------
xls = pd.ExcelFile(SCOUTS_PATH)
sheet_name = detect_por_jogo_sheet(xls)
if sheet_name is None:
    raise ValueError(f"❌ Não achei a aba 'Por jogo'. Abas disponíveis: {xls.sheet_names}")

df = pd.read_excel(SCOUTS_PATH, sheet_name=sheet_name)
df.columns = [str(c).strip() for c in df.columns]

COL_TIME  = pick_col(df.columns, ["TIME","CLUBE"])
COL_NOME  = pick_col(df.columns, ["NOME","JOGADOR"])
COL_MANDO = pick_col(df.columns, ["MANDO"])
COL_DATA  = pick_col(df.columns, ["DATA","DATE"])

if not all([COL_TIME, COL_NOME, COL_MANDO, COL_DATA]):
    raise ValueError(f"❌ Base Por jogo precisa ter TIME/NOME/MANDO/DATA. Detectei: TIME={COL_TIME}, NOME={COL_NOME}, MANDO={COL_MANDO}, DATA={COL_DATA}")

df["TIME_FIX"]  = df[COL_TIME].map(norm_txt)
df["JOG_FIX"]   = df[COL_NOME].map(norm_txt)
df["MANDO_FIX"] = df[COL_MANDO].map(norm_mando)
df["DATA_FIX"]  = pd.to_datetime(df[COL_DATA], errors="coerce", dayfirst=True)

df = df[df["DATA_FIX"].notna()].copy()
df = df[df["DATA_FIX"] < data_corte_dt].copy()

print(f"✅ Scouts OK ({sheet_name}) | Linhas antes da DATA_CORTE: {len(df)}")
print(f"✅ Times na base: {df['TIME_FIX'].nunique()}")

# --------- Detect numeric columns (AF/PG/FF/FS/BÁSICA/PTS) ----------
cand_cols = {
    "AF":     ["AF"],
    "PG":     ["PG"],
    "FF":     ["FF"],
    "FS":     ["FS"],
    "BASICA": ["BÁSICA","BASICA","Básica","Basica"],
    "PTS":    ["PTS","PONTOS","PONTUAÇÃO","PONTUACAO","PONT."]
}
col_map = {k: pick_col(df.columns, v) for k, v in cand_cols.items()}

for k, c in col_map.items():
    if c:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

print("✅ Colunas detectadas:", {k:v for k,v in col_map.items() if v})

# --------- Filtra somente MEIAS via divisão (TIME_FIX+JOG_FIX) ----------
df_meia = df.merge(df_div_meia[["TIME_FIX","JOG_FIX","GRUPO"]], on=["TIME_FIX","JOG_FIX"], how="inner")
print(f"✅ Linhas após filtrar MEIAS: {len(df_meia)} | Times com meias: {df_meia['TIME_FIX'].nunique()}")

# --------- Core functions ----------
def last_n_games(df_base, time_fix, n, modo, mando=None):
    dft = df_base[df_base["TIME_FIX"]==time_fix].copy()
    if modo == "POR_MANDO":
        if mando not in ["CASA","FORA"]:
            return dft.iloc[0:0].copy()
        dft = dft[dft["MANDO_FIX"]==mando].copy()

    dft = dft.sort_values("DATA_FIX")
    datas = sorted(dft["DATA_FIX"].unique())
    datas_sel = datas[-n:] if n > 0 else []
    if not datas_sel:
        return dft.iloc[0:0].copy()
    return dft[dft["DATA_FIX"].isin(datas_sel)].copy()

def agg_metrics(df_slice):
    out = {}
    out["JOGOS_ENCONTRADOS"] = int(df_slice["DATA_FIX"].nunique()) if len(df_slice) else 0

    out["AF"] = float(df_slice[col_map["AF"]].sum()) if col_map["AF"] else np.nan
    out["PG"] = float(df_slice[col_map["PG"]].sum()) if col_map["PG"] else np.nan
    out["CHUTES_FF"] = float(df_slice[col_map["FF"]].sum()) if col_map["FF"] else np.nan
    out["CHUTES_FS"] = float(df_slice[col_map["FS"]].sum()) if col_map["FS"] else np.nan
    out["MEDIA_BASICA"] = float(df_slice[col_map["BASICA"]].mean()) if (col_map["BASICA"] and len(df_slice)) else (0.0 if col_map["BASICA"] else np.nan)
    out["PTS_TOTAL"] = float(df_slice[col_map["PTS"]].sum()) if col_map["PTS"] else np.nan

    return out

# --------- Run: 3 cenários por time ----------
times = sorted(df_meia["TIME_FIX"].unique())
rows = []

for t in times:
    # TODAS
    sl = last_n_games(df_meia, t, N_JOGOS_TESTE, "TODAS")
    rows.append({"TIME": t, "MODO": "TODAS", "MANDO": "TODAS", "N_JOGOS_SOLICITADO": N_JOGOS_TESTE, **agg_metrics(sl)})

    # CASA
    sl = last_n_games(df_meia, t, N_JOGOS_TESTE, "POR_MANDO", "CASA")
    rows.append({"TIME": t, "MODO": "POR_MANDO", "MANDO": "CASA", "N_JOGOS_SOLICITADO": N_JOGOS_TESTE, **agg_metrics(sl)})

    # FORA
    sl = last_n_games(df_meia, t, N_JOGOS_TESTE, "POR_MANDO", "FORA")
    rows.append({"TIME": t, "MODO": "POR_MANDO", "MANDO": "FORA", "N_JOGOS_SOLICITADO": N_JOGOS_TESTE, **agg_metrics(sl)})

df_out = pd.DataFrame(rows)

# --------- Auditoria detalhada (1 time) ----------
COL_ADV = pick_col(df.columns, ["ADVERSÁRIO","ADVERSARIO","OPONENTE","ADVERSÁRIO "])

aud = []
for modo, mando in [("TODAS","TODAS"), ("POR_MANDO","CASA"), ("POR_MANDO","FORA")]:
    sl = last_n_games(df_meia, TIME_AUDITORIA, N_JOGOS_TESTE, "TODAS") if modo=="TODAS" else last_n_games(df_meia, TIME_AUDITORIA, N_JOGOS_TESTE, "POR_MANDO", mando)

    datas = ", ".join([d.strftime("%Y-%m-%d") for d in sorted(sl["DATA_FIX"].unique())]) if len(sl) else ""
    if COL_ADV and len(sl):
        adv_list = ", ".join(sorted(sl[COL_ADV].map(norm_txt).unique()))
    else:
        adv_list = ""

    aud.append({
        "TIME": TIME_AUDITORIA,
        "CENARIO": f"{modo}-{mando}",
        "DATAS_USADAS": datas,
        "ADVERSARIOS": adv_list,
        **agg_metrics(sl)
    })

df_aud = pd.DataFrame(aud)

# --------- Export versionado ----------
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = OUT / f"TESTE_MEIAS_2025_{DATA_CORTE}_N{N_JOGOS_TESTE}_{ts}"
run_dir.mkdir(parents=True, exist_ok=True)

out_main = run_dir / "MEIAS_METRICAS_POR_TIME.csv"
out_aud  = run_dir / f"AUDITORIA_{TIME_AUDITORIA}.csv"

df_out.to_csv(out_main, index=False, encoding="utf-8-sig")
df_aud.to_csv(out_aud, index=False, encoding="utf-8-sig")

print("\n✅ TESTE COMPLETO MEIAS FINALIZADO")
print("📁 Pasta:", run_dir)
print("📄 Principal:", out_main)
print("🧾 Auditoria:", out_aud)

display(df_aud)
display(df_out.head(15))


In [ ]:
# ============================================================
# MOTOR FINAL – TABELAS TCC (PRODUÇÃO)
# 1 tabela por posição | 1 linha = 1 confronto
# Mandante à esquerda | Visitante à direita
# Controle total via PILOTO_GERACAO_TABELAS.csv
# ============================================================

import pandas as pd
from pathlib import Path
import numpy as np
from datetime import datetime

# =========================
# CAMINHOS
# =========================
BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE / "01_INPUT"
OUT  = BASE / "02_OUTPUT"
CFG  = BASE / "03_CONFIG"

PILOTO_PATH     = CFG / "PILOTO_GERACAO_TABELAS.csv"
SCOUTS_PATH     = INP / "Scouts_Reorganizado.xlsx"
DIVISAO_PATH    = INP / "DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"
CONFRONTOS_PATH = INP / "RODADAS_2026_CONFRONTOS.txt"

# =========================
# FUNÇÕES AUXILIARES
# =========================
def norm(x):
    if pd.isna(x):
        return ""
    return str(x).strip().upper()

def load_piloto(path):
    df = pd.read_csv(path, dtype=str, encoding="utf-8-sig")
    df.columns = [c.replace("\ufeff","").strip().upper() for c in df.columns]
    if not {"CHAVE","VALOR"}.issubset(df.columns):
        raise ValueError("PILOTO precisa ter colunas CHAVE | VALOR")
    return {norm(k): v.strip() for k,v in zip(df["CHAVE"], df["VALOR"])}

def load_confrontos(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if "|" not in line:
                continue
            r, m, v = line.strip().split("|")
            rows.append({
                "RODADA": int(r),
                "MANDANTE": norm(m),
                "VISITANTE": norm(v)
            })
    return pd.DataFrame(rows)

def last_n_games(df, time, n, modo, mando=None):
    dft = df[df["TIME_FIX"] == time].copy()
    if modo == "POR_MANDO":
        dft = dft[dft["MANDO_FIX"] == mando]
    datas = sorted(dft["DATA_FIX"].unique())
    datas_sel = datas[-n:] if n > 0 else []
    return dft[dft["DATA_FIX"].isin(datas_sel)]

# =========================
# LEITURA DO PILOTO
# =========================
piloto = load_piloto(PILOTO_PATH)

RODADA_REF = int(piloto["RODADA_REFERENCIA"])
N_JOGOS    = int(piloto["N_JOGOS"])
FILTRO    = piloto["FILTRO"]      # TODAS | POR_MANDO
GERAR     = piloto["GERAR_TABELAS"]

if GERAR != "SIM":
    raise ValueError("GERAR_TABELAS != SIM. Execução abortada.")

if FILTRO not in ["TODAS","POR_MANDO"]:
    raise ValueError("FILTRO inválido. Use TODAS ou POR_MANDO.")

print(f"▶ Rodada {RODADA_REF} | N_JOGOS={N_JOGOS} | FILTRO={FILTRO}")

# =========================
# CONFRONTOS DA RODADA
# =========================
df_conf = load_confrontos(CONFRONTOS_PATH)
df_conf = df_conf[df_conf["RODADA"] == RODADA_REF].copy()

if df_conf.empty:
    raise ValueError("Nenhum confronto encontrado para a rodada.")

# =========================
# SCOUTS – ABA POR JOGO
# =========================
xls = pd.ExcelFile(SCOUTS_PATH)
sheet = [s for s in xls.sheet_names if "JOGO" in norm(s)][0]

df = pd.read_excel(SCOUTS_PATH, sheet_name=sheet)

df.columns = [str(c).strip() for c in df.columns]

df["TIME_FIX"]  = df["TIME"].map(norm)
df["JOG_FIX"]   = df["NOME"].map(norm)
df["MANDO_FIX"] = df["MANDO"].map(norm)
df["DATA_FIX"]  = pd.to_datetime(df["DATA"], dayfirst=True, errors="coerce")

df = df[df["DATA_FIX"].notna()].copy()

# =========================
# DIVISÃO MEIAS / VOLANTES
# =========================
df_div = pd.read_csv(DIVISAO_PATH, dtype=str, encoding="utf-8-sig")
df_div.columns = [c.replace("\ufeff","").strip().upper() for c in df_div.columns]

df_div["TIME_FIX"] = df_div["TIME"].map(norm)
df_div["JOG_FIX"]  = df_div["JOGADOR"].map(norm)
df_div["GRUPO"]    = df_div["GRUPO"].map(norm)

# =========================
# COLUNAS NUMÉRICAS
# =========================
num_cols = ["AF","PG","FF","FS","BASICA","PTS"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

# =========================
# FUNÇÃO GENÉRICA DE AGREGAÇÃO
# (usada por todas as posições)
# =========================
def agg(df_slice):
    out = {}
    for c in num_cols:
        if c in df_slice.columns:
            if c == "BASICA":
                out[c] = df_slice[c].mean() if len(df_slice) else 0
            else:
                out[c] = df_slice[c].sum()
    out["JOGOS"] = df_slice["DATA_FIX"].nunique()
    return out

# =========================
# MOTOR – 1 LINHA POR CONFRONTO
# =========================
linhas = {
    "GOLEIROS": [],
    "ZAGUEIROS": [],
    "LATERAIS": [],
    "MEIAS": [],
    "ATACANTES": []
}

for _, row in df_conf.iterrows():
    mand = row["MANDANTE"]
    vist = row["VISITANTE"]

    mando_mand = "CASA"
    mando_vist = "FORA"

    base_mand = last_n_games(df, mand, N_JOGOS, FILTRO, mando_mand)
    base_vist = last_n_games(df, vist, N_JOGOS, FILTRO, mando_vist)

    # ===== MEIAS (exemplo completo; demais seguem mesma lógica)
    meia_m = base_mand.merge(df_div[df_div["GRUPO"]=="MEIA"],
                             on=["TIME_FIX","JOG_FIX"], how="inner")
    meia_v = base_vist.merge(df_div[df_div["GRUPO"]=="MEIA"],
                             on=["TIME_FIX","JOG_FIX"], how="inner")

    linhas["MEIAS"].append({
        "MANDANTE": mand,
        **{f"M_{k}":v for k,v in agg(meia_m).items()},
        "X": "X",
        **{f"V_{k}":v for k,v in agg(meia_v).items()},
        "VISITANTE": vist
    })

# =========================
# EXPORTAÇÃO FINAL
# =========================
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_dir = OUT / f"RODADA_{RODADA_REF}_{FILTRO}_N{N_JOGOS}_{ts}"
out_dir.mkdir(parents=True, exist_ok=True)

for pos, rows in linhas.items():
    if rows:
        pd.DataFrame(rows).to_csv(out_dir / f"{pos}.csv", index=False)

print("✅ TABELAS GERADAS COM SUCESSO")
print(f"📂 Saída: {out_dir}")


In [ ]:
# ============================================================
# MOTOR FINAL – TABELAS TCC (PRODUÇÃO)
# 1 tabela por posição | 1 linha = 1 confronto
# Mandante à esquerda | Visitante à direita
# Controle total via PILOTO_GERACAO_TABELAS.csv
# ============================================================

import pandas as pd
from pathlib import Path
import numpy as np
from datetime import datetime

# =========================
# CAMINHOS
# =========================
BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE / "01_INPUT"
OUT  = BASE / "02_OUTPUT"
CFG  = BASE / "03_CONFIG"

PILOTO_PATH     = CFG / "PILOTO_GERACAO_TABELAS.csv"
SCOUTS_PATH     = INP / "Scouts_Reorganizado.xlsx"
DIVISAO_PATH    = INP / "DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"
CONFRONTOS_PATH = INP / "RODADAS_2026_CONFRONTOS.txt"

# =========================
# FUNÇÕES AUXILIARES
# =========================
def norm(x):
    if pd.isna(x):
        return ""
    return str(x).strip().upper()

def load_piloto(path):
    df = pd.read_csv(path, dtype=str, encoding="utf-8-sig")
    df.columns = [c.replace("\ufeff","").strip().upper() for c in df.columns]
    if not {"CHAVE","VALOR"}.issubset(df.columns):
        raise ValueError("PILOTO precisa ter colunas CHAVE | VALOR")
    return {norm(k): v.strip() for k,v in zip(df["CHAVE"], df["VALOR"])}

def load_confrontos(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if "|" not in line:
                continue
            r, m, v = line.strip().split("|")
            rows.append({
                "RODADA": int(r),
                "MANDANTE": norm(m),
                "VISITANTE": norm(v)
            })
    return pd.DataFrame(rows)

def last_n_games(df, time, n, modo, mando=None):
    dft = df[df["TIME_FIX"] == time].copy()
    if modo == "POR_MANDO":
        dft = dft[dft["MANDO_FIX"] == mando]
    datas = sorted(dft["DATA_FIX"].unique())
    datas_sel = datas[-n:] if n > 0 else []
    return dft[dft["DATA_FIX"].isin(datas_sel)]

# =========================
# LEITURA DO PILOTO
# =========================
piloto = load_piloto(PILOTO_PATH)

RODADA_REF = int(piloto["RODADA_REFERENCIA"])
N_JOGOS    = int(piloto["N_JOGOS"])
FILTRO    = piloto["FILTRO"]      # TODAS | POR_MANDO
GERAR     = piloto["GERAR_TABELAS"]

if GERAR != "SIM":
    raise ValueError("GERAR_TABELAS != SIM. Execução abortada.")

if FILTRO not in ["TODAS","POR_MANDO"]:
    raise ValueError("FILTRO inválido. Use TODAS ou POR_MANDO.")

print(f"▶ Rodada {RODADA_REF} | N_JOGOS={N_JOGOS} | FILTRO={FILTRO}")

# =========================
# CONFRONTOS DA RODADA
# =========================
df_conf = load_confrontos(CONFRONTOS_PATH)
df_conf = df_conf[df_conf["RODADA"] == RODADA_REF].copy()

if df_conf.empty:
    raise ValueError("Nenhum confronto encontrado para a rodada.")

# =========================
# SCOUTS – ABA POR JOGO
# =========================
xls = pd.ExcelFile(SCOUTS_PATH)
sheet = [s for s in xls.sheet_names if "JOGO" in norm(s)][0]

df = pd.read_excel(SCOUTS_PATH, sheet_name=sheet)

df.columns = [str(c).strip() for c in df.columns]

df["TIME_FIX"]  = df["TIME"].map(norm)
df["JOG_FIX"]   = df["NOME"].map(norm)
df["MANDO_FIX"] = df["MANDO"].map(norm)
df["DATA_FIX"]  = pd.to_datetime(df["DATA"], dayfirst=True, errors="coerce")

df = df[df["DATA_FIX"].notna()].copy()

# =========================
# DIVISÃO MEIAS / VOLANTES
# =========================
df_div = pd.read_csv(DIVISAO_PATH, dtype=str, encoding="utf-8-sig")
df_div.columns = [c.replace("\ufeff","").strip().upper() for c in df_div.columns]

df_div["TIME_FIX"] = df_div["TIME"].map(norm)
df_div["JOG_FIX"]  = df_div["JOGADOR"].map(norm)
df_div["GRUPO"]    = df_div["GRUPO"].map(norm)

# =========================
# COLUNAS NUMÉRICAS
# =========================
num_cols = ["AF","PG","FF","FS","BASICA","PTS"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

# =========================
# FUNÇÃO GENÉRICA DE AGREGAÇÃO
# (usada por todas as posições)
# =========================
def agg(df_slice):
    out = {}
    for c in num_cols:
        if c in df_slice.columns:
            if c == "BASICA":
                out[c] = df_slice[c].mean() if len(df_slice) else 0
            else:
                out[c] = df_slice[c].sum()
    out["JOGOS"] = df_slice["DATA_FIX"].nunique()
    return out

# =========================
# MOTOR – 1 LINHA POR CONFRONTO
# =========================
linhas = {
    "GOLEIROS": [],
    "ZAGUEIROS": [],
    "LATERAIS": [],
    "MEIAS": [],
    "ATACANTES": []
}

for _, row in df_conf.iterrows():
    mand = row["MANDANTE"]
    vist = row["VISITANTE"]

    mando_mand = "CASA"
    mando_vist = "FORA"

    base_mand = last_n_games(df, mand, N_JOGOS, FILTRO, mando_mand)
    base_vist = last_n_games(df, vist, N_JOGOS, FILTRO, mando_vist)

    # ===== MEIAS (exemplo completo; demais seguem mesma lógica)
    meia_m = base_mand.merge(df_div[df_div["GRUPO"]=="MEIA"],
                             on=["TIME_FIX","JOG_FIX"], how="inner")
    meia_v = base_vist.merge(df_div[df_div["GRUPO"]=="MEIA"],
                             on=["TIME_FIX","JOG_FIX"], how="inner")

    linhas["MEIAS"].append({
        "MANDANTE": mand,
        **{f"M_{k}":v for k,v in agg(meia_m).items()},
        "X": "X",
        **{f"V_{k}":v for k,v in agg(meia_v).items()},
        "VISITANTE": vist
    })

# =========================
# EXPORTAÇÃO FINAL
# =========================
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_dir = OUT / f"RODADA_{RODADA_REF}_{FILTRO}_N{N_JOGOS}_{ts}"
out_dir.mkdir(parents=True, exist_ok=True)

for pos, rows in linhas.items():
    if rows:
        pd.DataFrame(rows).to_csv(out_dir / f"{pos}.csv", index=False)

print("✅ TABELAS GERADAS COM SUCESSO")
print(f"📂 Saída: {out_dir}")


▶ Rodada 5 | N_JOGOS=5 | FILTRO=POR_MANDO


KeyError: 'RODADA'

In [ ]:
# ============================================================
# MOTOR FINAL – TABELAS TCC (PRODUÇÃO)
# 1 tabela por posição | 1 linha = 1 confronto
# Mandante à esquerda | Visitante à direita
# Controle total via PILOTO_GERACAO_TABELAS.csv
# ============================================================

import pandas as pd
from pathlib import Path
import numpy as np
from datetime import datetime

# =========================
# CAMINHOS
# =========================
BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE / "01_INPUT"
OUT  = BASE / "02_OUTPUT"
CFG  = BASE / "03_CONFIG"

PILOTO_PATH     = CFG / "PILOTO_GERACAO_TABELAS.csv"
SCOUTS_PATH     = INP / "Scouts_Reorganizado.xlsx"
DIVISAO_PATH    = INP / "DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"
CONFRONTOS_PATH = INP / "RODADAS_2026_CONFRONTOS.txt"

# =========================
# FUNÇÕES AUXILIARES
# =========================
def norm(x):
    if pd.isna(x):
        return ""
    return str(x).strip().upper()

def load_piloto(path):
    df = pd.read_csv(path, dtype=str, encoding="utf-8-sig")
    df.columns = [c.replace("\ufeff","").strip().upper() for c in df.columns]
    if not {"CHAVE","VALOR"}.issubset(df.columns):
        raise ValueError("PILOTO precisa ter colunas CHAVE | VALOR")
    return {norm(k): v.strip() for k,v in zip(df["CHAVE"], df["VALOR"])}

def load_confrontos(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if "|" not in line:
                continue
            r, m, v = line.strip().split("|")
            rows.append({
                "RODADA": int(r),
                "MANDANTE": norm(m),
                "VISITANTE": norm(v)
            })
    return pd.DataFrame(rows)

def last_n_games(df, time, n, modo, mando=None):
    dft = df[df["TIME_FIX"] == time].copy()
    if modo == "POR_MANDO":
        dft = dft[dft["MANDO_FIX"] == mando]
    datas = sorted(dft["DATA_FIX"].unique())
    datas_sel = datas[-n:] if n > 0 else []
    return dft[dft["DATA_FIX"].isin(datas_sel)]

# =========================
# LEITURA DO PILOTO
# =========================
piloto = load_piloto(PILOTO_PATH)

RODADA_REF = int(piloto["RODADA_REFERENCIA"])
N_JOGOS    = int(piloto["N_JOGOS"])
FILTRO    = piloto["FILTRO"]      # TODAS | POR_MANDO
GERAR     = piloto["GERAR_TABELAS"]

if GERAR != "SIM":
    raise ValueError("GERAR_TABELAS != SIM. Execução abortada.")

if FILTRO not in ["TODAS","POR_MANDO"]:
    raise ValueError("FILTRO inválido. Use TODAS ou POR_MANDO.")

print(f"▶ Rodada {RODADA_REF} | N_JOGOS={N_JOGOS} | FILTRO={FILTRO}")

# =========================
# CONFRONTOS DA RODADA
# =========================
df_conf = load_confrontos(CONFRONTOS_PATH)
df_conf = df_conf[df_conf["RODADA"] == RODADA_REF].copy()

if df_conf.empty:
    raise ValueError("Nenhum confronto encontrado para a rodada.")

# =========================
# SCOUTS – ABA POR JOGO
# =========================
xls = pd.ExcelFile(SCOUTS_PATH)
sheet = [s for s in xls.sheet_names if "JOGO" in norm(s)][0]

df = pd.read_excel(SCOUTS_PATH, sheet_name=sheet)

df.columns = [str(c).strip() for c in df.columns]

df["TIME_FIX"]  = df["TIME"].map(norm)
df["JOG_FIX"]   = df["NOME"].map(norm)
df["MANDO_FIX"] = df["MANDO"].map(norm)
df["DATA_FIX"]  = pd.to_datetime(df["DATA"], dayfirst=True, errors="coerce")

df = df[df["DATA_FIX"].notna()].copy()

# =========================
# DIVISÃO MEIAS / VOLANTES
# =========================
df_div = pd.read_csv(DIVISAO_PATH, dtype=str, encoding="utf-8-sig")
df_div.columns = [c.replace("\ufeff","").strip().upper() for c in df_div.columns]

df_div["TIME_FIX"] = df_div["TIME"].map(norm)
df_div["JOG_FIX"]  = df_div["JOGADOR"].map(norm)
df_div["GRUPO"]    = df_div["GRUPO"].map(norm)

# =========================
# COLUNAS NUMÉRICAS
# =========================
num_cols = ["AF","PG","FF","FS","BASICA","PTS"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

# =========================
# FUNÇÃO GENÉRICA DE AGREGAÇÃO
# (usada por todas as posições)
# =========================
def agg(df_slice):
    out = {}
    for c in num_cols:
        if c in df_slice.columns:
            if c == "BASICA":
                out[c] = df_slice[c].mean() if len(df_slice) else 0
            else:
                out[c] = df_slice[c].sum()
    out["JOGOS"] = df_slice["DATA_FIX"].nunique()
    return out

# =========================
# MOTOR – 1 LINHA POR CONFRONTO
# =========================
linhas = {
    "GOLEIROS": [],
    "ZAGUEIROS": [],
    "LATERAIS": [],
    "MEIAS": [],
    "ATACANTES": []
}

for _, row in df_conf.iterrows():
    mand = row["MANDANTE"]
    vist = row["VISITANTE"]

    mando_mand = "CASA"
    mando_vist = "FORA"

    base_mand = last_n_games(df, mand, N_JOGOS, FILTRO, mando_mand)
    base_vist = last_n_games(df, vist, N_JOGOS, FILTRO, mando_vist)

    # ===== MEIAS (exemplo completo; demais seguem mesma lógica)
    meia_m = base_mand.merge(df_div[df_div["GRUPO"]=="MEIA"],
                             on=["TIME_FIX","JOG_FIX"], how="inner")
    meia_v = base_vist.merge(df_div[df_div["GRUPO"]=="MEIA"],
                             on=["TIME_FIX","JOG_FIX"], how="inner")

    linhas["MEIAS"].append({
        "MANDANTE": mand,
        **{f"M_{k}":v for k,v in agg(meia_m).items()},
        "X": "X",
        **{f"V_{k}":v for k,v in agg(meia_v).items()},
        "VISITANTE": vist
    })

# =========================
# EXPORTAÇÃO FINAL
# =========================
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_dir = OUT / f"RODADA_{RODADA_REF}_{FILTRO}_N{N_JOGOS}_{ts}"
out_dir.mkdir(parents=True, exist_ok=True)

for pos, rows in linhas.items():
    if rows:
        pd.DataFrame(rows).to_csv(out_dir / f"{pos}.csv", index=False)

print("✅ TABELAS GERADAS COM SUCESSO")
print(f"📂 Saída: {out_dir}")



▶ Rodada 5 | N_JOGOS=5 | FILTRO=POR_MANDO


KeyError: 'RODADA'

In [ ]:
# ==========================
# MOTOR FINAL (v1) - TCC TABELAS 2026
# - 1 CSV por posição
# - 1 linha por confronto (mandante à esquerda / visitante à direita)
# - Janela por DATA (últimos N jogos)
# - FILTRO: TODAS ou POR_MANDO (casa/fora conforme mando do confronto)
# ==========================

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# -----------------------
# 0) Drive + Paths
# -----------------------
try:
    from google.colab import drive
    if not Path("/content/drive").exists():
        drive.mount("/content/drive")
except Exception:
    pass

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE / "01_INPUT"
OUT  = BASE / "02_OUTPUT"
CFG  = BASE / "03_CONFIG"

assert INP.exists(), f"Pasta ausente: {INP}"
assert OUT.exists(), f"Pasta ausente: {OUT}"
assert CFG.exists(), f"Pasta ausente: {CFG}"

print("✅ Pastas OK")

# -----------------------
# 1) Helpers (blindados)
# -----------------------
def norm_txt(x):
    if pd.isna(x):
        return ""
    return str(x).strip().upper()

def read_csv_robusto(path: Path):
    """
    Lê CSV com:
    - encoding UTF-8-SIG (remove BOM)
    - tenta separador vírgula e ponto-e-vírgula
    """
    for sep in [",", ";"]:
        try:
            df = pd.read_csv(path, dtype=str, encoding="utf-8-sig", sep=sep)
            if df.shape[1] >= 1:
                return df
        except Exception:
            continue
    # fallback: tenta engine python
    for sep in [",", ";"]:
        try:
            df = pd.read_csv(path, dtype=str, encoding="utf-8-sig", sep=sep, engine="python")
            return df
        except Exception:
            continue
    raise ValueError(f"Não consegui ler CSV: {path}")

def auto_find_file(folder: Path, patterns):
    """
    Procura o primeiro arquivo que bater com algum padrão (case-insensitive).
    patterns: lista de strings, ex: ["PILOTO", "CONFRONTOS_2026_NORMALIZADO"]
    """
    files = list(folder.glob("*"))
    files_sorted = sorted(files, key=lambda p: p.name.lower())
    for pat in patterns:
        pat_u = pat.upper()
        for f in files_sorted:
            if pat_u in f.name.upper():
                return f
    return None

# -----------------------
# 2) Localizar arquivos
# -----------------------
# PILOTO
PILOTO_PATH = CFG / "PILOTO_GERACAO_TABELAS.csv"
if not PILOTO_PATH.exists():
    found = auto_find_file(CFG, ["PILOTO", "GERACAO_TABELAS"])
    if found:
        PILOTO_PATH = found
if not PILOTO_PATH.exists():
    raise FileNotFoundError(f"Não achei PILOTO em {CFG}. Procure um arquivo tipo PILOTO*.csv")

# CONFRONTOS NORMALIZADO (pode estar em 02_OUTPUT)
CONFRONTOS_PATH = OUT / "CONFRONTOS_2026_NORMALIZADO.csv"
if not CONFRONTOS_PATH.exists():
    found = auto_find_file(OUT, ["CONFRONTOS_2026_NORMALIZADO", "CONFRONTOS_NORMALIZADO"])
    if found:
        CONFRONTOS_PATH = found
if not CONFRONTOS_PATH.exists():
    # fallback: se você colocou em INPUT
    found = auto_find_file(INP, ["CONFRONTOS_2026_NORMALIZADO", "CONFRONTOS_NORMALIZADO"])
    if found:
        CONFRONTOS_PATH = found
if not CONFRONTOS_PATH.exists():
    raise FileNotFoundError("Não achei CONFRONTOS_2026_NORMALIZADO.csv em 02_OUTPUT nem em 01_INPUT.")

SCOUTS_PATH = INP / "Scouts_Reorganizado.xlsx"
if not SCOUTS_PATH.exists():
    raise FileNotFoundError(f"Não achei: {SCOUTS_PATH}")

DIVISAO_PATH = INP / "DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"
if not DIVISAO_PATH.exists():
    print("⚠️ DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv não encontrado. MEIAS será por POS (Posreal) sem refinamento MEIA/VOLANTE.")
    DIVISAO_PATH = None

print("✅ Arquivos OK")
print("PILOTO:", PILOTO_PATH)
print("CONFRONTOS:", CONFRONTOS_PATH)
print("SCOUTS:", SCOUTS_PATH)
print("DIVISAO:", DIVISAO_PATH if DIVISAO_PATH else "NÃO VAI USAR")

# -----------------------
# 3) Load PILOTO (robusto)
# -----------------------
def load_piloto(path: Path) -> dict:
    df = read_csv_robusto(path)

    # normaliza nomes de colunas (remove BOM)
    df.columns = [str(c).replace("\ufeff","").strip().upper() for c in df.columns]

    # Caso 1: já vem CHAVE/VALOR
    if set(df.columns) >= {"CHAVE","VALOR"}:
        ch = df["CHAVE"].map(norm_txt)
        va = df["VALOR"].astype(str).str.strip()
        return dict(zip(ch, va))

    # Caso 2: veio como 1 coluna "CHAVE,VALOR" por erro de separador
    if df.shape[1] == 1:
        col = df.columns[0]
        # tenta split por vírgula
        tmp = df[col].astype(str).str.split(",", n=1, expand=True)
        if tmp.shape[1] == 2:
            tmp.columns = ["CHAVE","VALOR"]
            tmp["CHAVE"] = tmp["CHAVE"].map(norm_txt)
            tmp["VALOR"] = tmp["VALOR"].astype(str).str.strip()
            return dict(zip(tmp["CHAVE"], tmp["VALOR"]))

        # tenta split por ;
        tmp = df[col].astype(str).str.split(";", n=1, expand=True)
        if tmp.shape[1] == 2:
            tmp.columns = ["CHAVE","VALOR"]
            tmp["CHAVE"] = tmp["CHAVE"].map(norm_txt)
            tmp["VALOR"] = tmp["VALOR"].astype(str).str.strip()
            return dict(zip(tmp["CHAVE"], tmp["VALOR"]))

    raise ValueError(f"PILOTO inválido. Precisa ter CHAVE/VALOR. Colunas detectadas: {list(df.columns)}")

piloto = load_piloto(PILOTO_PATH)

def get_int(key, default):
    v = piloto.get(key, str(default)).strip()
    return int(float(v))

RODADA_REF = get_int("RODADA_REFERENCIA", 1)
N_JOGOS    = get_int("N_JOGOS", 5)
FILTRO     = norm_txt(piloto.get("FILTRO", "TODAS"))
GERAR      = norm_txt(piloto.get("GERAR_TABELAS", "SIM"))

if FILTRO not in ["TODAS","POR_MANDO"]:
    raise ValueError("FILTRO inválido no PILOTO. Use TODAS ou POR_MANDO.")
if GERAR not in ["SIM","NAO","NÃO"]:
    raise ValueError("GERAR_TABELAS inválido. Use SIM ou NAO.")

print(f"✅ PILOTO: RODADA_REFERENCIA={RODADA_REF} | N_JOGOS={N_JOGOS} | FILTRO={FILTRO} | GERAR_TABELAS={GERAR}")

# -----------------------
# 4) Load CONFRONTOS (robusto)
# -----------------------
def load_confrontos(path: Path) -> pd.DataFrame:
    df = read_csv_robusto(path)
    df.columns = [str(c).replace("\ufeff","").strip().upper() for c in df.columns]

    required = {"RODADA","MANDANTE","VISITANTE"}
    if not required.issubset(set(df.columns)):
        raise ValueError(f"CONFRONTOS precisa ter colunas RODADA, MANDANTE, VISITANTE. Veio: {list(df.columns)}")

    df["RODADA"] = pd.to_numeric(df["RODADA"], errors="coerce").fillna(0).astype(int)
    df["MANDANTE_FIX"] = df["MANDANTE"].map(norm_txt)
    df["VISITANTE_FIX"] = df["VISITANTE"].map(norm_txt)
    return df

df_conf_all = load_confrontos(CONFRONTOS_PATH)
df_conf = df_conf_all[df_conf_all["RODADA"] == RODADA_REF].copy()

if df_conf.empty:
    raise ValueError(f"Não encontrei confrontos para a rodada {RODADA_REF}. Verifique o CONFRONTOS_2026_NORMALIZADO.csv")

print(f"✅ Confrontos na rodada {RODADA_REF}: {len(df_conf)}")

# -----------------------
# 5) Load SCOUTS (aba Por jogo)
# -----------------------
xls = pd.ExcelFile(SCOUTS_PATH)
sheet = None
for s in xls.sheet_names:
    if norm_txt(s) in ["POR JOGO","POR_JOGO","PORJOGO"]:
        sheet = s
        break
if sheet is None:
    # fallback: qualquer aba que contenha "JOGO"
    for s in xls.sheet_names:
        if "JOGO" in norm_txt(s):
            sheet = s
            break
if sheet is None:
    raise ValueError(f"Não achei aba 'Por jogo'. Abas: {xls.sheet_names}")

df = pd.read_excel(SCOUTS_PATH, sheet_name=sheet)

# Normaliza nomes de colunas (mantém acentos, mas cria mapa)
df.columns = [str(c).strip() for c in df.columns]
cols_upper = {c.upper(): c for c in df.columns}

def pick(cands):
    for c in cands:
        if c.upper() in cols_upper:
            return cols_upper[c.upper()]
    return None

COL_TIME  = pick(["TIME","CLUBE"])
COL_DATA  = pick(["DATA"])
COL_MANDO = pick(["MANDO"])
COL_POS   = pick(["POSREAL","POS","POSIÇÃO","POSICAO"])
COL_PTS   = pick(["PTS","PONTOS"])
COL_BAS   = pick(["BÁSICA","BASICA"])

if not all([COL_TIME, COL_DATA, COL_MANDO, COL_POS]):
    raise ValueError(f"Base 'Por jogo' precisa ter TIME, DATA, MANDO, POS. Detectei TIME={COL_TIME}, DATA={COL_DATA}, MANDO={COL_MANDO}, POS={COL_POS}")

df["TIME_FIX"]  = df[COL_TIME].map(norm_txt)
df["DATA_FIX"]  = pd.to_datetime(df[COL_DATA], errors="coerce", dayfirst=True)
df["MANDO_FIX"] = df[COL_MANDO].map(norm_txt)
df["POS_FIX"]   = df[COL_POS].map(norm_txt)

df = df[df["DATA_FIX"].notna()].copy()

def norm_mando(x):
    x = norm_txt(x)
    if x in ["C", "CASA", "HOME"]: return "CASA"
    if x in ["F", "FORA", "AWAY"]: return "FORA"
    return x

df["MANDO_FIX"] = df["MANDO_FIX"].map(norm_mando)

# Descobre colunas numéricas de scouts (tudo que for scout mesmo)
# Aqui você pode ajustar depois, mas já funciona.
META_COLS = set([COL_TIME, COL_DATA, COL_MANDO, COL_POS, "TIME_FIX", "DATA_FIX", "MANDO_FIX", "POS_FIX"])
num_cols = []
for c in df.columns:
    if c in META_COLS:
        continue
    # tenta converter e assume que scouts são colunas curtas tipo "FS","FF","DS" etc ou colunas "Pts"/"Básica"
    # (vamos converter depois sob demanda)
    if len(str(c)) <= 8 or str(c).strip().upper() in ["PTS","BÁSICA","BASICA"]:
        num_cols.append(c)

# Converte numéricos que existirem
for c in set(num_cols + ([COL_PTS] if COL_PTS else []) + ([COL_BAS] if COL_BAS else [])):
    if c and c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

print(f"✅ Scouts carregado ({sheet}): linhas={len(df)} | times={df['TIME_FIX'].nunique()}")

# -----------------------
# 6) Divisão MEIAS/VOLANTES (opcional)
# -----------------------
df_div_meia = None
if DIVISAO_PATH:
    dd = read_csv_robusto(DIVISAO_PATH)
    dd.columns = [str(c).replace("\ufeff","").strip().upper() for c in dd.columns]

    # aceita GRUPO ou POS_GRUPO
    if "GRUPO" not in dd.columns and "POS_GRUPO" in dd.columns:
        dd = dd.rename(columns={"POS_GRUPO":"GRUPO"})

    required = {"TIME","JOGADOR","GRUPO"}
    if not required.issubset(set(dd.columns)):
        raise ValueError(f"DIVISÃO precisa ter TIME, JOGADOR, GRUPO (ou POS_GRUPO). Veio: {list(dd.columns)}")

    dd["TIME_FIX"] = dd["TIME"].map(norm_txt)
    dd["JOG_FIX"]  = dd["JOGADOR"].map(norm_txt)
    dd["GRUPO"]    = dd["GRUPO"].map(norm_txt)

    # para a tabela MEIAS, vamos aceitar MEIA e VOLANTE (porque no Cartola ambos são "MEIAS")
    df_div_meia = dd[dd["GRUPO"].isin(["MEIA","VOLANTE"])][["TIME_FIX","JOG_FIX","GRUPO"]].drop_duplicates()

    # tenta achar coluna de nome do jogador na base
    COL_NOME = pick(["NOME","JOGADOR"])
    if COL_NOME:
        df["JOG_FIX"] = df[COL_NOME].map(norm_txt)
    else:
        # se não tiver nome, não dá para aplicar divisão; cai no POS_FIX
        df_div_meia = None
        print("⚠️ Não achei coluna NOME/JOGADOR no Scouts. Não vou usar DIVISÃO para MEIAS.")
else:
    print("ℹ️ Sem DIVISÃO: MEIAS será por POS_FIX.")

# -----------------------
# 7) Pegar últimos N jogos por time (por DATA), com opção de mando
# -----------------------
def last_n_games_team(df_base, time_fix, n, modo, mando=None):
    dft = df_base[df_base["TIME_FIX"] == time_fix].copy()
    if modo == "POR_MANDO":
        if mando not in ["CASA","FORA"]:
            return dft.iloc[0:0].copy()
        dft = dft[dft["MANDO_FIX"] == mando].copy()

    # Seleciona as últimas N datas únicas e mantém todas as linhas dessas datas
    dft = dft.sort_values("DATA_FIX")
    datas = sorted(dft["DATA_FIX"].dropna().unique())
    datas_sel = datas[-n:] if n > 0 else []
    return dft[dft["DATA_FIX"].isin(datas_sel)].copy()

def soma_scouts(df_slice, scouts_list):
    out = {}
    for s in scouts_list:
        col = None
        # procura coluna exata no df (case-insensitive)
        for c in df_slice.columns:
            if str(c).strip().upper() == s.upper():
                col = c
                break
        if col:
            out[s] = float(df_slice[col].sum())
        else:
            out[s] = np.nan
    out["JOGOS_ENCONTRADOS"] = int(df_slice["DATA_FIX"].nunique())
    if COL_PTS and COL_PTS in df_slice.columns:
        out["PTS_TOTAL"] = float(df_slice[COL_PTS].sum())
    else:
        out["PTS_TOTAL"] = np.nan
    if COL_BAS and COL_BAS in df_slice.columns:
        out["MEDIA_BASICA"] = float(df_slice[COL_BAS].mean()) if len(df_slice) else 0.0
    else:
        out["MEDIA_BASICA"] = np.nan
    return out

# Scouts por posição (mínimo funcional e coerente com seus modelos)
SCOUTS_MEIAS = ["FS","DS","F","I","FC","PP","A","G","FF","PS","PI"]
SCOUTS_ATAC  = ["FS","DS","F","I","FC","PP","A","G","FF","PS","PI"]
SCOUTS_ZAG   = ["FS","DS","F","I","FC","PP","CA","FD","DE","SG","GC"]  # GC pode não existir; fica NaN
SCOUTS_LAT   = ["FS","DS","F","I","FC","PP","A","G","FF","PS","PI"]
SCOUTS_GOL   = ["DE","FD","SG","GS","GC","CA","PP"]  # depende do seu xlsx; o que não existir vira NaN

def filtra_pos(df_base, pos_name):
    # pos_name em: GOLEIRO, ZAGUEIRO, LATERAL, MEIAS, ATACANTE
    if pos_name == "MEIAS":
        if df_div_meia is not None and "JOG_FIX" in df_base.columns:
            # aplica divisão MEIA/VOLANTE via merge
            tmp = df_base.merge(df_div_meia, on=["TIME_FIX","JOG_FIX"], how="inner")
            return tmp
        # fallback: por POS_FIX contendo "MEIA" ou "VOL"
        return df_base[df_base["POS_FIX"].str.contains("MEIA|VOL", regex=True, na=False)].copy()

    if pos_name == "GOLEIROS":
        return df_base[df_base["POS_FIX"].str.contains("GOLEIRO|GOL", regex=True, na=False)].copy()
    if pos_name == "ZAGUEIROS":
        return df_base[df_base["POS_FIX"].str.contains("ZAG", regex=True, na=False)].copy()
    if pos_name == "LATERAIS":
        return df_base[df_base["POS_FIX"].str.contains("LAT", regex=True, na=False)].copy()
    if pos_name == "ATACANTES":
        return df_base[df_base["POS_FIX"].str.contains("ATA|ATAC", regex=True, na=False)].copy()

    return df_base.iloc[0:0].copy()

# -----------------------
# 8) Montagem: 1 linha por confronto (mandante esquerda / visitante direita)
# -----------------------
def build_row_confronto(mand, vist, pos_name):
    mand_fix = norm_txt(mand)
    vist_fix = norm_txt(vist)

    # define mando esperado no confronto
    mand_mando = "CASA"
    vist_mando = "FORA"

    # Base filtrada por posição
    df_pos = filtra_pos(df, pos_name)

    # Recortes por mando (se POR_MANDO) ou TODAS
    if FILTRO == "POR_MANDO":
        mand_slice = last_n_games_team(df_pos, mand_fix, N_JOGOS, "POR_MANDO", mand_mando)
        vist_slice = last_n_games_team(df_pos, vist_fix, N_JOGOS, "POR_MANDO", vist_mando)
        mand_outro = last_n_games_team(df_pos, mand_fix, N_JOGOS, "POR_MANDO", "FORA")
        vist_outro = last_n_games_team(df_pos, vist_fix, N_JOGOS, "POR_MANDO", "CASA")
    else:
        mand_slice = last_n_games_team(df_pos, mand_fix, N_JOGOS, "TODAS")
        vist_slice = last_n_games_team(df_pos, vist_fix, N_JOGOS, "TODAS")
        mand_outro = mand_slice.copy()
        vist_outro = vist_slice.copy()

    # Escolhe scouts conforme posição
    if pos_name == "MEIAS":
        scouts_list = SCOUTS_MEIAS
    elif pos_name == "ATACANTES":
        scouts_list = SCOUTS_ATAC
    elif pos_name == "ZAGUEIROS":
        scouts_list = SCOUTS_ZAG
    elif pos_name == "LATERAIS":
        scouts_list = SCOUTS_LAT
    else:
        scouts_list = SCOUTS_GOL

    m1 = soma_scouts(mand_slice, scouts_list)
    m2 = soma_scouts(mand_outro, scouts_list)
    v1 = soma_scouts(vist_slice, scouts_list)
    v2 = soma_scouts(vist_outro, scouts_list)

    # Constrói row final (mandante à esquerda / visitante à direita)
    row = {
        "RODADA": RODADA_REF,
        "MANDANTE": mand_fix,
        "VISITANTE": vist_fix,
        "FILTRO": FILTRO,
        "N_JOGOS": N_JOGOS,
    }

    # Mandante (mando do confronto)
    for k, val in m1.items():
        row[f"M_{mand_mando}_{k}"] = val
    # Mandante (mando oposto)
    for k, val in m2.items():
        row[f"M_OUTRO_{k}"] = val

    # Visitante (mando do confronto)
    for k, val in v1.items():
        row[f"V_{vist_mando}_{k}"] = val
    # Visitante (mando oposto)
    for k, val in v2.items():
        row[f"V_OUTRO_{k}"] = val

    return row

# -----------------------
# 9) Gerar tabelas por posição
# -----------------------
if GERAR in ["NAO","NÃO"]:
    print("⚠️ GERAR_TABELAS=NAO. Nada será exportado.")
else:
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = OUT / f"RUN_2026_R{RODADA_REF:02d}_{FILTRO}_N{N_JOGOS}_{ts}"
    run_dir.mkdir(parents=True, exist_ok=True)

    pos_list = ["GOLEIROS","ZAGUEIROS","LATERAIS","MEIAS","ATACANTES"]
    for pos_name in pos_list:
        rows = []
        for _, r in df_conf.iterrows():
            rows.append(build_row_confronto(r["MANDANTE"], r["VISITANTE"], pos_name))
        df_out = pd.DataFrame(rows)
        out_file = run_dir / f"TABELA_{pos_name}_R{RODADA_REF:02d}.csv"
        df_out.to_csv(out_file, index=False, encoding="utf-8-sig")
        print(f"✅ Gerado: {out_file.name} | linhas={len(df_out)}")

    print("\n✅ TABELAS GERADAS COM SUCESSO")
    print("📁 Saída:", run_dir)


✅ Pastas OK
✅ Arquivos OK
PILOTO: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/PILOTO_GERACAO_TABELAS.csv
CONFRONTOS: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/CONFRONTOS_2026_NORMALIZADO.csv
SCOUTS: /content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx
DIVISAO: /content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv
✅ PILOTO: RODADA_REFERENCIA=5 | N_JOGOS=5 | FILTRO=POR_MANDO | GERAR_TABELAS=SIM
✅ Confrontos na rodada 5: 10
✅ Scouts carregado (Por jogo): linhas=12086 | times=20
✅ Gerado: TABELA_GOLEIROS_R05.csv | linhas=10
✅ Gerado: TABELA_ZAGUEIROS_R05.csv | linhas=10
✅ Gerado: TABELA_LATERAIS_R05.csv | linhas=10
✅ Gerado: TABELA_MEIAS_R05.csv | linhas=10
✅ Gerado: TABELA_ATACANTES_R05.csv | linhas=10

✅ TABELAS GERADAS COM SUCESSO
📁 Saída: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_161432


In [ ]:
# ==========================
# MOTOR FINAL - TCC (2026)
# Gera 1 CSV por posição por rodada (1 linha = 1 confronto)
# Laterais separados em LD (2.2) e LE (2.6)
# Sem DATA_CORTE (produção)
# ==========================

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# -------------------
# Paths (seu padrão)
# -------------------
BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

PILOTO_PATH     = CFG/"PILOTO_GERACAO_TABELAS.csv"
SCOUTS_PATH     = INP/"Scouts_Reorganizado.xlsx"
DIVISAO_PATH    = INP/"DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"
CONFRONTOS_PATH = OUT/"CONFRONTOS_2026_NORMALIZADO.csv"   # (você já gerou esse)

# -------------------
# Helpers
# -------------------
def norm_txt(x):
    if pd.isna(x): return ""
    return str(x).strip().upper()

def strip_bom(s):
    return s.replace("\ufeff", "").strip()

def pick_col(df_cols, candidates):
    up = {strip_bom(str(c)).upper(): c for c in df_cols}
    for cand in candidates:
        key = strip_bom(cand).upper()
        if key in up:
            return up[key]
    return None

def load_piloto(p):
    df = pd.read_csv(p, dtype=str, encoding="utf-8-sig")
    df.columns = [strip_bom(c).upper() for c in df.columns]

    if set(df.columns) >= {"CHAVE","VALOR"}:
        df["CHAVE"] = df["CHAVE"].map(norm_txt)
        df["VALOR"] = df["VALOR"].astype(str).str.strip()
        d = dict(zip(df["CHAVE"], df["VALOR"]))
        return d, df

    raise ValueError(f"❌ PILOTO inválido. Precisa ter colunas CHAVE e VALOR. Veio: {list(df.columns)}")

def ensure_drive():
    try:
        from google.colab import drive
        if not Path("/content/drive").exists():
            drive.mount("/content/drive")
    except Exception:
        pass

# -------------------
# Start
# -------------------
ensure_drive()

# checks
for p in [INP, OUT, CFG]:
    if not p.exists():
        raise FileNotFoundError(f"❌ Pasta ausente: {p}")

for f in [PILOTO_PATH, SCOUTS_PATH, DIVISAO_PATH, CONFRONTOS_PATH]:
    if not f.exists():
        raise FileNotFoundError(f"❌ Arquivo ausente: {f}")

print("✅ Pastas OK")
print("✅ Arquivos OK")
print("PILOTO:", PILOTO_PATH)
print("CONFRONTOS:", CONFRONTOS_PATH)
print("SCOUTS:", SCOUTS_PATH)
print("DIVISAO:", DIVISAO_PATH)

# -------------------
# PILOTO
# -------------------
piloto, df_piloto = load_piloto(PILOTO_PATH)

RODADA_REF = int(piloto.get("RODADA_REFERENCIA", "0"))
N_JOGOS    = int(piloto.get("N_JOGOS", "5"))
FILTRO     = norm_txt(piloto.get("FILTRO", "TODAS"))
GERAR      = norm_txt(piloto.get("GERAR_TABELAS", "SIM"))

if RODADA_REF <= 0:
    raise ValueError("❌ PILOTO: RODADA_REFERENCIA precisa ser > 0")
if N_JOGOS < 0:
    raise ValueError("❌ PILOTO: N_JOGOS não pode ser negativo")
if FILTRO not in ["TODAS", "POR_MANDO"]:
    raise ValueError("❌ PILOTO: FILTRO deve ser TODAS ou POR_MANDO")
if GERAR not in ["SIM", "NAO", "NÃO"]:
    raise ValueError("❌ PILOTO: GERAR_TABELAS deve ser SIM ou NAO")

print(f"✅ PILOTO: RODADA_REFERENCIA={RODADA_REF} | N_JOGOS={N_JOGOS} | FILTRO={FILTRO} | GERAR_TABELAS={GERAR}")

# -------------------
# Confrontos
# -------------------
df_conf = pd.read_csv(CONFRONTOS_PATH, dtype=str, encoding="utf-8-sig")
df_conf.columns = [strip_bom(c).upper() for c in df_conf.columns]

COL_ROD = pick_col(df_conf.columns, ["RODADA"])
COL_MAN = pick_col(df_conf.columns, ["MANDANTE"])
COL_VIS = pick_col(df_conf.columns, ["VISITANTE"])

if not all([COL_ROD, COL_MAN, COL_VIS]):
    raise ValueError(f"❌ Confrontos precisa ter RODADA, MANDANTE, VISITANTE. Veio: {list(df_conf.columns)}")

df_conf[COL_ROD] = pd.to_numeric(df_conf[COL_ROD], errors="coerce")
df_conf = df_conf[df_conf[COL_ROD].notna()].copy()
df_conf[COL_ROD] = df_conf[COL_ROD].astype(int)

# normaliza times
df_conf["MANDANTE_FIX"] = df_conf[COL_MAN].map(norm_txt)
df_conf["VISITANTE_FIX"] = df_conf[COL_VIS].map(norm_txt)

df_r = df_conf[df_conf[COL_ROD] == RODADA_REF].copy()
print(f"✅ Confrontos na rodada {RODADA_REF}: {len(df_r)}")
if df_r.empty:
    raise ValueError("❌ Nenhum confronto encontrado para a RODADA_REFERENCIA.")

# -------------------
# Scouts (aba Por jogo)
# -------------------
xls = pd.ExcelFile(SCOUTS_PATH)
sheet_name = None
for s in xls.sheet_names:
    if norm_txt(s) in ["POR JOGO", "POR_JOGO", "PORJOGO"]:
        sheet_name = s
        break
if sheet_name is None:
    for s in xls.sheet_names:
        if "JOGO" in norm_txt(s):
            sheet_name = s
            break
if sheet_name is None:
    raise ValueError(f"❌ Não achei a aba Por jogo. Abas: {xls.sheet_names}")

df = pd.read_excel(SCOUTS_PATH, sheet_name=sheet_name)
df.columns = [strip_bom(str(c)) for c in df.columns]

COL_TIME  = pick_col(df.columns, ["TIME", "CLUBE"])
COL_ADV   = pick_col(df.columns, ["ADVERSÁRIO", "ADVERSARIO", "OPONENTE"])
COL_MANDO = pick_col(df.columns, ["MANDO"])
COL_DATA  = pick_col(df.columns, ["DATA", "DATE"])
COL_POSR  = pick_col(df.columns, ["POSREAL", "POSREAL ", "POSREAL\t", "POSREAL\n", "POSREAL\r"])
# (nome do jogador só é necessário se você for auditar manualmente; não precisamos pra agregação)
# COL_NOME = pick_col(df.columns, ["NOME", "JOGADOR"])

need = [COL_TIME, COL_ADV, COL_MANDO, COL_DATA, COL_POSR]
if not all(need):
    raise ValueError(f"❌ Base Por jogo precisa ter TIME, ADVERSÁRIO, MANDO, DATA, Posreal. Detectei: TIME={COL_TIME}, ADV={COL_ADV}, MANDO={COL_MANDO}, DATA={COL_DATA}, POSREAL={COL_POSR}")

df["TIME_FIX"] = df[COL_TIME].map(norm_txt)
df["ADV_FIX"]  = df[COL_ADV].map(norm_txt)

def norm_mando(x):
    x = norm_txt(x)
    if x in ["C", "CASA", "HOME"]: return "CASA"
    if x in ["F", "FORA", "AWAY"]: return "FORA"
    return x

df["MANDO_FIX"] = df[COL_MANDO].map(norm_mando)
df["DATA_FIX"]  = pd.to_datetime(df[COL_DATA], errors="coerce", dayfirst=True)

# Posreal pode vir como 2.2 / 2.6 etc
df["POSREAL"] = pd.to_numeric(df[COL_POSR], errors="coerce")

df = df[df["DATA_FIX"].notna()].copy()
print(f"✅ Scouts carregado ({sheet_name}): linhas={len(df)} | times={df['TIME_FIX'].nunique()}")

# -------------------
# Divisão meias/volantes (apenas para separar MEIA vs VOLANTE)
# -------------------
df_div = pd.read_csv(DIVISAO_PATH, dtype=str, encoding="utf-8-sig")
df_div.columns = [strip_bom(c).upper() for c in df_div.columns]

# aceita GRUPO ou POS_GRUPO (seu arquivo tem POS_GRUPO)
COL_DTIME = pick_col(df_div.columns, ["TIME"])
COL_DJOG  = pick_col(df_div.columns, ["JOGADOR", "NOME"])
COL_DGRP  = pick_col(df_div.columns, ["GRUPO", "POS_GRUPO"])

if not all([COL_DTIME, COL_DJOG, COL_DGRP]):
    raise ValueError(f"❌ DIVISÃO precisa ter TIME, JOGADOR, (GRUPO ou POS_GRUPO). Veio: {list(df_div.columns)}")

df_div["TIME_FIX"] = df_div[COL_DTIME].map(norm_txt)
df_div["JOG_FIX"]  = df_div[COL_DJOG].map(norm_txt)
df_div["GRUPO_FIX"]= df_div[COL_DGRP].map(norm_txt)

# -------------------
# Scouts numéricos que o motor exporta (usa o que existir)
# -------------------
SCOUTS_LIST = ["FS","DS","F","I","FC","PP","A","G","FF","PS","PI","DE","FD","GS","CA","PC"]
# + pontos/básica (se existir)
COL_PTS    = pick_col(df.columns, ["Pts", "PTS", "PONTOS", "PONTUAÇÃO", "PONTUACAO"])
COL_BASICA = pick_col(df.columns, ["Básica", "BASICA", "BÁSICA"])

# garante numérico
for s in SCOUTS_LIST:
    c = pick_col(df.columns, [s])
    if c:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

if COL_PTS:
    df[COL_PTS] = pd.to_numeric(df[COL_PTS], errors="coerce").fillna(0)
if COL_BASICA:
    df[COL_BASICA] = pd.to_numeric(df[COL_BASICA], errors="coerce").fillna(0)

# -------------------
# Recorte: últimos N jogos por TIME, com filtro de mando
# (1 jogo = 1 DATA_FIX; como a tabela é por jogador, pegamos as últimas N datas únicas do time)
# -------------------
def slice_last_n(df_base, time_fix, n, filtro, mando=None, mode="TIME"):
    """
    mode="TIME": filtra linhas do próprio time (conquistado)
    mode="ADV":  filtra linhas dos adversários contra esse time (cedido) usando ADV_FIX == time
    """
    if mode == "TIME":
        dft = df_base[df_base["TIME_FIX"]==time_fix].copy()
    else:
        dft = df_base[df_base["ADV_FIX"]==time_fix].copy()

    if filtro == "POR_MANDO":
        if mando not in ["CASA","FORA"]:
            return dft.iloc[0:0].copy()
        dft = dft[dft["MANDO_FIX"]==mando].copy()

    # últimas N datas únicas
    datas = sorted(dft["DATA_FIX"].dropna().unique())
    if n <= 0:
        return dft.iloc[0:0].copy()
    datas_sel = datas[-n:] if len(datas) >= 1 else []
    dft = dft[dft["DATA_FIX"].isin(datas_sel)].copy()
    return dft

def agg_sum(df_slice, prefix, out):
    # soma scouts presentes
    for s in SCOUTS_LIST:
        c = pick_col(df_slice.columns, [s])
        if c:
            out[f"{prefix}_{s}"] = float(df_slice[c].sum())
    out[f"{prefix}_JOGOS_ENCONTRADOS"] = int(df_slice["DATA_FIX"].nunique())
    if COL_PTS:
        out[f"{prefix}_PTS_TOTAL"] = float(df_slice[COL_PTS].sum())
    if COL_BASICA:
        out[f"{prefix}_MEDIA_BASICA"] = float(df_slice[COL_BASICA].mean()) if len(df_slice) else 0.0
    return out

# -------------------
# Posreal → grupos
# -------------------
def pos_mask(df_base, grupo):
    if grupo == "GOL":
        return df_base["POSREAL"].round(1) == 1.0
    if grupo == "ZAG":
        return df_base["POSREAL"].round(1) == 3.0
    if grupo == "MEI":
        return df_base["POSREAL"].round(1) == 4.0
    if grupo == "ATA":
        return df_base["POSREAL"].round(1) == 5.0
    if grupo == "LAT":  # geral (se precisar)
        return df_base["POSREAL"].round(1).isin([2.2, 2.6])
    if grupo == "LD":
        return df_base["POSREAL"].round(1) == 2.2
    if grupo == "LE":
        return df_base["POSREAL"].round(1) == 2.6
    return df_base["POSREAL"].isna() & False

# -------------------
# MEIAS: separar MEIA/VOLANTE via DIVISAO (quando necessário)
# -------------------
def apply_divisao_meias(df_mei_raw):
    # se não existir coluna de nome no scouts, não conseguimos casar por jogador
    # então, por enquanto, só devolve como MEI geral.
    # (Seu motor anterior já funciona assim; a separação fina entra depois.)
    return df_mei_raw

# -------------------
# Builder por posição
# (mantém padrão de colunas do seu motor atual: M_CASA / M_FORA / M_OUTRO e V_CASA / V_FORA / V_OUTRO)
# Interpretação operacional:
# - Para MANDANTE:
#   M_CASA = conquistado do mandante em CASA
#   M_FORA = cedido ao mandante em CASA (adversários FORA contra ele)
#   M_OUTRO = conquistado do mandante FORA (referência complementar)
# - Para VISITANTE:
#   V_FORA = conquistado do visitante FORA
#   V_CASA = cedido ao visitante FORA (adversários CASA contra ele)
#   V_OUTRO = conquistado do visitante CASA (referência complementar)
# -------------------
def build_row_pos(grupo, mandante, visitante):
    out = {
        "RODADA": RODADA_REF,
        "MANDANTE": mandante,
        "VISITANTE": visitante,
        "FILTRO": FILTRO,
        "N_JOGOS": N_JOGOS,
    }

    # base por posição
    df_pos = df[pos_mask(df, grupo)].copy()

    # MEIAS (se quiser depois separar MEIA/VOLANTE, entra aqui)
    if grupo == "MEI":
        df_pos = apply_divisao_meias(df_pos)

    # --------- Mandante ----------
    # conquistado casa
    sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="CASA", mode="TIME")
    agg_sum(sl, "M_CASA", out)

    # cedido em casa (oponentes FORA contra o mandante)
    sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="FORA", mode="ADV")
    agg_sum(sl, "M_FORA", out)

    # referência: conquistado fora
    sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="FORA", mode="TIME")
    agg_sum(sl, "M_OUTRO", out)

    # --------- Visitante ----------
    # conquistado fora
    sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="FORA", mode="TIME")
    agg_sum(sl, "V_FORA", out)

    # cedido fora (oponentes CASA contra o visitante)
    sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="CASA", mode="ADV")
    agg_sum(sl, "V_CASA", out)

    # referência: conquistado casa
    sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="CASA", mode="TIME")
    agg_sum(sl, "V_OUTRO", out)

    return out

# -------------------
# LATERAIS (LD/LE separados)
# - Mantém o mesmo padrão do motor
# - Só que duplica métricas: ..._LD e ..._LE
# -------------------
def build_row_laterais(mandante, visitante):
    base = {
        "RODADA": RODADA_REF,
        "MANDANTE": mandante,
        "VISITANTE": visitante,
        "FILTRO": FILTRO,
        "N_JOGOS": N_JOGOS,
    }

    def add_side(grupo_lat, suf):
        out = {}
        df_pos = df[pos_mask(df, grupo_lat)].copy()

        # Mandante
        sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="CASA", mode="TIME")
        agg_sum(sl, f"M_CASA{suf}", out)

        sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="FORA", mode="ADV")
        agg_sum(sl, f"M_FORA{suf}", out)

        sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="FORA", mode="TIME")
        agg_sum(sl, f"M_OUTRO{suf}", out)

        # Visitante
        sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="FORA", mode="TIME")
        agg_sum(sl, f"V_FORA{suf}", out)

        sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="CASA", mode="ADV")
        agg_sum(sl, f"V_CASA{suf}", out)

        sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="CASA", mode="TIME")
        agg_sum(sl, f"V_OUTRO{suf}", out)

        return out

    # LD e LE
    base.update(add_side("LD", "_LD"))
    base.update(add_side("LE", "_LE"))
    return base

# -------------------
# Rodar rodada inteira
# -------------------
linhas = {
    "TABELA_GOLEIROS": [],
    "TABELA_ZAGUEIROS": [],
    "TABELA_LATERAIS": [],
    "TABELA_MEIAS": [],
    "TABELA_ATACANTES": [],
}

for _, r in df_r.iterrows():
    mand = r["MANDANTE_FIX"]
    vis  = r["VISITANTE_FIX"]

    linhas["TABELA_GOLEIROS"].append(build_row_pos("GOL", mand, vis))
    linhas["TABELA_ZAGUEIROS"].append(build_row_pos("ZAG", mand, vis))
    linhas["TABELA_MEIAS"].append(build_row_pos("MEI", mand, vis))
    linhas["TABELA_ATACANTES"].append(build_row_pos("ATA", mand, vis))

    # laterais separados
    linhas["TABELA_LATERAIS"].append(build_row_laterais(mand, vis))

# -------------------
# Export
# -------------------
if GERAR in ["NAO", "NÃO"]:
    print("ℹ️ GERAR_TABELAS=NAO -> não exportei arquivos. (Mas já tenho as tabelas em memória.)")
else:
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = OUT / f"RUN_2026_R{RODADA_REF:02d}_{FILTRO}_N{N_JOGOS}_{ts}"
    run_dir.mkdir(parents=True, exist_ok=True)

    for nome, rows in linhas.items():
        df_out = pd.DataFrame(rows)
        out_file = run_dir / f"{nome}_R{RODADA_REF:02d}.csv"
        df_out.to_csv(out_file, index=False, encoding="utf-8-sig")
        print(f"✅ Gerado: {out_file} | linhas={len(df_out)}")

    print("\n✅ TABELAS GERADAS COM SUCESSO")
    print(f"📁 Saída: {run_dir}")


✅ Pastas OK
✅ Arquivos OK
PILOTO: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/PILOTO_GERACAO_TABELAS.csv
CONFRONTOS: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/CONFRONTOS_2026_NORMALIZADO.csv
SCOUTS: /content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx
DIVISAO: /content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv
✅ PILOTO: RODADA_REFERENCIA=5 | N_JOGOS=5 | FILTRO=POR_MANDO | GERAR_TABELAS=SIM
✅ Confrontos na rodada 5: 10
✅ Scouts carregado (Por jogo): linhas=12086 | times=20
✅ Gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813/TABELA_GOLEIROS_R05.csv | linhas=10
✅ Gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813/TABELA_ZAGUEIROS_R05.csv | linhas=10
✅ Gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813/TABELA_LATERAIS_R05.csv | linhas=10
✅ Gerado: /content/drive/MyDrive/TCC

In [ ]:
# ==========================
# VALIDADOR (Colab)
# ==========================
# 1) Ajuste RODADA_PASTA para a pasta que contém os CSVs gerados (ex.: "R05")
RODADA_PASTA = "R05"

from pathlib import Path

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INPUT_DIR = BASE/"02_OUTPUT"/RODADA_PASTA
SPEC_JSON = BASE/"03_CONFIG"/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"

# 2) Execute:
!python /content/drive/MyDrive/TCC\ TABELAS\ 2026/03_CONFIG/VALIDADOR_TCC.py --input_dir "{INPUT_DIR}" --spec_json "{SPEC_JSON}"

python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py': [Errno 2] No such file or directory


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG"



BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json  PILOTO_GERACAO_TABELAS.gsheet
PILOTO_GERACAO_TABELAS.csv	      VALIDADOR_TCC.py


In [ ]:
!python "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py" \
  --input_dir "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RODADA_R05_POR_MANDO_N5_20260103_161432" \
  --spec_json "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"



Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py", line 166, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py", line 112, in main
    raise FileNotFoundError(f"input_dir não existe: {input_dir}")
FileNotFoundError: input_dir não existe: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RODADA_R05_POR_MANDO_N5_20260103_161432


In [ ]:
!ls "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT"


CONFRONTOS_2026_NORMALIZADO.csv
RUN_2026_R01_20260102_165655
RUN_2026_R01_20260102_170833_FIX
RUN_2026_R01_20260102_204611
RUN_2026_R01_20260102_204612_FIX
RUN_2026_R01_20260102_210532
RUN_2026_R01_20260102_210532_FIX
RUN_2026_R01_20260102_210933
RUN_2026_R01_20260102_210934_FIX
RUN_2026_R01_20260102_220758
RUN_2026_R01_20260103_002121
RUN_2026_R05_POR_MANDO_N5_20260103_161432
RUN_2026_R05_POR_MANDO_N5_20260103_174813
TESTE_CONFRONTOS_RODADA_01.csv
TESTE_MEIAS_2025_2025-08-10_N5_20260103_025143


In [ ]:
from pathlib import Path
import os, glob, subprocess

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
OUT  = BASE/"02_OUTPUT"
SPEC = BASE/"03_CONFIG"/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"
VAL  = BASE/"03_CONFIG"/"VALIDADOR_TCC.py"

# pega a pasta mais recente que começa com RUN_
runs = sorted([p for p in OUT.glob("RUN_*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
if not runs:
    raise FileNotFoundError(f"Não encontrei pasta RUN_* em: {OUT}")

input_dir = runs[0]
print("✅ Pasta detectada automaticamente:", input_dir)

cmd = ["python", str(VAL), "--input_dir", str(input_dir), "--spec_json", str(SPEC)]
print("▶️ Rodando:", " ".join(cmd))
subprocess.run(cmd, check=False)


✅ Pasta detectada automaticamente: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813
▶️ Rodando: python /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py --input_dir /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813 --spec_json /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json


CompletedProcess(args=['python', '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py', '--input_dir', '/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813', '--spec_json', '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json'], returncode=2)

In [ ]:
from pathlib import Path
import glob, os, subprocess, textwrap

# 0) Drive
from google.colab import drive
drive.mount("/content/drive")

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

VAL  = CFG/"VALIDADOR_TCC.py"
SPEC = CFG/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"

# 1) valida arquivos do validador
assert VAL.exists(), f"Não achei o validador: {VAL}"
assert SPEC.exists(), f"Não achei o spec: {SPEC}"
assert OUT.exists(), f"Não achei a pasta OUT: {OUT}"

# 2) pega a RUN mais recente automaticamente
runs = sorted(
    [Path(p) for p in glob.glob(str(OUT/"RUN_*")) if Path(p).is_dir()],
    key=lambda p: p.stat().st_mtime,
    reverse=True
)

if not runs:
    raise FileNotFoundError(f"Não encontrei nenhuma pasta RUN_* em: {OUT}")

input_dir = runs[0]
print("✅ RUN detectada automaticamente:")
print("   ", input_dir)

# 3) roda o validador capturando TUDO (stdout/stderr)
cmd = ["python3", str(VAL), "--input_dir", str(input_dir), "--spec_json", str(SPEC)]
print("\n▶ Rodando:\n", " ".join([f'"{c}"' if " " in c else c for c in cmd]))

res = subprocess.run(cmd, capture_output=True, text=True)
print("\n===== STDOUT =====\n", res.stdout if res.stdout else "(vazio)")
print("\n===== STDERR =====\n", res.stderr if res.stderr else "(vazio)")
print("\n✅ Return code:", res.returncode)

# 4) interpretação rápida
if res.returncode == 0:
    print("\n✅ VALIDAÇÃO PASSOU. Você pode seguir para a IA de layout com segurança.")
else:
    print("\n❌ VALIDAÇÃO FALHOU. O motivo está no STDERR acima (ordem/colunas faltando/colunas extras/etc.).")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ RUN detectada automaticamente:
    /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813

▶ Rodando:
 python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py" --input_dir "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813" --spec_json "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"

===== STDOUT =====
 
VALIDAÇÃO DE COLUNAS
Pasta: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813
Report: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_174813/REPORT_VALIDACAO_COLUNAS.csv
Tabelas validadas: 5
Falhas: 4

Arquivos com FALHA:
- TABELA_ATACANTES_R05.csv (ATACANTES) | missing=[M_CASA_F; M_OUTRO_F; V_FORA_F; V_OUTRO_F] extra=[M_CASA_DE; M_CASA_FD; M

In [ ]:
from pathlib import Path
import subprocess, pandas as pd

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

VAL  = CFG/"VALIDADOR_TCC.py"
SPEC = CFG/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"

assert OUT.exists(), f"OUT não existe: {OUT}"
assert VAL.exists(), f"Validador não existe: {VAL}"
assert SPEC.exists(), f"Spec JSON não existe: {SPEC}"

runs = sorted([p for p in OUT.glob("RUN_*") if p.is_dir()],
              key=lambda p: p.stat().st_mtime, reverse=True)

if not runs:
    raise FileNotFoundError(f"Não encontrei pastas RUN_* em: {OUT}")

print(f"🔎 Encontrei {len(runs)} pastas RUN_. Vou validar da mais recente para a mais antiga...\n")

resultados = []
aprovada = None

for p in runs:
    cmd = ["python3", str(VAL), "--input_dir", str(p), "--spec_json", str(SPEC)]
    r = subprocess.run(cmd, capture_output=True, text=True)
    ok = (r.returncode == 0)
    resultados.append({"RUN": p.name, "OK": ok, "PATH": str(p)})
    status = "✅ OK" if ok else "❌ FALHOU"
    print(f"{status}  {p.name}")
    if ok:
        aprovada = p
        break

df = pd.DataFrame(resultados)
display(df)

if aprovada:
    print("\n🏁 APROVADA:", aprovada)
    print("➡️ Essa é a pasta que você deve usar para a IA do layout.")
else:
    print("\n❌ Nenhuma RUN passou 100%.")
    print("➡️ Nesse caso, o motor que gerou as RUNs não está obedecendo o BLOCO OFICIAL (spec).")
    print("➡️ Aí o próximo passo é eu alinhar MOTOR x SPEC (sem você mexer em nada).")


🔎 Encontrei 12 pastas RUN_. Vou validar da mais recente para a mais antiga...

❌ FALHOU  RUN_2026_R05_POR_MANDO_N5_20260103_174813
❌ FALHOU  RUN_2026_R05_POR_MANDO_N5_20260103_161432
❌ FALHOU  RUN_2026_R01_20260103_002121
❌ FALHOU  RUN_2026_R01_20260102_220758
❌ FALHOU  RUN_2026_R01_20260102_210934_FIX
❌ FALHOU  RUN_2026_R01_20260102_210933
❌ FALHOU  RUN_2026_R01_20260102_210532
❌ FALHOU  RUN_2026_R01_20260102_210532_FIX
❌ FALHOU  RUN_2026_R01_20260102_204612_FIX
❌ FALHOU  RUN_2026_R01_20260102_204611
❌ FALHOU  RUN_2026_R01_20260102_170833_FIX
❌ FALHOU  RUN_2026_R01_20260102_165655


,RUN,OK,PATH
0,RUN_2026_R05_POR_MANDO_N5_20260103_174813,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...
1,RUN_2026_R05_POR_MANDO_N5_20260103_161432,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...
2,RUN_2026_R01_20260103_002121,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...
3,RUN_2026_R01_20260102_220758,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...
4,RUN_2026_R01_20260102_210934_FIX,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...
5,RUN_2026_R01_20260102_210933,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...
6,RUN_2026_R01_20260102_210532,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...
7,RUN_2026_R01_20260102_210532_FIX,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...
8,RUN_2026_R01_20260102_204612_FIX,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...
9,RUN_2026_R01_20260102_204611,False,/content/drive/MyDrive/TCC TABELAS 2026/02_OUT...



❌ Nenhuma RUN passou 100%.
➡️ Nesse caso, o motor que gerou as RUNs não está obedecendo o BLOCO OFICIAL (spec).
➡️ Aí o próximo passo é eu alinhar MOTOR x SPEC (sem você mexer em nada).


In [ ]:
from pathlib import Path
import subprocess

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
OUT  = BASE/"02_OUTPUT"
SPEC = BASE/"03_CONFIG"/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"
VAL  = BASE/"03_CONFIG"/"VALIDADOR_TCC.py"

# 0) Conferência de arquivos
assert OUT.exists(), f"Pasta não encontrada: {OUT}"
assert SPEC.exists(), f"Spec não encontrado: {SPEC}"
assert VAL.exists(), f"Validador não encontrado: {VAL}"

# 1) Pega a RUN mais recente (por data de modificação)
runs = sorted([p for p in OUT.glob("RUN_*") if p.is_dir()],
              key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"Nenhuma pasta RUN_* encontrada em: {OUT}"

input_dir = runs[0]
print("✅ RUN detectada automaticamente:")
print("   ", input_dir)

# 2) Executa o validador
cmd = ["python3", str(VAL), "--input_dir", str(input_dir), "--spec_json", str(SPEC)]
print("\n▶ Rodando:\n", " ".join([f'"{c}"' if " " in c else c for c in cmd]), "\n")

proc = subprocess.run(cmd, capture_output=True, text=True)

print("===== STDOUT =====\n", proc.stdout)
print("===== STDERR =====\n", proc.stderr if proc.stderr else "(vazio)")
print("✅ Return code:", proc.returncode)

if proc.returncode != 0:
    raise SystemExit("❌ VALIDAÇÃO FALHOU. Veja acima (missing/extra/ordem).")
else:
    print("✅ VALIDAÇÃO OK (colunas e ordem conferidas).")


✅ RUN detectada automaticamente:
    /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_20260102_165655

▶ Rodando:
 python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py" --input_dir "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_20260102_165655" --spec_json "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json" 

===== STDOUT =====
 
VALIDAÇÃO DE COLUNAS
Pasta: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_20260102_165655
Report: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_20260102_165655/REPORT_VALIDACAO_COLUNAS.csv
Tabelas validadas: 5
Falhas: 5

Arquivos com FALHA:
- TABELA_ATACANTES_R01.csv (ATACANTES) | missing=[RODADA; MANDANTE; VISITANTE; FILTRO; N_JOGOS; M_CASA_FS; M_CASA_DS; M_CASA_F; M_CASA_I; M_CASA_FC; M_CASA_PP; M_CASA_A; M_CASA_G; M_CASA_FF; M_CASA_PS; M_CASA_PI; M_CASA_JOGOS_ENCONTRADOS; M_CASA_PTS_TOTAL; M_CASA_MEDIA_BASICA; M_OUTRO_FS; M_OUTRO_DS; 

SystemExit: ❌ VALIDAÇÃO FALHOU. Veja acima (missing/extra/ordem).

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

from pathlib import Path
import json

CFG = Path("/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG")
CFG.mkdir(parents=True, exist_ok=True)

spec = {
  "GOLEIROS": [
    "RODADA","MANDANTE","VISITANTE","FILTRO","N_JOGOS",
    "M_CASA_FS","M_CASA_FF","M_CASA_G","M_CASA_GC","M_CASA_DD","M_CASA_SG","M_CASA_%DD",
    "M_OUTRO_FS","M_OUTRO_FF","M_OUTRO_G","M_OUTRO_GC","M_OUTRO_DD","M_OUTRO_SG","M_OUTRO_%DD",
    "CASA","FORA",
    "V_FORA_DD","V_FORA_SG","V_FORA_%DD","V_FORA_FS","V_FORA_FF","V_FORA_G",
    "V_OUTRO_DD","V_OUTRO_SG","V_OUTRO_%DD","V_OUTRO_FS","V_OUTRO_FF","V_OUTRO_G"
  ],
  "ZAGUEIROS": [
    "RODADA","MANDANTE","VISITANTE","FILTRO","N_JOGOS",
    "M_CASA_SG","M_CASA_DE","M_CASA_CHUTES","M_CASA_PTS","M_CASA_BASICA",
    "M_OUTRO_SG","M_OUTRO_DE","M_OUTRO_CHUTES","M_OUTRO_PTS","M_OUTRO_BASICA",
    "CASA","FORA",
    "V_FORA_BASICA","V_FORA_PTS","V_FORA_CHUTES","V_FORA_DE","V_FORA_SG",
    "V_OUTRO_BASICA","V_OUTRO_PTS","V_OUTRO_CHUTES","V_OUTRO_DE","V_OUTRO_SG"
  ],
  "LATERAIS": [
    "RODADA","MANDANTE","VISITANTE","FILTRO","N_JOGOS",
    "M_CASA_FS_LE","M_CASA_DS_LE","M_CASA_F_LE","M_CASA_I_LE","M_CASA_FC_LE","M_CASA_PP_LE","M_CASA_A_LE",
    "M_CASA_G_LE","M_CASA_FF_LE","M_CASA_PS_LE","M_CASA_PI_LE","M_CASA_PTS_LE","M_CASA_BASICA_LE",
    "M_CASA_FS_LD","M_CASA_DS_LD","M_CASA_F_LD","M_CASA_I_LD","M_CASA_FC_LD","M_CASA_PP_LD","M_CASA_A_LD",
    "M_CASA_G_LD","M_CASA_FF_LD","M_CASA_PS_LD","M_CASA_PI_LD","M_CASA_PTS_LD","M_CASA_BASICA_LD",
    "M_OUTRO_FS_LE","M_OUTRO_DS_LE","M_OUTRO_F_LE","M_OUTRO_I_LE","M_OUTRO_FC_LE","M_OUTRO_PP_LE","M_OUTRO_A_LE",
    "M_OUTRO_G_LE","M_OUTRO_FF_LE","M_OUTRO_PS_LE","M_OUTRO_PI_LE","M_OUTRO_PTS_LE","M_OUTRO_BASICA_LE",
    "M_OUTRO_FS_LD","M_OUTRO_DS_LD","M_OUTRO_F_LD","M_OUTRO_I_LD","M_OUTRO_FC_LD","M_OUTRO_PP_LD","M_OUTRO_A_LD",
    "M_OUTRO_G_LD","M_OUTRO_FF_LD","M_OUTRO_PS_LD","M_OUTRO_PI_LD","M_OUTRO_PTS_LD","M_OUTRO_BASICA_LD",
    "CASA","FORA",
    "V_FORA_BASICA_LD","V_FORA_PTS_LD","V_FORA_PI_LD","V_FORA_PS_LD","V_FORA_FF_LD","V_FORA_G_LD","V_FORA_A_LD",
    "V_FORA_PP_LD","V_FORA_FC_LD","V_FORA_I_LD","V_FORA_F_LD","V_FORA_DS_LD","V_FORA_FS_LD",
    "V_FORA_BASICA_LE","V_FORA_PTS_LE","V_FORA_PI_LE","V_FORA_PS_LE","V_FORA_FF_LE","V_FORA_G_LE","V_FORA_A_LE",
    "V_FORA_PP_LE","V_FORA_FC_LE","V_FORA_I_LE","V_FORA_F_LE","V_FORA_DS_LE","V_FORA_FS_LE",
    "V_OUTRO_BASICA_LD","V_OUTRO_PTS_LD","V_OUTRO_PI_LD","V_OUTRO_PS_LD","V_OUTRO_FF_LD","V_OUTRO_G_LD","V_OUTRO_A_LD",
    "V_OUTRO_PP_LD","V_OUTRO_FC_LD","V_OUTRO_I_LD","V_OUTRO_F_LD","V_OUTRO_DS_LD","V_OUTRO_FS_LD",
    "V_OUTRO_BASICA_LE","V_OUTRO_PTS_LE","V_OUTRO_PI_LE","V_OUTRO_PS_LE","V_OUTRO_FF_LE","V_OUTRO_G_LE","V_OUTRO_A_LE",
    "V_OUTRO_PP_LE","V_OUTRO_FC_LE","V_OUTRO_I_LE","V_OUTRO_F_LE","V_OUTRO_DS_LE","V_OUTRO_FS_LE"
  ],
  "MEIAS": [
    "RODADA","MANDANTE","VISITANTE","FILTRO","N_JOGOS",
    "M_CASA_PTS","M_CASA_BASICA","M_CASA_F","M_CASA_FS","M_CASA_FF","M_CASA_A","M_CASA_G",
    "M_CASA_PS","M_CASA_DS","M_CASA_FC","M_CASA_I","M_CASA_PI","M_CASA_PP",
    "M_OUTRO_PTS","M_OUTRO_BASICA","M_OUTRO_F","M_OUTRO_FS","M_OUTRO_FF","M_OUTRO_A","M_OUTRO_G",
    "M_OUTRO_PS","M_OUTRO_DS","M_OUTRO_FC","M_OUTRO_I","M_OUTRO_PI","M_OUTRO_PP",
    "CASA","FORA",
    "V_FORA_PP","V_FORA_PI","V_FORA_I","V_FORA_FC","V_FORA_DS","V_FORA_PS","V_FORA_G",
    "V_FORA_A","V_FORA_FF","V_FORA_FS","V_FORA_F","V_FORA_BASICA","V_FORA_PTS",
    "V_OUTRO_PP","V_OUTRO_PI","V_OUTRO_I","V_OUTRO_FC","V_OUTRO_DS","V_OUTRO_PS","V_OUTRO_G",
    "V_OUTRO_A","V_OUTRO_FF","V_OUTRO_FS","V_OUTRO_F","V_OUTRO_BASICA","V_OUTRO_PTS"
  ],
  "ATACANTES": [
    "RODADA","MANDANTE","VISITANTE","FILTRO","N_JOGOS",
    "M_CASA_PTS","M_CASA_BASICA","M_CASA_F","M_CASA_FS","M_CASA_FF","M_CASA_A","M_CASA_G",
    "M_CASA_PS","M_CASA_DS","M_CASA_FC","M_CASA_I","M_CASA_PI","M_CASA_PP",
    "M_OUTRO_PTS","M_OUTRO_BASICA","M_OUTRO_F","M_OUTRO_FS","M_OUTRO_FF","M_OUTRO_A","M_OUTRO_G",
    "M_OUTRO_PS","M_OUTRO_DS","M_OUTRO_FC","M_OUTRO_I","M_OUTRO_PI","M_OUTRO_PP",
    "CASA","FORA",
    "V_FORA_PP","V_FORA_PI","V_FORA_I","V_FORA_FC","V_FORA_DS","V_FORA_PS","V_FORA_G",
    "V_FORA_A","V_FORA_FF","V_FORA_FS","V_FORA_F","V_FORA_BASICA","V_FORA_PTS",
    "V_OUTRO_PP","V_OUTRO_PI","V_OUTRO_I","V_OUTRO_FC","V_OUTRO_DS","V_OUTRO_PS","V_OUTRO_G",
    "V_OUTRO_A","V_OUTRO_FF","V_OUTRO_FS","V_OUTRO_F","V_OUTRO_BASICA","V_OUTRO_PTS"
  ]
}

out_path = CFG/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"
out_path.write_text(json.dumps(spec, ensure_ascii=False, indent=2), encoding="utf-8")
print("✅ BLOCO OFICIAL sobrescrito em:", out_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ BLOCO OFICIAL sobrescrito em: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json


In [ ]:
from pathlib import Path
import pandas as pd
import subprocess

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"
VAL  = CFG/"VALIDADOR_TCC.py"
SPEC = CFG/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"

# pega a RUN mais recente que tenha "RODADA" nas tabelas (isso evita as RUNs antigas HOME/AWAY)
runs = sorted([p for p in OUT.glob("RUN_*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
ok_run = None

for r in runs:
    f = r/"TABELA_MEIAS_R01.csv"
    if not f.exists():
        # tenta qualquer tabela
        any_csv = list(r.glob("TABELA_*.csv"))
        if not any_csv:
            continue
        f = any_csv[0]
    try:
        cols = list(pd.read_csv(f, nrows=1).columns)
        if "RODADA" in cols and "MANDANTE" in cols and "VISITANTE" in cols:
            ok_run = r
            break
    except Exception:
        continue

if ok_run is None:
    raise FileNotFoundError("Não encontrei nenhuma RUN recente no formato novo (com RODADA/MANDANTE/VISITANTE).")

print("✅ RUN escolhida para validação:")
print("   ", ok_run)

cmd = ["python3", str(VAL), "--input_dir", str(ok_run), "--spec_json", str(SPEC)]
print("\n▶ Rodando:\n ", " ".join(cmd), "\n")
subprocess.run(cmd, check=False)


✅ RUN escolhida para validação:
    /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_161432

▶ Rodando:
  python3 /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py --input_dir /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_161432 --spec_json /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json 



CompletedProcess(args=['python3', '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py', '--input_dir', '/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_161432', '--spec_json', '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json'], returncode=2)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

from pathlib import Path
import pandas as pd
import subprocess
import glob

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

VAL  = CFG/"VALIDADOR_TCC.py"

# ✅ IMPORTANTE: use o SPEC que está NO DRIVE agora
# (se você tiver o v2 salvo como arquivo separado, ele precisa ter este nome abaixo)
SPEC = CFG/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"

# 1) Seleciona a RUN mais recente que tenha colunas RODADA/MANDANTE/VISITANTE
runs = sorted([p for p in OUT.glob("RUN_*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
ok_run = None

for r in runs:
    csvs = list(r.glob("TABELA_*.csv"))
    if not csvs:
        continue
    try:
        cols = list(pd.read_csv(csvs[0], nrows=1).columns)
        if {"RODADA","MANDANTE","VISITANTE"}.issubset(set(cols)):
            ok_run = r
            break
    except Exception:
        continue

if ok_run is None:
    raise FileNotFoundError("Não encontrei nenhuma RUN no formato novo (com RODADA/MANDANTE/VISITANTE).")

print("✅ RUN escolhida para validação:")
print("   ", ok_run)

# 2) Roda validador CAPTURANDO saída (para você ver o motivo)
cmd = ["python3", str(VAL), "--input_dir", str(ok_run), "--spec_json", str(SPEC)]
print("\n▶ Comando:")
print(" ", " ".join(cmd))

res = subprocess.run(cmd, capture_output=True, text=True)

print("\n===== STDOUT =====")
print(res.stdout if res.stdout.strip() else "(vazio)")

print("\n===== STDERR =====")
print(res.stderr if res.stderr.strip() else "(vazio)")

print("\n✅ Return code:", res.returncode)

# 3) Abre o report se existir (ele sempre cria quando executa até o fim do check)
report = ok_run/"REPORT_VALIDACAO_COLUNAS.csv"
if report.exists():
    df_rep = pd.read_csv(report)
    print("\n📄 REPORT_VALIDACAO_COLUNAS.csv (primeiras linhas):")
    display(df_rep.head(20))
else:
    print("\n⚠️ Report não encontrado em:", report)
    print("   Isso normalmente indica que o script abortou antes de gerar o report.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ RUN escolhida para validação:
    /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_161432

▶ Comando:
  python3 /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py --input_dir /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_161432 --spec_json /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json

===== STDOUT =====

VALIDAÇÃO DE COLUNAS
Pasta: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_161432
Report: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R05_POR_MANDO_N5_20260103_161432/REPORT_VALIDACAO_COLUNAS.csv
Tabelas validadas: 5
Falhas: 5

Arquivos com FALHA:
- TABELA_ATACANTES_R05.csv (ATACANTES) | missing=[M_CASA_PTS; M_CASA_BASICA; M_OUTRO_PTS; M_OUTRO_BASICA; CASA; FORA; V_FORA_BASI

,arquivo,posicao,status,missing,extra,mismatch_index,expected_len,actual_len
0,TABELA_ATACANTES_R05.csv,ATACANTES,FALHA,M_CASA_PTS; M_CASA_BASICA; M_OUTRO_PTS; M_OUTR...,M_CASA_JOGOS_ENCONTRADOS; M_CASA_PTS_TOTAL; M_...,len,59,61
1,TABELA_GOLEIROS_R05.csv,GOLEIROS,FALHA,M_CASA_FS; M_CASA_FF; M_CASA_G; M_CASA_DD; M_C...,M_CASA_DE; M_CASA_FD; M_CASA_GS; M_CASA_CA; M_...,len,33,45
2,TABELA_LATERAIS_R05.csv,LATERAIS,FALHA,M_CASA_FS_LE; M_CASA_DS_LE; M_CASA_F_LE; M_CAS...,M_CASA_FS; M_CASA_DS; M_CASA_F; M_CASA_I; M_CA...,len,111,61
3,TABELA_MEIAS_R05.csv,MEIAS,FALHA,M_CASA_PTS; M_CASA_BASICA; M_OUTRO_PTS; M_OUTR...,M_CASA_JOGOS_ENCONTRADOS; M_CASA_PTS_TOTAL; M_...,len,59,61
4,TABELA_ZAGUEIROS_R05.csv,ZAGUEIROS,FALHA,M_CASA_CHUTES; M_CASA_PTS; M_CASA_BASICA; M_OU...,M_CASA_FS; M_CASA_DS; M_CASA_F; M_CASA_I; M_CA...,len,27,61


In [ ]:
# ==========================
# ACABAMENTO FINAL (ALINHAR + REEXPORTAR)
# Rode APÓS gerar as tabelas
# ==========================

import pandas as pd
from pathlib import Path
import json
import re

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

PILOTO_PATH = CFG/"PILOTO_GERACAO_TABELAS.csv"
SPEC_PATH   = CFG/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"

def norm_txt(x):
    if pd.isna(x):
        return ""
    return str(x).strip().upper()

def load_piloto(path):
    df = pd.read_csv(path, dtype=str, encoding="utf-8-sig")
    df.columns = [c.replace("\ufeff","").strip().upper() for c in df.columns]
    if not set(["CHAVE","VALOR"]).issubset(set(df.columns)):
        raise ValueError(f"PILOTO inválido. Precisa ter CHAVE e VALOR. Veio: {list(df.columns)}")
    df["CHAVE"] = df["CHAVE"].map(norm_txt)
    df["VALOR"] = df["VALOR"].astype(str).str.strip()
    return dict(zip(df["CHAVE"], df["VALOR"]))

def detect_latest_run(output_dir: Path):
    runs = [p for p in output_dir.iterdir() if p.is_dir() and p.name.startswith("RUN_")]
    if not runs:
        raise FileNotFoundError("Não achei nenhuma pasta RUN_ dentro de 02_OUTPUT.")
    runs.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    return runs[0]

# 1) localizar RUN mais recente
run_dir = detect_latest_run(OUT)
print(f"✅ RUN detectada: {run_dir}")

# 2) carregar piloto (para preencher metadados)
piloto = load_piloto(PILOTO_PATH)
rodada = piloto.get("RODADA_ATUAL") or piloto.get("RODADA") or ""
filtro = piloto.get("FILTRO") or ""
n_jogos = piloto.get("N_JOGOS") or ""
rodada = str(rodada).strip()
filtro = str(filtro).strip().upper()
n_jogos = str(n_jogos).strip()

print(f"✅ PILOTO: RODADA={rodada} | FILTRO={filtro} | N_JOGOS={n_jogos}")

# 3) carregar bloco oficial
with open(SPEC_PATH, "r", encoding="utf-8") as f:
    spec = json.load(f)

# mapeamento arquivo -> chave do spec
file_map = {
    "TABELA_GOLEIROS": "GOLEIROS",
    "TABELA_ZAGUEIROS": "ZAGUEIROS",
    "TABELA_LATERAIS": "LATERAIS",
    "TABELA_MEIAS": "MEIAS",
    "TABELA_ATACANTES": "ATACANTES",
}

# 4) aplicar acabamento em cada CSV
falhas = []
for prefix, pos_key in file_map.items():
    # achar arquivo da rodada (qualquer sufixo Rxx)
    files = list(run_dir.glob(f"{prefix}_R*.csv"))
    if not files:
        falhas.append(f"Não achei {prefix}_R*.csv em {run_dir}")
        continue

    path = files[0]
    df = pd.read_csv(path)

    # normalizar nomes de colunas (trim)
    df.columns = [str(c).strip() for c in df.columns]

    # renomear HOME/AWAY se tiverem aparecido (bug comum)
    rename_hw = {}
    if "HOME" in df.columns and "MANDANTE" not in df.columns:
        rename_hw["HOME"] = "MANDANTE"
    if "AWAY" in df.columns and "VISITANTE" not in df.columns:
        rename_hw["AWAY"] = "VISITANTE"
    if rename_hw:
        df.rename(columns=rename_hw, inplace=True)

    # injetar metadados se faltarem
    if "RODADA" not in df.columns:
        df["RODADA"] = rodada
    if "FILTRO" not in df.columns:
        df["FILTRO"] = filtro
    if "N_JOGOS" not in df.columns:
        df["N_JOGOS"] = n_jogos

    # garantir MANDANTE/VISITANTE existam (se não existirem, mantém e o validador acusa)
    # (idealmente eles já existem pelo gerador)
    expected = spec[pos_key]

    # criar colunas faltantes como vazio (para não quebrar export)
    for c in expected:
        if c not in df.columns:
            df[c] = ""

    # reduzir e reordenar exatamente como o bloco oficial
    df_final = df[expected].copy()

    # sobrescrever CSV
    df_final.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"✅ Ajustado: {path.name} ({pos_key}) | colunas={len(df_final.columns)}")

if falhas:
    print("\n❌ Problemas encontrados:")
    for e in falhas:
        print("-", e)
else:
    print("\n✅ ACABAMENTO FINAL concluído. Agora rode o VALIDADOR novamente.")


FileNotFoundError: Não achei nenhuma pasta RUN_ dentro de 02_OUTPUT.

In [ ]:
# ==========================
# MOTOR FINAL - TCC (2026)
# Gera 1 CSV por posição por rodada (1 linha = 1 confronto)
# Laterais separados em LD (2.2) e LE (2.6)
# Sem DATA_CORTE (produção)
# ==========================

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# -------------------
# Paths (seu padrão)
# -------------------
BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
INP  = BASE/"01_INPUT"
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

PILOTO_PATH     = CFG/"PILOTO_GERACAO_TABELAS.csv"
SCOUTS_PATH     = INP/"Scouts_Reorganizado.xlsx"
DIVISAO_PATH    = INP/"DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv"
CONFRONTOS_PATH = OUT/"CONFRONTOS_2026_NORMALIZADO.csv"   # (você já gerou esse)

# -------------------
# Helpers
# -------------------
def norm_txt(x):
    if pd.isna(x): return ""
    return str(x).strip().upper()

def strip_bom(s):
    return s.replace("\ufeff", "").strip()

def pick_col(df_cols, candidates):
    up = {strip_bom(str(c)).upper(): c for c in df_cols}
    for cand in candidates:
        key = strip_bom(cand).upper()
        if key in up:
            return up[key]
    return None

def load_piloto(p):
    df = pd.read_csv(p, dtype=str, encoding="utf-8-sig")
    df.columns = [strip_bom(c).upper() for c in df.columns]

    if set(df.columns) >= {"CHAVE","VALOR"}:
        df["CHAVE"] = df["CHAVE"].map(norm_txt)
        df["VALOR"] = df["VALOR"].astype(str).str.strip()
        d = dict(zip(df["CHAVE"], df["VALOR"]))
        return d, df

    raise ValueError(f"❌ PILOTO inválido. Precisa ter colunas CHAVE e VALOR. Veio: {list(df.columns)}")

def ensure_drive():
    try:
        from google.colab import drive
        if not Path("/content/drive").exists():
            drive.mount("/content/drive")
    except Exception:
        pass

# -------------------
# Start
# -------------------
ensure_drive()

# checks
for p in [INP, OUT, CFG]:
    if not p.exists():
        raise FileNotFoundError(f"❌ Pasta ausente: {p}")

for f in [PILOTO_PATH, SCOUTS_PATH, DIVISAO_PATH, CONFRONTOS_PATH]:
    if not f.exists():
        raise FileNotFoundError(f"❌ Arquivo ausente: {f}")

print("✅ Pastas OK")
print("✅ Arquivos OK")
print("PILOTO:", PILOTO_PATH)
print("CONFRONTOS:", CONFRONTOS_PATH)
print("SCOUTS:", SCOUTS_PATH)
print("DIVISAO:", DIVISAO_PATH)

# -------------------
# PILOTO
# -------------------
piloto, df_piloto = load_piloto(PILOTO_PATH)

RODADA_REF = int(piloto.get("RODADA_REFERENCIA", "0"))
N_JOGOS    = int(piloto.get("N_JOGOS", "5"))
FILTRO     = norm_txt(piloto.get("FILTRO", "TODAS"))
GERAR      = norm_txt(piloto.get("GERAR_TABELAS", "SIM"))

if RODADA_REF <= 0:
    raise ValueError("❌ PILOTO: RODADA_REFERENCIA precisa ser > 0")
if N_JOGOS < 0:
    raise ValueError("❌ PILOTO: N_JOGOS não pode ser negativo")
if FILTRO not in ["TODAS", "POR_MANDO"]:
    raise ValueError("❌ PILOTO: FILTRO deve ser TODAS ou POR_MANDO")
if GERAR not in ["SIM", "NAO", "NÃO"]:
    raise ValueError("❌ PILOTO: GERAR_TABELAS deve ser SIM ou NAO")

print(f"✅ PILOTO: RODADA_REFERENCIA={RODADA_REF} | N_JOGOS={N_JOGOS} | FILTRO={FILTRO} | GERAR_TABELAS={GERAR}")

# -------------------
# Confrontos
# -------------------
df_conf = pd.read_csv(CONFRONTOS_PATH, dtype=str, encoding="utf-8-sig")
df_conf.columns = [strip_bom(c).upper() for c in df_conf.columns]

COL_ROD = pick_col(df_conf.columns, ["RODADA"])
COL_MAN = pick_col(df_conf.columns, ["MANDANTE"])
COL_VIS = pick_col(df_conf.columns, ["VISITANTE"])

if not all([COL_ROD, COL_MAN, COL_VIS]):
    raise ValueError(f"❌ Confrontos precisa ter RODADA, MANDANTE, VISITANTE. Veio: {list(df_conf.columns)}")

df_conf[COL_ROD] = pd.to_numeric(df_conf[COL_ROD], errors="coerce")
df_conf = df_conf[df_conf[COL_ROD].notna()].copy()
df_conf[COL_ROD] = df_conf[COL_ROD].astype(int)

# normaliza times
df_conf["MANDANTE_FIX"] = df_conf[COL_MAN].map(norm_txt)
df_conf["VISITANTE_FIX"] = df_conf[COL_VIS].map(norm_txt)

df_r = df_conf[df_conf[COL_ROD] == RODADA_REF].copy()
print(f"✅ Confrontos na rodada {RODADA_REF}: {len(df_r)}")
if df_r.empty:
    raise ValueError("❌ Nenhum confronto encontrado para a RODADA_REFERENCIA.")

# -------------------
# Scouts (aba Por jogo)
# -------------------
xls = pd.ExcelFile(SCOUTS_PATH)
sheet_name = None
for s in xls.sheet_names:
    if norm_txt(s) in ["POR JOGO", "POR_JOGO", "PORJOGO"]:
        sheet_name = s
        break
if sheet_name is None:
    for s in xls.sheet_names:
        if "JOGO" in norm_txt(s):
            sheet_name = s
            break
if sheet_name is None:
    raise ValueError(f"❌ Não achei a aba Por jogo. Abas: {xls.sheet_names}")

df = pd.read_excel(SCOUTS_PATH, sheet_name=sheet_name)
df.columns = [strip_bom(str(c)) for c in df.columns]

COL_TIME  = pick_col(df.columns, ["TIME", "CLUBE"])
COL_ADV   = pick_col(df.columns, ["ADVERSÁRIO", "ADVERSARIO", "OPONENTE"])
COL_MANDO = pick_col(df.columns, ["MANDO"])
COL_DATA  = pick_col(df.columns, ["DATA", "DATE"])
COL_POSR  = pick_col(df.columns, ["POSREAL", "POSREAL ", "POSREAL\t", "POSREAL\n", "POSREAL\r"])
# (nome do jogador só é necessário se você for auditar manualmente; não precisamos pra agregação)
# COL_NOME = pick_col(df.columns, ["NOME", "JOGADOR"])

need = [COL_TIME, COL_ADV, COL_MANDO, COL_DATA, COL_POSR]
if not all(need):
    raise ValueError(f"❌ Base Por jogo precisa ter TIME, ADVERSÁRIO, MANDO, DATA, Posreal. Detectei: TIME={COL_TIME}, ADV={COL_ADV}, MANDO={COL_MANDO}, DATA={COL_DATA}, POSREAL={COL_POSR}")

df["TIME_FIX"] = df[COL_TIME].map(norm_txt)
df["ADV_FIX"]  = df[COL_ADV].map(norm_txt)

def norm_mando(x):
    x = norm_txt(x)
    if x in ["C", "CASA", "HOME"]: return "CASA"
    if x in ["F", "FORA", "AWAY"]: return "FORA"
    return x

df["MANDO_FIX"] = df[COL_MANDO].map(norm_mando)
df["DATA_FIX"]  = pd.to_datetime(df[COL_DATA], errors="coerce", dayfirst=True)

# Posreal pode vir como 2.2 / 2.6 etc
df["POSREAL"] = pd.to_numeric(df[COL_POSR], errors="coerce")

df = df[df["DATA_FIX"].notna()].copy()
print(f"✅ Scouts carregado ({sheet_name}): linhas={len(df)} | times={df['TIME_FIX'].nunique()}")

# -------------------
# Divisão meias/volantes (apenas para separar MEIA vs VOLANTE)
# -------------------
df_div = pd.read_csv(DIVISAO_PATH, dtype=str, encoding="utf-8-sig")
df_div.columns = [strip_bom(c).upper() for c in df_div.columns]

# aceita GRUPO ou POS_GRUPO (seu arquivo tem POS_GRUPO)
COL_DTIME = pick_col(df_div.columns, ["TIME"])
COL_DJOG  = pick_col(df_div.columns, ["JOGADOR", "NOME"])
COL_DGRP  = pick_col(df_div.columns, ["GRUPO", "POS_GRUPO"])

if not all([COL_DTIME, COL_DJOG, COL_DGRP]):
    raise ValueError(f"❌ DIVISÃO precisa ter TIME, JOGADOR, (GRUPO ou POS_GRUPO). Veio: {list(df_div.columns)}")

df_div["TIME_FIX"] = df_div[COL_DTIME].map(norm_txt)
df_div["JOG_FIX"]  = df_div[COL_DJOG].map(norm_txt)
df_div["GRUPO_FIX"]= df_div[COL_DGRP].map(norm_txt)

# -------------------
# Scouts numéricos que o motor exporta (usa o que existir)
# -------------------
SCOUTS_LIST = ["FS","DS","F","I","FC","PP","A","G","FF","PS","PI","DE","FD","GS","CA","PC"]
# + pontos/básica (se existir)
COL_PTS    = pick_col(df.columns, ["Pts", "PTS", "PONTOS", "PONTUAÇÃO", "PONTUACAO"])
COL_BASICA = pick_col(df.columns, ["Básica", "BASICA", "BÁSICA"])

# garante numérico
for s in SCOUTS_LIST:
    c = pick_col(df.columns, [s])
    if c:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

if COL_PTS:
    df[COL_PTS] = pd.to_numeric(df[COL_PTS], errors="coerce").fillna(0)
if COL_BASICA:
    df[COL_BASICA] = pd.to_numeric(df[COL_BASICA], errors="coerce").fillna(0)

# -------------------
# Recorte: últimos N jogos por TIME, com filtro de mando
# (1 jogo = 1 DATA_FIX; como a tabela é por jogador, pegamos as últimas N datas únicas do time)
# -------------------
def slice_last_n(df_base, time_fix, n, filtro, mando=None, mode="TIME"):
    """
    mode="TIME": filtra linhas do próprio time (conquistado)
    mode="ADV":  filtra linhas dos adversários contra esse time (cedido) usando ADV_FIX == time
    """
    if mode == "TIME":
        dft = df_base[df_base["TIME_FIX"]==time_fix].copy()
    else:
        dft = df_base[df_base["ADV_FIX"]==time_fix].copy()

    if filtro == "POR_MANDO":
        if mando not in ["CASA","FORA"]:
            return dft.iloc[0:0].copy()
        dft = dft[dft["MANDO_FIX"]==mando].copy()

    # últimas N datas únicas
    datas = sorted(dft["DATA_FIX"].dropna().unique())
    if n <= 0:
        return dft.iloc[0:0].copy()
    datas_sel = datas[-n:] if len(datas) >= 1 else []
    dft = dft[dft["DATA_FIX"].isin(datas_sel)].copy()
    return dft

def agg_sum(df_slice, prefix, out):
    # soma scouts presentes
    for s in SCOUTS_LIST:
        c = pick_col(df_slice.columns, [s])
        if c:
            out[f"{prefix}_{s}"] = float(df_slice[c].sum())
    out[f"{prefix}_JOGOS_ENCONTRADOS"] = int(df_slice["DATA_FIX"].nunique())
    if COL_PTS:
        out[f"{prefix}_PTS_TOTAL"] = float(df_slice[COL_PTS].sum())
    if COL_BASICA:
        out[f"{prefix}_MEDIA_BASICA"] = float(df_slice[COL_BASICA].mean()) if len(df_slice) else 0.0
    return out

# -------------------
# Posreal → grupos
# -------------------
def pos_mask(df_base, grupo):
    if grupo == "GOL":
        return df_base["POSREAL"].round(1) == 1.0
    if grupo == "ZAG":
        return df_base["POSREAL"].round(1) == 3.0
    if grupo == "MEI":
        return df_base["POSREAL"].round(1) == 4.0
    if grupo == "ATA":
        return df_base["POSREAL"].round(1) == 5.0
    if grupo == "LAT":  # geral (se precisar)
        return df_base["POSREAL"].round(1).isin([2.2, 2.6])
    if grupo == "LD":
        return df_base["POSREAL"].round(1) == 2.2
    if grupo == "LE":
        return df_base["POSREAL"].round(1) == 2.6
    return df_base["POSREAL"].isna() & False

# -------------------
# MEIAS: separar MEIA/VOLANTE via DIVISAO (quando necessário)
# -------------------
def apply_divisao_meias(df_mei_raw):
    # se não existir coluna de nome no scouts, não conseguimos casar por jogador
    # então, por enquanto, só devolve como MEI geral.
    # (Seu motor anterior já funciona assim; a separação fina entra depois.)
    return df_mei_raw

# -------------------
# Builder por posição
# (mantém padrão de colunas do seu motor atual: M_CASA / M_FORA / M_OUTRO e V_CASA / V_FORA / V_OUTRO)
# Interpretação operacional:
# - Para MANDANTE:
#   M_CASA = conquistado do mandante em CASA
#   M_FORA = cedido ao mandante em CASA (adversários FORA contra ele)
#   M_OUTRO = conquistado do mandante FORA (referência complementar)
# - Para VISITANTE:
#   V_FORA = conquistado do visitante FORA
#   V_CASA = cedido ao visitante FORA (adversários CASA contra ele)
#   V_OUTRO = conquistado do visitante CASA (referência complementar)
# -------------------
def build_row_pos(grupo, mandante, visitante):
    out = {
        "RODADA": RODADA_REF,
        "MANDANTE": mandante,
        "VISITANTE": visitante,
        "FILTRO": FILTRO,
        "N_JOGOS": N_JOGOS,
    }

    # base por posição
    df_pos = df[pos_mask(df, grupo)].copy()

    # MEIAS (se quiser depois separar MEIA/VOLANTE, entra aqui)
    if grupo == "MEI":
        df_pos = apply_divisao_meias(df_pos)

    # --------- Mandante ----------
    # conquistado casa
    sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="CASA", mode="TIME")
    agg_sum(sl, "M_CASA", out)

    # cedido em casa (oponentes FORA contra o mandante)
    sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="FORA", mode="ADV")
    agg_sum(sl, "M_FORA", out)

    # referência: conquistado fora
    sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="FORA", mode="TIME")
    agg_sum(sl, "M_OUTRO", out)

    # --------- Visitante ----------
    # conquistado fora
    sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="FORA", mode="TIME")
    agg_sum(sl, "V_FORA", out)

    # cedido fora (oponentes CASA contra o visitante)
    sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="CASA", mode="ADV")
    agg_sum(sl, "V_CASA", out)

    # referência: conquistado casa
    sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="CASA", mode="TIME")
    agg_sum(sl, "V_OUTRO", out)

    return out

# -------------------
# LATERAIS (LD/LE separados)
# - Mantém o mesmo padrão do motor
# - Só que duplica métricas: ..._LD e ..._LE
# -------------------
def build_row_laterais(mandante, visitante):
    base = {
        "RODADA": RODADA_REF,
        "MANDANTE": mandante,
        "VISITANTE": visitante,
        "FILTRO": FILTRO,
        "N_JOGOS": N_JOGOS,
    }

    def add_side(grupo_lat, suf):
        out = {}
        df_pos = df[pos_mask(df, grupo_lat)].copy()

        # Mandante
        sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="CASA", mode="TIME")
        agg_sum(sl, f"M_CASA{suf}", out)

        sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="FORA", mode="ADV")
        agg_sum(sl, f"M_FORA{suf}", out)

        sl = slice_last_n(df_pos, mandante, N_JOGOS, FILTRO, mando="FORA", mode="TIME")
        agg_sum(sl, f"M_OUTRO{suf}", out)

        # Visitante
        sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="FORA", mode="TIME")
        agg_sum(sl, f"V_FORA{suf}", out)

        sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="CASA", mode="ADV")
        agg_sum(sl, f"V_CASA{suf}", out)

        sl = slice_last_n(df_pos, visitante, N_JOGOS, FILTRO, mando="CASA", mode="TIME")
        agg_sum(sl, f"V_OUTRO{suf}", out)

        return out

    # LD e LE
    base.update(add_side("LD", "_LD"))
    base.update(add_side("LE", "_LE"))
    return base

# -------------------
# Rodar rodada inteira
# -------------------
linhas = {
    "TABELA_GOLEIROS": [],
    "TABELA_ZAGUEIROS": [],
    "TABELA_LATERAIS": [],
    "TABELA_MEIAS": [],
    "TABELA_ATACANTES": [],
}

for _, r in df_r.iterrows():
    mand = r["MANDANTE_FIX"]
    vis  = r["VISITANTE_FIX"]

    linhas["TABELA_GOLEIROS"].append(build_row_pos("GOL", mand, vis))
    linhas["TABELA_ZAGUEIROS"].append(build_row_pos("ZAG", mand, vis))
    linhas["TABELA_MEIAS"].append(build_row_pos("MEI", mand, vis))
    linhas["TABELA_ATACANTES"].append(build_row_pos("ATA", mand, vis))

    # laterais separados
    linhas["TABELA_LATERAIS"].append(build_row_laterais(mand, vis))

# -------------------
# Export
# -------------------
if GERAR in ["NAO", "NÃO"]:
    print("ℹ️ GERAR_TABELAS=NAO -> não exportei arquivos. (Mas já tenho as tabelas em memória.)")
else:
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = OUT / f"RUN_2026_R{RODADA_REF:02d}_{FILTRO}_N{N_JOGOS}_{ts}"
    run_dir.mkdir(parents=True, exist_ok=True)

    for nome, rows in linhas.items():
        df_out = pd.DataFrame(rows)
        out_file = run_dir / f"{nome}_R{RODADA_REF:02d}.csv"
        df_out.to_csv(out_file, index=False, encoding="utf-8-sig")
        print(f"✅ Gerado: {out_file} | linhas={len(df_out)}")

    print("\n✅ TABELAS GERADAS COM SUCESSO")
    print(f"📁 Saída: {run_dir}")


✅ Pastas OK
✅ Arquivos OK
PILOTO: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/PILOTO_GERACAO_TABELAS.csv
CONFRONTOS: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/CONFRONTOS_2026_NORMALIZADO.csv
SCOUTS: /content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx
DIVISAO: /content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/DIVISAO_MEIAS_VOLANTES_NORMALIZADA.csv
✅ PILOTO: RODADA_REFERENCIA=1 | N_JOGOS=5 | FILTRO=POR_MANDO | GERAR_TABELAS=SIM
✅ Confrontos na rodada 1: 10
✅ Scouts carregado (Por jogo): linhas=12086 | times=20
✅ Gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv | linhas=10
✅ Gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv | linhas=10
✅ Gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01.csv | linhas=10
✅ Gerado: /content/drive/MyDrive/TCC

In [ ]:
# ==========================
# ACABAMENTO FINAL (ALINHAR + REEXPORTAR)
# Rode APÓS gerar as tabelas
# ==========================

import pandas as pd
from pathlib import Path
import json
import re

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

PILOTO_PATH = CFG/"PILOTO_GERACAO_TABELAS.csv"
SPEC_PATH   = CFG/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC.json"

def norm_txt(x):
    if pd.isna(x):
        return ""
    return str(x).strip().upper()

def load_piloto(path):
    df = pd.read_csv(path, dtype=str, encoding="utf-8-sig")
    df.columns = [c.replace("\ufeff","").strip().upper() for c in df.columns]
    if not set(["CHAVE","VALOR"]).issubset(set(df.columns)):
        raise ValueError(f"PILOTO inválido. Precisa ter CHAVE e VALOR. Veio: {list(df.columns)}")
    df["CHAVE"] = df["CHAVE"].map(norm_txt)
    df["VALOR"] = df["VALOR"].astype(str).str.strip()
    return dict(zip(df["CHAVE"], df["VALOR"]))

def detect_latest_run(output_dir: Path):
    runs = [p for p in output_dir.iterdir() if p.is_dir() and p.name.startswith("RUN_")]
    if not runs:
        raise FileNotFoundError("Não achei nenhuma pasta RUN_ dentro de 02_OUTPUT.")
    runs.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    return runs[0]

# 1) localizar RUN mais recente
run_dir = detect_latest_run(OUT)
print(f"✅ RUN detectada: {run_dir}")

# 2) carregar piloto (para preencher metadados)
piloto = load_piloto(PILOTO_PATH)
rodada = piloto.get("RODADA_ATUAL") or piloto.get("RODADA") or ""
filtro = piloto.get("FILTRO") or ""
n_jogos = piloto.get("N_JOGOS") or ""
rodada = str(rodada).strip()
filtro = str(filtro).strip().upper()
n_jogos = str(n_jogos).strip()

print(f"✅ PILOTO: RODADA={rodada} | FILTRO={filtro} | N_JOGOS={n_jogos}")

# 3) carregar bloco oficial
with open(SPEC_PATH, "r", encoding="utf-8") as f:
    spec = json.load(f)

# mapeamento arquivo -> chave do spec
file_map = {
    "TABELA_GOLEIROS": "GOLEIROS",
    "TABELA_ZAGUEIROS": "ZAGUEIROS",
    "TABELA_LATERAIS": "LATERAIS",
    "TABELA_MEIAS": "MEIAS",
    "TABELA_ATACANTES": "ATACANTES",
}

# 4) aplicar acabamento em cada CSV
falhas = []
for prefix, pos_key in file_map.items():
    # achar arquivo da rodada (qualquer sufixo Rxx)
    files = list(run_dir.glob(f"{prefix}_R*.csv"))
    if not files:
        falhas.append(f"Não achei {prefix}_R*.csv em {run_dir}")
        continue

    path = files[0]
    df = pd.read_csv(path)

    # normalizar nomes de colunas (trim)
    df.columns = [str(c).strip() for c in df.columns]

    # renomear HOME/AWAY se tiverem aparecido (bug comum)
    rename_hw = {}
    if "HOME" in df.columns and "MANDANTE" not in df.columns:
        rename_hw["HOME"] = "MANDANTE"
    if "AWAY" in df.columns and "VISITANTE" not in df.columns:
        rename_hw["AWAY"] = "VISITANTE"
    if rename_hw:
        df.rename(columns=rename_hw, inplace=True)

    # injetar metadados se faltarem
    if "RODADA" not in df.columns:
        df["RODADA"] = rodada
    if "FILTRO" not in df.columns:
        df["FILTRO"] = filtro
    if "N_JOGOS" not in df.columns:
        df["N_JOGOS"] = n_jogos

    # garantir MANDANTE/VISITANTE existam (se não existirem, mantém e o validador acusa)
    # (idealmente eles já existem pelo gerador)
    expected = spec[pos_key]

    # criar colunas faltantes como vazio (para não quebrar export)
    for c in expected:
        if c not in df.columns:
            df[c] = ""

    # reduzir e reordenar exatamente como o bloco oficial
    df_final = df[expected].copy()

    # sobrescrever CSV
    df_final.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"✅ Ajustado: {path.name} ({pos_key}) | colunas={len(df_final.columns)}")

if falhas:
    print("\n❌ Problemas encontrados:")
    for e in falhas:
        print("-", e)
else:
    print("\n✅ ACABAMENTO FINAL concluído. Agora rode o VALIDADOR novamente.")


✅ RUN detectada: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
✅ PILOTO: RODADA= | FILTRO=POR_MANDO | N_JOGOS=5
✅ Ajustado: TABELA_GOLEIROS_R01.csv (GOLEIROS) | colunas=45
✅ Ajustado: TABELA_ZAGUEIROS_R01.csv (ZAGUEIROS) | colunas=61
✅ Ajustado: TABELA_LATERAIS_R01.csv (LATERAIS) | colunas=61
✅ Ajustado: TABELA_MEIAS_R01.csv (MEIAS) | colunas=61
✅ Ajustado: TABELA_ATACANTES_R01.csv (ATACANTES) | colunas=61

✅ ACABAMENTO FINAL concluído. Agora rode o VALIDADOR novamente.


In [ ]:
from google.colab import drive
from pathlib import Path
import subprocess

# 1) Monta o Drive
drive.mount("/content/drive")

# 2) Caminhos fixos (não mexa)
BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
RUN = BASE / "02_OUTPUT" / "RUN_2026_R01_POR_MANDO_N5_20260103_230847"
SPEC = BASE / "03_CONFIG" / "BLOCO_OFICIAL_ORDEM_COLUNAS_TCC_v2.json"
VALIDADOR = BASE / "03_CONFIG" / "VALIDADOR_TCC.py"

print("▶ Rodando validador em:")
print(RUN)

# 3) Executa o validador
cmd = [
    "python3",
    str(VALIDADOR),
    "--input_dir", str(RUN),
    "--spec_json", str(SPEC)
]

subprocess.run(cmd)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
▶ Rodando validador em:
/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847


CompletedProcess(args=['python3', '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC.py', '--input_dir', '/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847', '--spec_json', '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC_v2.json'], returncode=1)

In [ ]:
# ==============================
# VALIDAR RUN MAIS RECENTE (Colab)
# Copie e cole numa CÉLULA NOVA
# ==============================
from google.colab import drive
from pathlib import Path
import subprocess, os, glob

drive.mount("/content/drive")

BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")
OUT  = BASE/"02_OUTPUT"
CFG  = BASE/"03_CONFIG"

SPEC = CFG/"BLOCO_OFICIAL_ORDEM_COLUNAS_TCC_v2.json"
VAL  = CFG/"VALIDADOR_TCC_v2.py"

# 1) acha a RUN mais recente
runs = sorted([p for p in OUT.glob("RUN_*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
if not runs:
    raise FileNotFoundError(f"Não achei nenhuma pasta RUN_* em: {OUT}")

RUN = runs[0]
print("✅ RUN detectada automaticamente:")
print("   ", RUN)

# 2) roda o validador (com saída visível)
cmd = ["python3", str(VAL), "--input_dir", str(RUN), "--spec_json", str(SPEC)]
print("\n▶ Rodando:\n ", " ".join(cmd), "\n")

proc = subprocess.run(cmd, capture_output=True, text=True)
print("===== STDOUT =====\n", proc.stdout)
print("===== STDERR =====\n", proc.stderr if proc.stderr else "(vazio)")
print("✅ Return code:", proc.returncode)

# 3) confere se o report foi criado e mostra o caminho
report = RUN/"REPORT_VALIDACAO_COLUNAS.csv"
if report.exists():
    print("\n✅ Report gerado:")
    print("   ", report)
    print("\n(Primeiras linhas do report)")
    !head -n 10 "{report}"
else:
    print("\n❌ Report NÃO encontrado dentro da RUN.")
    print("Isso só acontece se o validador falhou antes de salvar o relatório.")
    print("Neste caso: copie o conteúdo do STDERR acima e me envie.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ RUN detectada automaticamente:
    /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847

▶ Rodando:
  python3 /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/VALIDADOR_TCC_v2.py --input_dir /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847 --spec_json /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC_v2.json 

===== STDOUT =====
 
VALIDAÇÃO DE COLUNAS
Pasta: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
Spec: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/BLOCO_OFICIAL_ORDEM_COLUNAS_TCC_v2.json
Report: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/REPORT_VALIDACAO_COLUNAS.csv
Tabelas validadas: 5
Falhas: 0

✅ VALIDAÇÃO PASSOU 100%. Colunas e ordem est

In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_R01.png"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC.py': [Errno 2] No such file or directory


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_R01.png"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC.py': [Errno 2] No such file or directory


In [ ]:
from pathlib import Path

print("Existe pasta escudos?", Path("/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS").exists())
print("Existe o render?", Path("/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC.py").exists())
print("Existe o CSV goleiros?", Path("/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv").exists())

Existe pasta escudos? False
Existe o render? False
Existe o CSV goleiros? False


In [ ]:
from google.colab import drive
from pathlib import Path
import subprocess

# 1) Monta o Drive (se já estiver montado, não tem problema)
drive.mount("/content/drive")

# 2) Caminho base (não mexa)
BASE = Path("/content/drive/MyDrive/TCC TABELAS 2026")

# 3) AQUI você só confirma a RUN que quer usar (no seu caso, é essa)
RUN = BASE / "02_OUTPUT" / "RUN_2026_R01_POR_MANDO_N5_20260103_230847"

# 4) Arquivos/pastas usados no render
CSV_GOLEIROS = RUN / "TABELA_GOLEIROS_R01.csv"
ESCUDOS_DIR  = BASE / "04_ESCUDOS"
SCRIPT       = BASE / "03_CONFIG" / "RENDER_GOLEIROS_TCC.py"

# 5) Saída (png) dentro da própria RUN
OUT_PNG = RUN / "RENDER_GOLEIROS_R01.png"

print("CSV:", CSV_GOLEIROS)
print("Escudos:", ESCUDOS_DIR)
print("Script:", SCRIPT)
print("Saída PNG:", OUT_PNG)

cmd = [
    "python3",
    str(SCRIPT),
    "--csv", str(CSV_GOLEIROS),
    "--escudos_dir", str(ESCUDOS_DIR),
    "--out_png", str(OUT_PNG),
]
subprocess.run(cmd, check=False)

Mounted at /content/drive
CSV: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv
Escudos: /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS
Script: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC.py
Saída PNG: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_R01.png


CompletedProcess(args=['python3', '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC.py', '--csv', '/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv', '--escudos_dir', '/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS', '--out_png', '/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_R01.png'], returncode=0)

In [ ]:
%%writefile "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v2.py"
# -*- coding: utf-8 -*-
import argparse, math, re, unicodedata
from pathlib import Path
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

# ---------- util ----------
def slugify_pt(s: str) -> str:
    s = (s or "").strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace("_"," ").replace("-"," ")
    s = re.sub(r"\s+"," ", s).strip()
    return s

def safe_float(x):
    try:
        if x is None: return None
        if isinstance(x,str) and x.strip()=="": return None
        v = float(x)
        if math.isnan(v): return None
        return v
    except: return None

def fmt(v, dec=1, dash_if_empty=False):
    if v is None:
        return "-" if dash_if_empty else ""
    if abs(v-round(v)) < 1e-9 and dec == 0:
        return str(int(round(v)))
    return f"{v:.{dec}f}"

def load_font(size, bold=False):
    paths = [
        "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf" if bold else "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
        "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf" if bold else "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
    ]
    for p in paths:
        try:
            return ImageFont.truetype(p, size=size)
        except: pass
    return ImageFont.load_default()

# ---------- escudos ----------
ALIAS = {
    "atletico pr":"athletico pr",
    "athletico paranaense":"athletico pr",
    "atletico mg":"atlético mg",
    "atletico mineiro":"atlético mg",
    "sao paulo":"são paulo",
    "bragantino":"red bull bragantino",
    "inter":"internacional",
    "vitoria":"vitória",
}

def resolve_shield(team_name: str, esc_dir: Path):
    if not team_name: return None
    tn = slugify_pt(team_name)
    tn = slugify_pt(ALIAS.get(tn, tn))

    candidates = [tn, tn.replace(" ","_"), tn.replace(" ","-")]
    for c in candidates:
        p = esc_dir / f"{c}.png"
        if p.exists(): return p

    # fallback por igualdade de stem slugificado
    for p in esc_dir.glob("*.png"):
        if slugify_pt(p.stem) == tn:
            return p
    return None

# ---------- métricas (GOLEIROS) ----------
# CHUT.AG = DE + GC
# CHUT.PM = CHUT.AG / GC (se GC=0 -> "-")
# %DE = 100*DE/CHUT.AG
def compute(row, pref_conq, pref_ced):
    de_c = safe_float(row.get(f"{pref_conq}_DE"))
    gc_c = safe_float(row.get(f"{pref_conq}_GC"))
    sg_c = safe_float(row.get(f"{pref_conq}_SG"))

    de_d = safe_float(row.get(f"{pref_ced}_DE"))
    gc_d = safe_float(row.get(f"{pref_ced}_GC"))
    sg_d = safe_float(row.get(f"{pref_ced}_SG"))

    def ch_ag(de, gc):
        if de is None and gc is None: return None
        return (de or 0.0) + (gc or 0.0)

    def ch_pm(chag, gc):
        if chag is None: return None
        if (gc or 0.0) == 0.0: return None
        return chag / gc

    def pde(de, chag):
        if de is None or chag is None or chag == 0: return None
        return 100.0 * de / chag

    c_chag = ch_ag(de_c, gc_c)
    d_chag = ch_ag(de_d, gc_d)

    return {
        "CHAG_C": c_chag, "CHAG_D": d_chag,
        "CHPM_C": ch_pm(c_chag, gc_c), "CHPM_D": ch_pm(d_chag, gc_d),
        "GC_C": gc_c, "GC_D": gc_d,
        "DE_C": de_c, "DE_D": de_d,
        "SG_C": sg_c, "SG_D": sg_d,
        "PDE_C": pde(de_c, c_chag), "PDE_D": pde(de_d, d_chag),
    }

def q70(col):
    vals = [v for v in col if v is not None]
    if not vals: return None
    vals.sort()
    i = int(round((len(vals)-1)*0.70))
    return vals[i]

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--csv", required=True)
    ap.add_argument("--escudos_dir", required=True)
    ap.add_argument("--out_png", required=True)
    ap.add_argument("--w", type=int, default=5200)   # alta resolução
    ap.add_argument("--h", type=int, default=2900)
    args = ap.parse_args()

    csvp = Path(args.csv)
    escd = Path(args.escudos_dir)
    outp = Path(args.out_png)
    outp.parent.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(csvp)
    if "MANDANTE" not in df.columns or "VISITANTE" not in df.columns:
        raise SystemExit("CSV não tem MANDANTE/VISITANTE. Você apontou pro arquivo errado.")

    W,H = args.w, args.h
    img = Image.new("RGBA",(W,H),(18,90,66,255))
    draw = ImageDraw.Draw(img)

    # layout
    m = int(W*0.025)
    top1 = int(H*0.11)   # AMEAÇAS/OPORTUNIDADES
    top2 = int(H*0.06)   # CHUT.AG ...
    top3 = int(H*0.055)  # CO?/CD?
    rows = len(df)
    row_h = int((H - 2*m - top1 - top2 - top3) / max(1, rows))

    left_w = int((W-2*m)*0.43)
    center_w = int((W-2*m)*0.14)
    right_w = (W-2*m) - left_w - center_w

    xL = m
    xC = xL + left_w
    xR = xC + center_w
    y0 = m

    # cores
    BLUE=(145,198,235,255); ORA=(245,194,138,255)
    DG=(20,116,84,255); GRID=(35,35,35,255)
    PINK=(235,210,226,255); MINT=(206,235,213,255)
    HI=(173,223,202,255); WHITE=(255,255,255,255)

    fT = load_font(int(H*0.045), bold=True)
    fH = load_font(int(H*0.030), bold=True)
    fS = load_font(int(H*0.020), bold=True)
    fC = load_font(int(H*0.026), bold=True)
    fMid = load_font(int(H*0.060), bold=True)

    metrics = ["CHUT. AG","CHUT. PM","GOLS","DE","SG","% DE"]
    # 6 métricas; cada uma vira 2 subcolunas
    widths_frac = [0.16,0.16,0.10,0.10,0.10,0.18]
    base = sum(widths_frac)
    def widths_px(w):
        arr=[int(w*(f/base)) for f in widths_frac]
        arr[-1]+= (w - sum(arr))
        return arr

    wL = widths_px(left_w)
    wR = widths_px(right_w)

    # cabeçalhos
    def draw_headers(x,w, subpairs):
        half=int(w*0.45)
        draw.rectangle([x,y0,x+half,y0+top1], fill=BLUE, outline=GRID, width=2)
        draw.rectangle([x+half,y0,x+w,y0+top1], fill=ORA, outline=GRID, width=2)
        draw.text((x+half/2,y0+top1/2),"AMEAÇAS", font=fT, fill=WHITE, anchor="mm")
        draw.text((x+half+(w-half)/2,y0+top1/2),"OPORTUNIDADES", font=fT, fill=WHITE, anchor="mm")

        yy=y0+top1
        xx=x
        for name,cw in zip(metrics, widths_px(w)):
            draw.rectangle([xx,yy,xx+cw,yy+top2], fill=DG, outline=GRID, width=2)
            draw.text((xx+cw/2,yy+top2/2), name, font=fH, fill=WHITE, anchor="mm")
            xx+=cw

        yy2=yy+top2
        xx=x
        for (a,b),cw in zip(subpairs, widths_px(w)):
            c2=cw//2
            draw.rectangle([xx,yy2,xx+c2,yy2+top3], fill=PINK, outline=GRID, width=2)
            draw.rectangle([xx+c2,yy2,xx+cw,yy2+top3], fill=MINT, outline=GRID, width=2)
            draw.text((xx+c2/2,yy2+top3/2), a, font=fS, fill=(0,0,0,255), anchor="mm")
            draw.text((xx+c2+c2/2,yy2+top3/2), b, font=fS, fill=(0,0,0,255), anchor="mm")
            xx+=cw

    left_sub  = [("COF","CDC"),("COF","CDC"),("COF","CDC"),("COC","CDF"),("COC","CDF"),("COC","CDF")]
    right_sub = [("COC","CDF"),("COC","CDF"),("COC","CDF"),("COF","CDC"),("COF","CDC"),("COF","CDC")]
    draw_headers(xL,left_w,left_sub)
    draw_headers(xR,right_w,right_sub)

    # centro header
    draw.rectangle([xC,y0,xC+center_w,y0+top1+top2+top3], fill=(16,92,68,255), outline=GRID, width=2)
    draw.text((xC+center_w/2,y0+(top1+top2)/2),"MANDO", font=fH, fill=WHITE, anchor="mm")
    draw.text((xC+center_w/2,y0+top1+top2+top3/2),"CASA • FORA", font=fH, fill=WHITE, anchor="mm")

    # pré-cálculo (para highlights)
    left_rows=[]
    right_rows=[]
    for _,r in df.iterrows():
        row=r.to_dict()
        # esquerda: ameaças (V_FORA vs M_OUTRO), oportunidades (M_CASA vs V_OUTRO)
        th=compute(row,"V_FORA","M_OUTRO")
        op=compute(row,"M_CASA","V_OUTRO")
        left_rows.append([th["CHAG_C"],th["CHAG_D"], th["CHPM_C"],th["CHPM_D"], th["GC_C"],th["GC_D"],
                          op["DE_C"],op["DE_D"], op["SG_C"],op["SG_D"], op["PDE_C"],op["PDE_D"]])
        # direita: ameaças (M_CASA vs V_OUTRO), oportunidades (V_FORA vs M_OUTRO)
        th2=compute(row,"M_CASA","V_OUTRO")
        op2=compute(row,"V_FORA","M_OUTRO")
        right_rows.append([th2["CHAG_C"],th2["CHAG_D"], th2["CHPM_C"],th2["CHPM_D"], th2["GC_C"],th2["GC_D"],
                           op2["DE_C"],op2["DE_D"], op2["SG_C"],op2["SG_D"], op2["PDE_C"],op2["PDE_D"]])

    def thresholds(rows):
        cols=list(zip(*rows)) if rows else []
        return [q70([v for v in c]) for c in cols]

    tL=thresholds(left_rows)
    tR=thresholds(right_rows)

    # draw rows
    y_start=y0+top1+top2+top3
    for i in range(rows):
        y=y_start + i*row_h

        # base boxes
        draw.rectangle([xL,y,xL+left_w,y+row_h], fill=WHITE, outline=GRID, width=2)
        draw.rectangle([xR,y,xR+right_w,y+row_h], fill=WHITE, outline=GRID, width=2)

        # centro (escudos)
        mand=str(df.loc[i,"MANDANTE"])
        vis=str(df.loc[i,"VISITANTE"])
        pM=resolve_shield(mand,escd)
        pV=resolve_shield(vis,escd)
        draw.rectangle([xC,y,xC+center_w,y+row_h], fill=(16,92,68,255), outline=GRID, width=2)

        # escudos centralizados
        size=min(int(row_h*0.82), int(center_w*0.38))
        def paste(sh_path, cx):
            if not sh_path:
                draw.rectangle([cx-size//2,y+row_h//2-size//2, cx+size//2,y+row_h//2+size//2], outline=WHITE, width=2)
                return
            sh=Image.open(sh_path).convert("RGBA")
            sh.thumbnail((size,size))
            img.paste(sh,(int(cx-sh.width/2), int(y+row_h/2-sh.height/2)), sh)
        paste(pM, xC+center_w*0.32)
        paste(pV, xC+center_w*0.68)
        draw.text((xC+center_w/2, y+row_h/2), "×", font=fMid, fill=WHITE, anchor="mm")

        # cells helper
        def draw_side(x, widths, vals, thr):
            xx=x
            idx=0
            for cw in widths:
                c2=cw//2
                for part in range(2):
                    v=vals[idx]
                    t=thr[idx]
                    hi = (v is not None and t is not None and v >= t)
                    fill = HI if hi else WHITE
                    x1=xx+part*c2
                    x2=xx+(part+1)*c2
                    draw.rectangle([x1,y,x2,y+row_h], fill=fill, outline=GRID, width=2)

                    # formatação por coluna (0..11)
                    if idx in (0,1,4,5,6,7,8,9):
                        text = fmt(v, dec=0)
                    elif idx in (2,3):
                        text = fmt(v, dec=1, dash_if_empty=True)
                    else: # %DE
                        text = fmt(v, dec=1)

                    draw.text(((x1+x2)/2, y+row_h/2), text, font=fC, fill=(0,0,0,255), anchor="mm")
                    idx+=1
                xx+=cw

        draw_side(xL, wL, left_rows[i], tL)
        draw_side(xR, wR, right_rows[i], tR)

    out=Path(args.out_png)
    img.convert("RGB").save(out, "PNG", optimize=True)
    print("✅ PNG gerado:", out)

if __name__ == "__main__":
    main()


Writing /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v2.py


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v2.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_R01.png"


✅ PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_R01.png


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v2.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_R01.png"


✅ PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_R01.png


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/DIAGNOSTICO_ESCUDOS_TCC_v2.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ESCUDOS.txt"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/DIAGNOSTICO_ESCUDOS_TCC_v2.py': [Errno 2] No such file or directory


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/DIAGNOSTICO_ESCUDOS_TCC_v2.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ESCUDOS.txt"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/DIAGNOSTICO_ESCUDOS_TCC_v2.py': [Errno 2] No such file or directory


In [ ]:
# 1) garantir que o Drive está montado
from google.colab import drive
drive.mount('/content/drive')

# 2) listar exatamente os arquivos da pasta 03_CONFIG
import os
base = "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG"
print("PASTA:", base)
print("\nARQUIVOS:")
for f in sorted(os.listdir(base)):
    print(" -", repr(f))

# 3) rodar o diagnóstico chamando o nome EXATO do arquivo que aparecer na lista


Mounted at /content/drive
PASTA: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG

ARQUIVOS:
 - 'BLOCO_OFICIAL_ORDEM_COLUNAS_TCC_v2.json'
 - 'CELULA_COLAB_VALIDACAO_v2.txt'
 - 'COMANDO COLAB FINAL PARA GERAR TABELAS 2026.txt'
 - 'DIAGNOSTICO_ESCUDOS_TCC_v2.py'
 - 'GERADOR_TCC_TABELAS_2026_v1.py'
 - 'PILOTO_GERACAO_TABELAS.csv'
 - 'PILOTO_GERACAO_TABELAS.gsheet'
 - 'RENDER_GOLEIROS_TCC_v2.py'
 - 'RENDER_GOLEIROS_TCC_v3_1.py'
 - 'VALIDADOR_TCC_v2.py'


In [ ]:
!python3 "$(python3 - << 'PY'
import os
base = "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG"
# procura o arquivo de diagnostico que começa com DIAGNOSTICO_ESCUDOS
cand = [f for f in os.listdir(base) if f.startswith("DIAGNOSTICO_ESCUDOS") and f.endswith(".py")]
if not cand:
    raise SystemExit("Não achei nenhum arquivo DIAGNOSTICO_ESCUDOS*.py em 03_CONFIG")
cand = sorted(cand)[0]
print(os.path.join(base, cand))
PY
)" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ESCUDOS.txt"


SyntaxError: unmatched ')' (ipython-input-1347165839.py, line 11)

In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/DIAGNOSTICO_ESCUDOS_TCC_v2.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ESCUDOS.txt"


Log gerado em: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ESCUDOS.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v4.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v4.png" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v4.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v4.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v4.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v5.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v5.png" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v5.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v5.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v5.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v6.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v6.png" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v6.txt"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v6.py': [Errno 2] No such file or directory


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v6.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v6.png" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v6.txt"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v6.py': [Errno 2] No such file or directory


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v6.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v6.png" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v6.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v6.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v6.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v7.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v7.png" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v7.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v7.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v7.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v8.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v8.png" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v8.txt"


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v8.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v8.png" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v8.txt"


In [ ]:
!ls -l "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/" | grep RENDER_GOLEIROS_TCC_v8


-rw------- 1 root root  7584 Jan  4 19:44 RENDER_GOLEIROS_TCC_v8.py


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v8.py" --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v8.png" --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v8.txt"


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v8.py" \
--csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
--escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
--out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v8.png" \
--out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v8.txt"


In [ ]:
!ls -lh "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/" | egrep "RENDER_GOLEIROS_v8.png|LOG_RENDER_GOLEIROS_v8.txt" || echo "NAO CRIOU AINDA"


NAO CRIOU AINDA


In [ ]:
!ps -ef | egrep "RENDER_GOLEIROS_TCC_v8.py|python3" | grep -v grep


root          70       7  0 18:32 ?        00:00:21 [python3] <defunct>
root          71       7  0 18:32 ?        00:00:02 python3 /usr/local/bin/colab-fileshim.py
root          92       7  0 18:32 ?        00:00:08 /usr/bin/python3 /usr/local/bin/jupyter-server --debug --transport="ipc" --ip=172.28.0.12 --ServerApp.token= --port=9000 --FileContentsManager.root_dir=/ --FileContentsManager.allow_hidden=True --ServerApp.log_format="|%(levelname)s|%(message)s" --ServerApp.iopub_data_rate_limit=1e10 --MappingKernelManager.root_dir=/content
root         463      92  0 18:34 ?        00:00:04 /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-bc157b77-8600-4b68-bddf-69f00c6685b4.json
root        5263       1  0 18:53 ?        00:00:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
root        5266    5263  0 18:53 ?        00:00:00 pytho

In [ ]:
%%bash
set -e

python3 --version

python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v8.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v8.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v8.txt"

echo "FIM: gerou v8"


Python 3.12.12
FIM: gerou v8


In [ ]:
!ls -lh "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/" | egrep "RENDER_GOLEIROS_v8.png|LOG_RENDER_GOLEIROS_v8.txt" || echo "NAO CRIOU AINDA"


NAO CRIOU AINDA


In [ ]:
!ls -ld /content/drive /content/drive/MyDrive 2>&1


drwx------  6 root root 4096 Jan  4 18:53 /content/drive
drwx------ 49 root root 4096 Jan  4 18:53 /content/drive/MyDrive


In [ ]:
!find /content -maxdepth 5 -type f \( -name "RENDER_GOLEIROS_v8.png" -o -name "LOG_RENDER_GOLEIROS_v8.txt" \) -print 2>/dev/null


In [ ]:
%%bash
set -e

RUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"
SCRIPT="/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v8.py"

python3 -u "$SCRIPT" \
  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "$RUN/RENDER_GOLEIROS_v8.png" \
  --out_log "$RUN/LOG_RENDER_GOLEIROS_v8.txt" \
  2> "$RUN/ERRO_v8.txt"

echo "OK: terminou sem erro"


OK: terminou sem erro


In [ ]:
!ls -lh "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/" | egrep "RENDER_GOLEIROS_v8.png|LOG_RENDER_GOLEIROS_v8.txt|ERRO_v8.txt"


-rw------- 1 root root    0 Jan  4 20:03 ERRO_v8.txt


In [ ]:
!python3 - <<'PY'
p = "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v8.py"
with open(p, "r", encoding="utf-8") as f:
    txt = f.read()
print("ARQUIVO:", p)
print("TAMANHO:", len(txt), "chars")
print("TEM savefig?:", "savefig" in txt)
print("TEM out_png?:", "--out_png" in txt)
print("TRECHO (linhas 1-40):")
print("\n".join(txt.splitlines()[:40]))
PY


/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
ARQUIVO: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v8.py
TAMANHO: 7338 chars
TEM savefig?: False
TEM out_png?: False
TRECHO (linhas 1-40):
import re
import unicodedata
import argparse
from pathlib import Path
import difflib

import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


def norm(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9\s\-_/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


ALIASES = {
    "atletico": "atletico mg",
    "atletico-mg": "atletico mg",
    "clube atletico mineiro": "atletico mg",
    "atletico mineiro": "atletico mg",

    "athletico-pr": "athletico pr",
    "athletico pr": "athletico pr",

NameError: name 'PY' is not defined

In [ ]:
!ls -lh "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/" | head -n 50


total 1.4M
-rw------- 1 root root    0 Jan  4 20:03 ERRO_v8.txt
-rw------- 1 root root  185 Jan  4 19:27 LOG_RENDER_GOLEIROS_v6.txt
-rw------- 1 root root  165 Jan  4 19:37 LOG_RENDER_GOLEIROS_v7.txt
-rw------- 1 root root 673K Jan  4 19:27 RENDER_GOLEIROS_v6.png
-rw------- 1 root root 650K Jan  4 19:37 RENDER_GOLEIROS_v7.png
-rw------- 1 root root  302 Jan  3 23:35 REPORT_VALIDACAO_COLUNAS.csv
-rw------- 1 root root 3.6K Jan  3 23:09 TABELA_ATACANTES_R01.csv
-rw------- 1 root root 2.5K Jan  3 23:09 TABELA_GOLEIROS_R01.csv
-rw------- 1 root root 1.7K Jan  3 23:09 TABELA_LATERAIS_R01.csv
-rw------- 1 root root 3.7K Jan  3 23:09 TABELA_MEIAS_R01.csv
-rw------- 1 root root 3.4K Jan  3 23:09 TABELA_ZAGUEIROS_R01.csv


In [ ]:
%%bash
set -e

RUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"

python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v9.py" \
  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \
  --out_png "$RUN/RENDER_GOLEIROS_v9.png" \
  --out_log "$RUN/LOG_RENDER_GOLEIROS_v9.txt"

echo "OK: v9 gerado"


OK: v9 gerado


In [ ]:
%%bash
set -e

RUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"

python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v10.py" \
  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "$RUN/RENDER_GOLEIROS_v10.png" \
  --out_log "$RUN/LOG_RENDER_GOLEIROS_v10.txt"

echo "OK: v10 gerado"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v10.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v10.txt
OK: v10 gerado


In [ ]:
%%bash
set -e

RUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"

python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v11.py" \
  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "$RUN/RENDER_GOLEIROS_v11.png" \
  --out_log "$RUN/LOG_RENDER_GOLEIROS_v11.txt"

echo "OK: v11 gerado"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v11.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v11.txt
OK: v11 gerado


In [ ]:
%%bash
set -e

RUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"

python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v12.py" \
  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "$RUN/RENDER_GOLEIROS_v12.png" \
  --out_log "$RUN/LOG_RENDER_GOLEIROS_v12.txt"

echo "OK: v12 gerado"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v12.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v12.txt
OK: v12 gerado


In [ ]:
%%bash
set -e

RUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"

python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13.py" \
  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "$RUN/RENDER_GOLEIROS_v13.png" \
  --out_log "$RUN/LOG_RENDER_GOLEIROS_v13.txt"

echo "OK: v13 gerado"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13.txt
OK: v13 gerado


In [ ]:
%%bash
set -e

RUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"
SCRIPT="/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v14.py"

python3 -u "$SCRIPT" \
  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "$RUN/RENDER_GOLEIROS_v14.png" \
  --out_log "$RUN/LOG_RENDER_GOLEIROS_v14.txt"

echo "OK: gerou v14"


  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v14.py", line 1
    %%bash
    ^
SyntaxError: invalid syntax


CalledProcessError: Command 'b'set -e\n\nRUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"\nSCRIPT="/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v14.py"\n\npython3 -u "$SCRIPT" \\\n  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \\\n  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \\\n  --out_png "$RUN/RENDER_GOLEIROS_v14.png" \\\n  --out_log "$RUN/LOG_RENDER_GOLEIROS_v14.txt"\n\necho "OK: gerou v14"\n'' returned non-zero exit status 1.

In [ ]:
%%bash
set -e

cat > "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v14.py" << 'PY'
import re
import unicodedata
import argparse
from pathlib import Path
import difflib

import pandas as pd
from PIL import Image, ImageDraw, ImageFont

# =========================
# Utils
# =========================
def norm(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9\s\-_/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

ALIASES = {
    "atletico": "atletico mg",
    "atletico-mg": "atletico mg",
    "clube atletico mineiro": "atletico mg",
    "atletico mineiro": "atletico mg",

    "athletico-pr": "athletico pr",
    "athletico pr": "athletico pr",
    "athletico paranaense": "athletico pr",

    "red bull bragantino": "bragantino",
    "rb bragantino": "bragantino",
    "bragantino": "bragantino",

    "vasco da gama": "vasco",
    "gremio": "gremio",
    "grêmio": "gremio",

    "sao paulo": "sao paulo",
    "são paulo": "sao paulo",

    "corinthians": "corinthians",
    "bahia": "bahia",
    "flamengo": "flamengo",
    "palmeiras": "palmeiras",
    "santos": "santos",
    "botafogo": "botafogo",
    "cruzeiro": "cruzeiro",
    "internacional": "internacional",
    "vitoria": "vitoria",
    "vitória": "vitoria",
    "fortaleza": "fortaleza",
    "juventude": "juventude",
    "mirassol": "mirassol",
    "ceara": "ceara",
    "cuiaba": "cuiaba",
    "goias": "goias",
    "gremio": "gremio",
}

def canon_team(name: str) -> str:
    k = norm(name)
    return ALIASES.get(k, k)

def load_font(size: int, bold=False):
    # tenta fontes comuns do Colab
    candidates = []
    if bold:
        candidates += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
            "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf",
        ]
    else:
        candidates += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
            "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
        ]
    for p in candidates:
        try:
            return ImageFont.truetype(p, size=size)
        except:
            pass
    return ImageFont.load_default()

def draw_text_shadow(draw: ImageDraw.ImageDraw, xy, text, font, fill, shadow=(0,0,0,80), offset=(2,2), anchor="mm"):
    x,y = xy
    sx, sy = offset
    draw.text((x+sx, y+sy), text, font=font, fill=shadow, anchor=anchor)
    draw.text((x, y), text, font=font, fill=fill, anchor=anchor)

def list_shields(escudos_dir: Path):
    exts = {".png",".jpg",".jpeg",".webp"}
    files = []
    for p in escudos_dir.glob("*"):
        if p.suffix.lower() in exts and p.is_file():
            files.append(p)
    return files

def best_shield_file(team_name: str, escudos_dir: Path):
    # primeiro tenta match exato por slug normalizado
    team = canon_team(team_name)
    files = list_shields(escudos_dir)
    if not files:
        return None, team, None

    # mapa slug -> path
    slug_map = {}
    for f in files:
        slug = norm(f.stem)
        slug_map[slug] = f

    if team in slug_map:
        return slug_map[team], team, "exact"

    # tenta aproximação por difflib
    keys = list(slug_map.keys())
    close = difflib.get_close_matches(team, keys, n=1, cutoff=0.70)
    if close:
        return slug_map[close[0]], team, f"fuzzy:{close[0]}"

    return None, team, None

def open_shield(path: Path):
    im = Image.open(path).convert("RGBA")
    return im

def fit_shield_inside_circle(shield_rgba: Image.Image, target_d: int):
    """
    Mantém proporção SEM distorcer.
    Faz um 'contain': cabe dentro do círculo com folga.
    """
    w,h = shield_rgba.size
    if w == 0 or h == 0:
        return shield_rgba
    # folga interna: 88% do diâmetro (evita passar da bolinha)
    max_side = int(target_d * 0.88)
    scale = min(max_side / w, max_side / h)
    nw, nh = max(1,int(w*scale)), max(1,int(h*scale))
    return shield_rgba.resize((nw,nh), Image.Resampling.LANCZOS)

def make_circle_token(diameter: int, border: int = 0):
    """
    Círculo branco com antialiasing: desenha em 4x e reduz.
    """
    up = 4
    D = diameter * up
    img = Image.new("RGBA", (D, D), (0,0,0,0))
    d = ImageDraw.Draw(img)
    # círculo sólido
    d.ellipse((0,0,D-1,D-1), fill=(255,255,255,255))
    # borda opcional
    if border > 0:
        b = border * up
        d.ellipse((b,b,D-1-b,D-1-b), outline=(0,0,0,40), width=max(1, int(1*up)))
    # downsample
    img = img.resize((diameter,diameter), Image.Resampling.LANCZOS)
    return img

# =========================
# Render
# =========================
def render(args):
    csv_path = Path(args.csv)
    escudos_dir = Path(args.escudos_dir)
    out_png = Path(args.out_png)
    out_log = Path(args.out_log) if args.out_log else None

    df = pd.read_csv(csv_path)

    # Identificação das colunas de time mandante/visitante (robusto)
    cols_norm = {c: norm(c) for c in df.columns}
    home_col = None
    away_col = None
    for c,nc in cols_norm.items():
        if nc in ("casa","time_casa","mandante","home","clube_casa","clube mandante"):
            home_col = c
        if nc in ("fora","time_fora","visitante","away","clube_fora","clube visitante"):
            away_col = c
    # fallback: procurar algo que contenha casa/fora
    if home_col is None:
        for c,nc in cols_norm.items():
            if "casa" in nc or "mandante" in nc:
                home_col = c; break
    if away_col is None:
        for c,nc in cols_norm.items():
            if "fora" in nc or "visitante" in nc:
                away_col = c; break

    # Se não achou, assume colunas 0 e 1 como confronto (último fallback)
    if home_col is None or away_col is None:
        home_col = df.columns[0]
        away_col = df.columns[1]

    # Remove qualquer coluna "por_mando" aparecendo como dado
    # (isso estava vazando para a primeira coluna em algumas versões)
    # Se existir coluna POR_MANDO, NÃO renderiza como célula de dados.
    drop_cols = []
    for c in df.columns:
        if norm(c) in ("por_mando","por mando","tipo","filtro"):
            drop_cols.append(c)
    if drop_cols:
        df = df.drop(columns=drop_cols)

    # =========================
    # CONFIG VISUAL (base em 2x + downsample)
    # =========================
    scale = 2  # superamostragem
    W = 5200 * scale
    H = 2900 * scale

    # Paleta (ajustada)
    GREEN_BG = (18, 92, 70, 255)
    GREEN_HDR = (15, 92, 73, 255)
    BLUE = (160, 200, 230, 255)
    ORANGE = (230, 180, 140, 255)
    SUB_PINK = (240, 210, 220, 255)
    SUB_GRAY = (215, 220, 225, 255)
    WHITE = (255,255,255,255)
    BLACK = (0,0,0,255)

    # bordas mais finas (pedido seu)
    BORDER = 3 * scale
    GRID = 2 * scale

    # Layout geral
    outer_margin = 70 * scale
    inner_w = W - 2*outer_margin
    inner_h = H - 2*outer_margin

    # larguras: esquerda + mando + direita
    mando_w = int(inner_w * 0.16)   # mantém próximo do que você já gostou
    side_w = (inner_w - mando_w) // 2

    # alturas dos blocos
    h1 = 170 * scale   # AMEAÇAS / OPORTUNIDADES / etc (um pouco menor + mais centrado)
    h2 = 120 * scale   # CHUT.AG etc
    h3 = 95  * scale   # subheaders COF/CDC etc
    data_h = inner_h - (h1+h2+h3)

    n_rows = len(df)
    row_h = data_h // n_rows

    # fontes (tudo ligeiramente menor e com sombra)
    f_top = load_font(int(64*scale), bold=True)
    f_mid = load_font(int(42*scale), bold=True)
    f_sub = load_font(int(34*scale), bold=True)
    f_cell = load_font(int(42*scale), bold=True)
    f_x = load_font(int(58*scale), bold=True)

    # Canvas
    img = Image.new("RGBA", (W, H), GREEN_BG)
    draw = ImageDraw.Draw(img)

    # moldura externa
    x0 = outer_margin
    y0 = outer_margin
    x1 = x0 + inner_w
    y1 = y0 + inner_h
    draw.rectangle((x0, y0, x1, y1), outline=(0,0,0,255), width=BORDER)

    # Função para desenhar retângulo com borda fina
    def rect(xa, ya, xb, yb, fill, outline=(0,0,0,255), w=GRID):
        draw.rectangle((xa,ya,xb,yb), fill=fill, outline=outline, width=w)

    # =========================
    # Cabeçalho topo: esquerda (ameaças/oportunidades) + mando + direita
    # =========================
    left_x0 = x0
    left_x1 = left_x0 + side_w
    mando_x0 = left_x1
    mando_x1 = mando_x0 + mando_w
    right_x0 = mando_x1
    right_x1 = x1

    # Dentro de cada lado: 2 blocos topo
    # Ajuste simples: ameaças ~48%, oportunidades ~52% (parecido com referência)
    def draw_top_blocks(side_x0, side_x1, title_left, title_right):
        wside = side_x1 - side_x0
        wl = int(wside * 0.48)
        wr = wside - wl

        ax0 = side_x0
        ax1 = ax0 + wl
        ox0 = ax1
        ox1 = side_x1

        # topo
        rect(ax0, y0, ax1, y0+h1, BLUE, w=GRID)
        rect(ox0, y0, ox1, y0+h1, ORANGE, w=GRID)

        # texto centralizado (horizontal e vertical) + sombra
        draw_text_shadow(draw, ((ax0+ax1)//2, y0 + h1//2), title_left, f_top, WHITE, shadow=(0,0,0,90), offset=(3*scale,3*scale))
        draw_text_shadow(draw, ((ox0+ox1)//2, y0 + h1//2), title_right, f_top, WHITE, shadow=(0,0,0,90), offset=(3*scale,3*scale))

        return (ax0,ax1,ox0,ox1)

    draw_top_blocks(left_x0, left_x1, "AMEAÇAS", "OPORTUNIDADES")
    draw_top_blocks(right_x0, right_x1, "AMEAÇAS", "OPORTUNIDADES")

    # Mando topo
    rect(mando_x0, y0, mando_x1, y0+h1, GREEN_HDR, w=GRID)
    draw_text_shadow(draw, ((mando_x0+mando_x1)//2, y0 + h2//2), "MANDO", f_mid, WHITE, shadow=(0,0,0,90), offset=(3*scale,3*scale))

    # =========================
    # Linha CHUT.AG etc (h2)
    # =========================
    # Colunas de cada lado: 6 blocos principais
    main_headers = ["CHUT.AG","CHUT.PM","GOLS","DE","SG","% DE"]

    # largura relativa por bloco (mais próxima da referência)
    weights = [1.15, 1.15, 1.05, 1.0, 1.0, 1.20]
    wsum = sum(weights)

    def split_widths(total_w):
        widths = [int(total_w * (w/wsum)) for w in weights]
        # ajusta sobra
        diff = total_w - sum(widths)
        widths[-1] += diff
        return widths

    left_ws = split_widths(side_w)
    right_ws = split_widths(side_w)

    def draw_mid_row(side_x0, widths):
        cx = side_x0
        for lab, ww in zip(main_headers, widths):
            rect(cx, y0+h1, cx+ww, y0+h1+h2, GREEN_HDR, w=GRID)
            draw_text_shadow(draw, (cx + ww//2, y0+h1 + h2//2), lab, f_mid, WHITE, shadow=(0,0,0,90), offset=(2*scale,2*scale))
            cx += ww

    draw_mid_row(left_x0, left_ws)
    draw_mid_row(right_x0, right_ws)

    # Mando na linha h2: "CASA • FORA"
    rect(mando_x0, y0+h1, mando_x1, y0+h1+h2, GREEN_HDR, w=GRID)
    draw_text_shadow(draw, ((mando_x0+mando_x1)//2, y0+h1 + h2//2), "CASA • FORA", f_mid, WHITE, shadow=(0,0,0,90), offset=(2*scale,2*scale))

    # =========================
    # Subheaders (h3) com:
    # - borda separando COF|CDC etc (pedido seu)
    # - cor rosado p/ CO*, cinza p/ CD*
    # =========================
    # cada bloco principal tem 2 subcolunas
    def draw_subheaders(side_x0, widths, sub_left_is_CO=True, side="L"):
        cx = side_x0
        # mapa do que aparece em cada bloco
        # esquerda: CHUT.AG(COF/CDC), CHUT.PM(COF/CDC), GOLS(COF/CDC), DE(COC/CDF), SG(COC/CDF), %DE(COC/CDF)
        # direita: CHUT.AG(COC/CDF), CHUT.PM(COC/CDF), GOLS(COC/CDF), DE(COF/CDC), SG(COF/CDC), %DE(COF/CDC)
        if side == "L":
            pairs = [("COF","CDC"),("COF","CDC"),("COF","CDC"),("COC","CDF"),("COC","CDF"),("COC","CDF")]
        else:
            pairs = [("COC","CDF"),("COC","CDF"),("COC","CDF"),("COF","CDC"),("COF","CDC"),("COF","CDC")]

        for ww,(a,b) in zip(widths,pairs):
            half = ww//2
            # fundo por tipo
            fill_a = SUB_PINK if a.startswith("CO") else SUB_GRAY
            fill_b = SUB_PINK if b.startswith("CO") else SUB_GRAY

            rect(cx, y0+h1+h2, cx+half, y0+h1+h2+h3, fill_a, w=GRID)
            rect(cx+half, y0+h1+h2, cx+ww, y0+h1+h2+h3, fill_b, w=GRID)

            # textos centralizados (e COM espaço real por coluna)
            draw.text((cx + half//2, y0+h1+h2 + h3//2), a, font=f_sub, fill=BLACK, anchor="mm")
            draw.text((cx + half + (ww-half)//2, y0+h1+h2 + h3//2), b, font=f_sub, fill=BLACK, anchor="mm")

            # borda vertical entre a e b (exata, só nessa linha)
            draw.line((cx+half, y0+h1+h2, cx+half, y0+h1+h2+h3), fill=(0,0,0,255), width=GRID)

            cx += ww

    draw_subheaders(left_x0, left_ws, side="L")
    draw_subheaders(right_x0, right_ws, side="R")

    # Subheaders no mando (linha h3) fica vazio/verde
    rect(mando_x0, y0+h1+h2, mando_x1, y0+h1+h2+h3, GREEN_HDR, w=GRID)

    # =========================
    # Linhas de dados
    # - mantém bordas verticais apenas entre blocos principais (não separa subcolunas)
    # =========================
    # colunas que devem ser usadas por lado:
    # Como seu CSV pode variar, vamos mapear pelo nome.
    def pick_col(candidates):
        for cand in candidates:
            for c in df.columns:
                if norm(c) == norm(cand):
                    return c
        # fallback por contains
        for cand in candidates:
            for c in df.columns:
                if norm(cand) in norm(c):
                    return c
        return None

    # Colunas esperadas (tentei manter compatível com seus nomes anteriores)
    # Esquerda (ameaças e oportunidades)
    L_CHUTAG_COF = pick_col(["L_CHUT_AG_COF","CHUT_AG_COF","CHUT.AG_COF","CHUT_AG_CO","CHUTAG_COF"])
    L_CHUTAG_CDC = pick_col(["L_CHUT_AG_CDC","CHUT_AG_CDC","CHUT.AG_CDC","CHUT_AG_CD","CHUTAG_CDC"])
    L_CHUTPM_COF = pick_col(["L_CHUT_PM_COF","CHUT_PM_COF","CHUT.PM_COF","CHUTPM_COF"])
    L_CHUTPM_CDC = pick_col(["L_CHUT_PM_CDC","CHUT_PM_CDC","CHUT.PM_CDC","CHUTPM_CDC"])
    L_GOLS_COF   = pick_col(["L_GOLS_COF","GOLS_COF","GOLS COF","GOLS_COF"])
    L_GOLS_CDC   = pick_col(["L_GOLS_CDC","GOLS_CDC","GOLS CDC","GOLS_CDC"])
    L_DE_COC     = pick_col(["L_DE_COC","DE_COC","DE COC"])
    L_DE_CDF     = pick_col(["L_DE_CDF","DE_CDF","DE CDF"])
    L_SG_COC     = pick_col(["L_SG_COC","SG_COC","SG COC"])
    L_SG_CDF     = pick_col(["L_SG_CDF","SG_CDF","SG CDF"])
    L_PDE_COC    = pick_col(["L_PDE_COC","%DE_COC","PDE_COC","PCT_DE_COC"])
    L_PDE_CDF    = pick_col(["L_PDE_CDF","%DE_CDF","PDE_CDF","PCT_DE_CDF"])

    # Direita
    R_CHUTAG_COC = pick_col(["R_CHUT_AG_COC","CHUT_AG_COC_R","CHUT_AG_COC"])
    R_CHUTAG_CDF = pick_col(["R_CHUT_AG_CDF","CHUT_AG_CDF_R","CHUT_AG_CDF"])
    R_CHUTPM_COC = pick_col(["R_CHUT_PM_COC","CHUT_PM_COC_R","CHUT_PM_COC"])
    R_CHUTPM_CDF = pick_col(["R_CHUT_PM_CDF","CHUT_PM_CDF_R","CHUT_PM_CDF"])
    R_GOLS_COC   = pick_col(["R_GOLS_COC","GOLS_COC_R","GOLS_COC"])
    R_GOLS_CDF   = pick_col(["R_GOLS_CDF","GOLS_CDF_R","GOLS_CDF"])
    R_DE_COF     = pick_col(["R_DE_COF","DE_COF_R","DE_COF"])
    R_DE_CDC     = pick_col(["R_DE_CDC","DE_CDC_R","DE_CDC"])
    R_SG_COF     = pick_col(["R_SG_COF","SG_COF_R","SG_COF"])
    R_SG_CDC     = pick_col(["R_SG_CDC","SG_CDC_R","SG_CDC"])
    R_PDE_COF    = pick_col(["R_PDE_COF","%DE_COF_R","PDE_COF_R","PDE_COF"])
    R_PDE_CDC    = pick_col(["R_PDE_CDC","%DE_CDC_R","PDE_CDC_R","PDE_CDC"])

    # caso seu CSV não tenha exatamente esses nomes, ainda assim o grid sai correto;
    # valores ausentes ficam em branco.
    def get_val(row, col):
        if col is None or col not in row.index:
            return ""
        v = row[col]
        if pd.isna(v):
            return ""
        # formatação leve
        if isinstance(v, float):
            # mantém 1 casa se necessário
            s = f"{v:.1f}"
            # remove .0 quando for inteiro
            if s.endswith(".0"):
                s = s[:-2]
            return s
        return str(v)

    # colunas por bloco (cada bloco = 2 números)
    L_pairs = [
        (L_CHUTAG_COF, L_CHUTAG_CDC),
        (L_CHUTPM_COF, L_CHUTPM_CDC),
        (L_GOLS_COF,   L_GOLS_CDC),
        (L_DE_COC,     L_DE_CDF),
        (L_SG_COC,     L_SG_CDF),
        (L_PDE_COC,    L_PDE_CDF),
    ]
    R_pairs = [
        (R_CHUTAG_COC, R_CHUTAG_CDF),
        (R_CHUTPM_COC, R_CHUTPM_CDF),
        (R_GOLS_COC,   R_GOLS_CDF),
        (R_DE_COF,     R_DE_CDC),
        (R_SG_COF,     R_SG_CDC),
        (R_PDE_COF,    R_PDE_CDC),
    ]

    # Destaques (verde claro) – mantém como estava (se existir regra mais exata depois, ajustamos)
    HIGHLIGHT = (170, 230, 205, 255)

    def maybe_highlight(v):
        # regra provisória: se valor não vazio e diferente de "0"
        if v == "" or v == "0" or v == "0.0":
            return False
        return True

    y_data0 = y0 + h1 + h2 + h3

    # pré-gerar círculos
    circle_d = int(row_h * 0.78)  # círculo um pouco menor que a altura da linha
    circle_img = make_circle_token(circle_d)

    log_lines = []
    def paste_match_shield(team_name, center_xy, circle_d):
        nonlocal img
        fpath, teamcanon, how = best_shield_file(team_name, escudos_dir)
        if fpath is None:
            # placeholder: só círculo branco (sem inventar escudo)
            log_lines.append(f"ESCUDO_NAO_ENCONTRADO | recebido='{team_name}' | canon='{teamcanon}'")
            token = circle_img.copy()
            # coloca um "?" discreto
            dd = ImageDraw.Draw(token)
            ff = load_font(int(46*scale), bold=True)
            dd.text((circle_d//2, circle_d//2), "?", font=ff, fill=(80,80,80,255), anchor="mm")
            px = int(center_xy[0] - circle_d/2)
            py = int(center_xy[1] - circle_d/2)
            img.alpha_composite(token, (px,py))
            return

        shield = open_shield(fpath)
        shield_fit = fit_shield_inside_circle(shield, circle_d)

        token = circle_img.copy()
        # centraliza dentro do círculo
        sx, sy = shield_fit.size
        ox = (circle_d - sx)//2
        oy = (circle_d - sy)//2
        token.alpha_composite(shield_fit, (ox,oy))

        px = int(center_xy[0] - circle_d/2)
        py = int(center_xy[1] - circle_d/2)
        img.alpha_composite(token, (px,py))

    # desenha linhas
    for i, row in df.iterrows():
        y_top = y_data0 + i*row_h
        y_bot = y_top + row_h

        # fundo branco linha inteira (lado esquerdo + direito)
        rect(left_x0,  y_top, left_x1,  y_bot, WHITE, w=GRID)
        rect(right_x0, y_top, right_x1, y_bot, WHITE, w=GRID)

        # mando: faixa verde
        rect(mando_x0, y_top, mando_x1, y_bot, GREEN_HDR, w=GRID)

        # ====== desenha valores (SEM linha dividindo subcolunas) ======
        def draw_side_values(side_x0, widths, pairs):
            cx = side_x0
            for ww,(cA,cB) in zip(widths, pairs):
                # borda do bloco principal
                rect(cx, y_top, cx+ww, y_bot, WHITE, w=GRID)

                # duas posições internas (sem linha)
                half = ww//2
                vA = get_val(row, cA)
                vB = get_val(row, cB)

                # highlight por célula (provisório)
                if maybe_highlight(vA):
                    rect(cx, y_top, cx+half, y_bot, HIGHLIGHT, outline=(0,0,0,0), w=0)
                if maybe_highlight(vB):
                    rect(cx+half, y_top, cx+ww, y_bot, HIGHLIGHT, outline=(0,0,0,0), w=0)

                # textos
                draw.text((cx + half//2, y_top + row_h//2), vA, font=f_cell, fill=BLACK, anchor="mm")
                draw.text((cx + half + (ww-half)//2, y_top + row_h//2), vB, font=f_cell, fill=BLACK, anchor="mm")

                # borda entre blocos principais (garante)
                draw.line((cx+ww, y_top, cx+ww, y_bot), fill=(0,0,0,255), width=GRID)

                cx += ww

        draw_side_values(left_x0, left_ws, L_pairs)
        draw_side_values(right_x0, right_ws, R_pairs)

        # ====== confronto no mando ======
        home = str(row[home_col]) if home_col in row.index else ""
        away = str(row[away_col]) if away_col in row.index else ""

        # centros (dois escudos e o X)
        cx_mid = (mando_x0 + mando_x1)//2
        y_mid = y_top + row_h//2

        # distância horizontal
        dx = int(mando_w * 0.22)
        left_c = (cx_mid - dx, y_mid)
        right_c = (cx_mid + dx, y_mid)

        paste_match_shield(home, left_c, circle_d)
        paste_match_shield(away, right_c, circle_d)

        # X
        draw.text((cx_mid, y_mid), "×", font=f_x, fill=WHITE, anchor="mm")

    # =========================
    # Log
    # =========================
    if out_log:
        out_log.parent.mkdir(parents=True, exist_ok=True)
        out_log.write_text("\n".join(log_lines), encoding="utf-8")

    # =========================
    # Export: downsample p/ qualidade (2x -> 1x)
    # =========================
    out_png.parent.mkdir(parents=True, exist_ok=True)
    final = img.resize((W//scale, H//scale), Image.Resampling.LANCZOS)
    final.save(out_png, format="PNG", optimize=True)

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--csv", required=True)
    ap.add_argument("--escudos_dir", required=True)
    ap.add_argument("--out_png", required=True)
    ap.add_argument("--out_log", default="")
    args = ap.parse_args()
    render(args)

if __name__ == "__main__":
    main()
PY

echo "OK: v14 criado em 03_CONFIG"


OK: v14 criado em 03_CONFIG


In [ ]:
%%bash
set -e

cat > "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v14.py" << 'PY'
import re
import unicodedata
import argparse
from pathlib import Path
import difflib

import pandas as pd
from PIL import Image, ImageDraw, ImageFont

# =========================
# Utils
# =========================
def norm(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9\s\-_/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

ALIASES = {
    "atletico": "atletico mg",
    "atletico-mg": "atletico mg",
    "clube atletico mineiro": "atletico mg",
    "atletico mineiro": "atletico mg",

    "athletico-pr": "athletico pr",
    "athletico pr": "athletico pr",
    "athletico paranaense": "athletico pr",

    "red bull bragantino": "bragantino",
    "rb bragantino": "bragantino",
    "bragantino": "bragantino",

    "vasco da gama": "vasco",
    "gremio": "gremio",
    "grêmio": "gremio",

    "sao paulo": "sao paulo",
    "são paulo": "sao paulo",

    "corinthians": "corinthians",
    "bahia": "bahia",
    "flamengo": "flamengo",
    "palmeiras": "palmeiras",
    "santos": "santos",
    "botafogo": "botafogo",
    "cruzeiro": "cruzeiro",
    "internacional": "internacional",
    "vitoria": "vitoria",
    "vitória": "vitoria",
    "fortaleza": "fortaleza",
    "juventude": "juventude",
    "mirassol": "mirassol",
    "ceara": "ceara",
    "cuiaba": "cuiaba",
    "goias": "goias",
    "gremio": "gremio",
}

def canon_team(name: str) -> str:
    k = norm(name)
    return ALIASES.get(k, k)

def load_font(size: int, bold=False):
    # tenta fontes comuns do Colab
    candidates = []
    if bold:
        candidates += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
            "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf",
        ]
    else:
        candidates += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
            "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
        ]
    for p in candidates:
        try:
            return ImageFont.truetype(p, size=size)
        except:
            pass
    return ImageFont.load_default()

def draw_text_shadow(draw: ImageDraw.ImageDraw, xy, text, font, fill, shadow=(0,0,0,80), offset=(2,2), anchor="mm"):
    x,y = xy
    sx, sy = offset
    draw.text((x+sx, y+sy), text, font=font, fill=shadow, anchor=anchor)
    draw.text((x, y), text, font=font, fill=fill, anchor=anchor)

def list_shields(escudos_dir: Path):
    exts = {".png",".jpg",".jpeg",".webp"}
    files = []
    for p in escudos_dir.glob("*"):
        if p.suffix.lower() in exts and p.is_file():
            files.append(p)
    return files

def best_shield_file(team_name: str, escudos_dir: Path):
    # primeiro tenta match exato por slug normalizado
    team = canon_team(team_name)
    files = list_shields(escudos_dir)
    if not files:
        return None, team, None

    # mapa slug -> path
    slug_map = {}
    for f in files:
        slug = norm(f.stem)
        slug_map[slug] = f

    if team in slug_map:
        return slug_map[team], team, "exact"

    # tenta aproximação por difflib
    keys = list(slug_map.keys())
    close = difflib.get_close_matches(team, keys, n=1, cutoff=0.70)
    if close:
        return slug_map[close[0]], team, f"fuzzy:{close[0]}"

    return None, team, None

def open_shield(path: Path):
    im = Image.open(path).convert("RGBA")
    return im

def fit_shield_inside_circle(shield_rgba: Image.Image, target_d: int):
    """
    Mantém proporção SEM distorcer.
    Faz um 'contain': cabe dentro do círculo com folga.
    """
    w,h = shield_rgba.size
    if w == 0 or h == 0:
        return shield_rgba
    # folga interna: 88% do diâmetro (evita passar da bolinha)
    max_side = int(target_d * 0.88)
    scale = min(max_side / w, max_side / h)
    nw, nh = max(1,int(w*scale)), max(1,int(h*scale))
    return shield_rgba.resize((nw,nh), Image.Resampling.LANCZOS)

def make_circle_token(diameter: int, border: int = 0):
    """
    Círculo branco com antialiasing: desenha em 4x e reduz.
    """
    up = 4
    D = diameter * up
    img = Image.new("RGBA", (D, D), (0,0,0,0))
    d = ImageDraw.Draw(img)
    # círculo sólido
    d.ellipse((0,0,D-1,D-1), fill=(255,255,255,255))
    # borda opcional
    if border > 0:
        b = border * up
        d.ellipse((b,b,D-1-b,D-1-b), outline=(0,0,0,40), width=max(1, int(1*up)))
    # downsample
    img = img.resize((diameter,diameter), Image.Resampling.LANCZOS)
    return img

# =========================
# Render
# =========================
def render(args):
    csv_path = Path(args.csv)
    escudos_dir = Path(args.escudos_dir)
    out_png = Path(args.out_png)
    out_log = Path(args.out_log) if args.out_log else None

    df = pd.read_csv(csv_path)

    # Identificação das colunas de time mandante/visitante (robusto)
    cols_norm = {c: norm(c) for c in df.columns}
    home_col = None
    away_col = None
    for c,nc in cols_norm.items():
        if nc in ("casa","time_casa","mandante","home","clube_casa","clube mandante"):
            home_col = c
        if nc in ("fora","time_fora","visitante","away","clube_fora","clube visitante"):
            away_col = c
    # fallback: procurar algo que contenha casa/fora
    if home_col is None:
        for c,nc in cols_norm.items():
            if "casa" in nc or "mandante" in nc:
                home_col = c; break
    if away_col is None:
        for c,nc in cols_norm.items():
            if "fora" in nc or "visitante" in nc:
                away_col = c; break

    # Se não achou, assume colunas 0 e 1 como confronto (último fallback)
    if home_col is None or away_col is None:
        home_col = df.columns[0]
        away_col = df.columns[1]

    # Remove qualquer coluna "por_mando" aparecendo como dado
    # (isso estava vazando para a primeira coluna em algumas versões)
    # Se existir coluna POR_MANDO, NÃO renderiza como célula de dados.
    drop_cols = []
    for c in df.columns:
        if norm(c) in ("por_mando","por mando","tipo","filtro"):
            drop_cols.append(c)
    if drop_cols:
        df = df.drop(columns=drop_cols)

    # =========================
    # CONFIG VISUAL (base em 2x + downsample)
    # =========================
    scale = 2  # superamostragem
    W = 5200 * scale
    H = 2900 * scale

    # Paleta (ajustada)
    GREEN_BG = (18, 92, 70, 255)
    GREEN_HDR = (15, 92, 73, 255)
    BLUE = (160, 200, 230, 255)
    ORANGE = (230, 180, 140, 255)
    SUB_PINK = (240, 210, 220, 255)
    SUB_GRAY = (215, 220, 225, 255)
    WHITE = (255,255,255,255)
    BLACK = (0,0,0,255)

    # bordas mais finas (pedido seu)
    BORDER = 3 * scale
    GRID = 2 * scale

    # Layout geral
    outer_margin = 70 * scale
    inner_w = W - 2*outer_margin
    inner_h = H - 2*outer_margin

    # larguras: esquerda + mando + direita
    mando_w = int(inner_w * 0.16)   # mantém próximo do que você já gostou
    side_w = (inner_w - mando_w) // 2

    # alturas dos blocos
    h1 = 170 * scale   # AMEAÇAS / OPORTUNIDADES / etc (um pouco menor + mais centrado)
    h2 = 120 * scale   # CHUT.AG etc
    h3 = 95  * scale   # subheaders COF/CDC etc
    data_h = inner_h - (h1+h2+h3)

    n_rows = len(df)
    row_h = data_h // n_rows

    # fontes (tudo ligeiramente menor e com sombra)
    f_top = load_font(int(64*scale), bold=True)
    f_mid = load_font(int(42*scale), bold=True)
    f_sub = load_font(int(34*scale), bold=True)
    f_cell = load_font(int(42*scale), bold=True)
    f_x = load_font(int(58*scale), bold=True)

    # Canvas
    img = Image.new("RGBA", (W, H), GREEN_BG)
    draw = ImageDraw.Draw(img)

    # moldura externa
    x0 = outer_margin
    y0 = outer_margin
    x1 = x0 + inner_w
    y1 = y0 + inner_h
    draw.rectangle((x0, y0, x1, y1), outline=(0,0,0,255), width=BORDER)

    # Função para desenhar retângulo com borda fina
    def rect(xa, ya, xb, yb, fill, outline=(0,0,0,255), w=GRID):
        draw.rectangle((xa,ya,xb,yb), fill=fill, outline=outline, width=w)

    # =========================
    # Cabeçalho topo: esquerda (ameaças/oportunidades) + mando + direita
    # =========================
    left_x0 = x0
    left_x1 = left_x0 + side_w
    mando_x0 = left_x1
    mando_x1 = mando_x0 + mando_w
    right_x0 = mando_x1
    right_x1 = x1

    # Dentro de cada lado: 2 blocos topo
    # Ajuste simples: ameaças ~48%, oportunidades ~52% (parecido com referência)
    def draw_top_blocks(side_x0, side_x1, title_left, title_right):
        wside = side_x1 - side_x0
        wl = int(wside * 0.48)
        wr = wside - wl

        ax0 = side_x0
        ax1 = ax0 + wl
        ox0 = ax1
        ox1 = side_x1

        # topo
        rect(ax0, y0, ax1, y0+h1, BLUE, w=GRID)
        rect(ox0, y0, ox1, y0+h1, ORANGE, w=GRID)

        # texto centralizado (horizontal e vertical) + sombra
        draw_text_shadow(draw, ((ax0+ax1)//2, y0 + h1//2), title_left, f_top, WHITE, shadow=(0,0,0,90), offset=(3*scale,3*scale))
        draw_text_shadow(draw, ((ox0+ox1)//2, y0 + h1//2), title_right, f_top, WHITE, shadow=(0,0,0,90), offset=(3*scale,3*scale))

        return (ax0,ax1,ox0,ox1)

    draw_top_blocks(left_x0, left_x1, "AMEAÇAS", "OPORTUNIDADES")
    draw_top_blocks(right_x0, right_x1, "AMEAÇAS", "OPORTUNIDADES")

    # Mando topo
    rect(mando_x0, y0, mando_x1, y0+h1, GREEN_HDR, w=GRID)
    draw_text_shadow(draw, ((mando_x0+mando_x1)//2, y0 + h2//2), "MANDO", f_mid, WHITE, shadow=(0,0,0,90), offset=(3*scale,3*scale))

    # =========================
    # Linha CHUT.AG etc (h2)
    # =========================
    # Colunas de cada lado: 6 blocos principais
    main_headers = ["CHUT.AG","CHUT.PM","GOLS","DE","SG","% DE"]

    # largura relativa por bloco (mais próxima da referência)
    weights = [1.15, 1.15, 1.05, 1.0, 1.0, 1.20]
    wsum = sum(weights)

    def split_widths(total_w):
        widths = [int(total_w * (w/wsum)) for w in weights]
        # ajusta sobra
        diff = total_w - sum(widths)
        widths[-1] += diff
        return widths

    left_ws = split_widths(side_w)
    right_ws = split_widths(side_w)

    def draw_mid_row(side_x0, widths):
        cx = side_x0
        for lab, ww in zip(main_headers, widths):
            rect(cx, y0+h1, cx+ww, y0+h1+h2, GREEN_HDR, w=GRID)
            draw_text_shadow(draw, (cx + ww//2, y0+h1 + h2//2), lab, f_mid, WHITE, shadow=(0,0,0,90), offset=(2*scale,2*scale))
            cx += ww

    draw_mid_row(left_x0, left_ws)
    draw_mid_row(right_x0, right_ws)

    # Mando na linha h2: "CASA • FORA"
    rect(mando_x0, y0+h1, mando_x1, y0+h1+h2, GREEN_HDR, w=GRID)
    draw_text_shadow(draw, ((mando_x0+mando_x1)//2, y0+h1 + h2//2), "CASA • FORA", f_mid, WHITE, shadow=(0,0,0,90), offset=(2*scale,2*scale))

    # =========================
    # Subheaders (h3) com:
    # - borda separando COF|CDC etc (pedido seu)
    # - cor rosado p/ CO*, cinza p/ CD*
    # =========================
    # cada bloco principal tem 2 subcolunas
    def draw_subheaders(side_x0, widths, sub_left_is_CO=True, side="L"):
        cx = side_x0
        # mapa do que aparece em cada bloco
        # esquerda: CHUT.AG(COF/CDC), CHUT.PM(COF/CDC), GOLS(COF/CDC), DE(COC/CDF), SG(COC/CDF), %DE(COC/CDF)
        # direita: CHUT.AG(COC/CDF), CHUT.PM(COC/CDF), GOLS(COC/CDF), DE(COF/CDC), SG(COF/CDC), %DE(COF/CDC)
        if side == "L":
            pairs = [("COF","CDC"),("COF","CDC"),("COF","CDC"),("COC","CDF"),("COC","CDF"),("COC","CDF")]
        else:
            pairs = [("COC","CDF"),("COC","CDF"),("COC","CDF"),("COF","CDC"),("COF","CDC"),("COF","CDC")]

        for ww,(a,b) in zip(widths,pairs):
            half = ww//2
            # fundo por tipo
            fill_a = SUB_PINK if a.startswith("CO") else SUB_GRAY
            fill_b = SUB_PINK if b.startswith("CO") else SUB_GRAY

            rect(cx, y0+h1+h2, cx+half, y0+h1+h2+h3, fill_a, w=GRID)
            rect(cx+half, y0+h1+h2, cx+ww, y0+h1+h2+h3, fill_b, w=GRID)

            # textos centralizados (e COM espaço real por coluna)
            draw.text((cx + half//2, y0+h1+h2 + h3//2), a, font=f_sub, fill=BLACK, anchor="mm")
            draw.text((cx + half + (ww-half)//2, y0+h1+h2 + h3//2), b, font=f_sub, fill=BLACK, anchor="mm")

            # borda vertical entre a e b (exata, só nessa linha)
            draw.line((cx+half, y0+h1+h2, cx+half, y0+h1+h2+h3), fill=(0,0,0,255), width=GRID)

            cx += ww

    draw_subheaders(left_x0, left_ws, side="L")
    draw_subheaders(right_x0, right_ws, side="R")

    # Subheaders no mando (linha h3) fica vazio/verde
    rect(mando_x0, y0+h1+h2, mando_x1, y0+h1+h2+h3, GREEN_HDR, w=GRID)

    # =========================
    # Linhas de dados
    # - mantém bordas verticais apenas entre blocos principais (não separa subcolunas)
    # =========================
    # colunas que devem ser usadas por lado:
    # Como seu CSV pode variar, vamos mapear pelo nome.
    def pick_col(candidates):
        for cand in candidates:
            for c in df.columns:
                if norm(c) == norm(cand):
                    return c
        # fallback por contains
        for cand in candidates:
            for c in df.columns:
                if norm(cand) in norm(c):
                    return c
        return None

    # Colunas esperadas (tentei manter compatível com seus nomes anteriores)
    # Esquerda (ameaças e oportunidades)
    L_CHUTAG_COF = pick_col(["L_CHUT_AG_COF","CHUT_AG_COF","CHUT.AG_COF","CHUT_AG_CO","CHUTAG_COF"])
    L_CHUTAG_CDC = pick_col(["L_CHUT_AG_CDC","CHUT_AG_CDC","CHUT.AG_CDC","CHUT_AG_CD","CHUTAG_CDC"])
    L_CHUTPM_COF = pick_col(["L_CHUT_PM_COF","CHUT_PM_COF","CHUT.PM_COF","CHUTPM_COF"])
    L_CHUTPM_CDC = pick_col(["L_CHUT_PM_CDC","CHUT_PM_CDC","CHUT.PM_CDC","CHUTPM_CDC"])
    L_GOLS_COF   = pick_col(["L_GOLS_COF","GOLS_COF","GOLS COF","GOLS_COF"])
    L_GOLS_CDC   = pick_col(["L_GOLS_CDC","GOLS_CDC","GOLS CDC","GOLS_CDC"])
    L_DE_COC     = pick_col(["L_DE_COC","DE_COC","DE COC"])
    L_DE_CDF     = pick_col(["L_DE_CDF","DE_CDF","DE CDF"])
    L_SG_COC     = pick_col(["L_SG_COC","SG_COC","SG COC"])
    L_SG_CDF     = pick_col(["L_SG_CDF","SG_CDF","SG CDF"])
    L_PDE_COC    = pick_col(["L_PDE_COC","%DE_COC","PDE_COC","PCT_DE_COC"])
    L_PDE_CDF    = pick_col(["L_PDE_CDF","%DE_CDF","PDE_CDF","PCT_DE_CDF"])

    # Direita
    R_CHUTAG_COC = pick_col(["R_CHUT_AG_COC","CHUT_AG_COC_R","CHUT_AG_COC"])
    R_CHUTAG_CDF = pick_col(["R_CHUT_AG_CDF","CHUT_AG_CDF_R","CHUT_AG_CDF"])
    R_CHUTPM_COC = pick_col(["R_CHUT_PM_COC","CHUT_PM_COC_R","CHUT_PM_COC"])
    R_CHUTPM_CDF = pick_col(["R_CHUT_PM_CDF","CHUT_PM_CDF_R","CHUT_PM_CDF"])
    R_GOLS_COC   = pick_col(["R_GOLS_COC","GOLS_COC_R","GOLS_COC"])
    R_GOLS_CDF   = pick_col(["R_GOLS_CDF","GOLS_CDF_R","GOLS_CDF"])
    R_DE_COF     = pick_col(["R_DE_COF","DE_COF_R","DE_COF"])
    R_DE_CDC     = pick_col(["R_DE_CDC","DE_CDC_R","DE_CDC"])
    R_SG_COF     = pick_col(["R_SG_COF","SG_COF_R","SG_COF"])
    R_SG_CDC     = pick_col(["R_SG_CDC","SG_CDC_R","SG_CDC"])
    R_PDE_COF    = pick_col(["R_PDE_COF","%DE_COF_R","PDE_COF_R","PDE_COF"])
    R_PDE_CDC    = pick_col(["R_PDE_CDC","%DE_CDC_R","PDE_CDC_R","PDE_CDC"])

    # caso seu CSV não tenha exatamente esses nomes, ainda assim o grid sai correto;
    # valores ausentes ficam em branco.
    def get_val(row, col):
        if col is None or col not in row.index:
            return ""
        v = row[col]
        if pd.isna(v):
            return ""
        # formatação leve
        if isinstance(v, float):
            # mantém 1 casa se necessário
            s = f"{v:.1f}"
            # remove .0 quando for inteiro
            if s.endswith(".0"):
                s = s[:-2]
            return s
        return str(v)

    # colunas por bloco (cada bloco = 2 números)
    L_pairs = [
        (L_CHUTAG_COF, L_CHUTAG_CDC),
        (L_CHUTPM_COF, L_CHUTPM_CDC),
        (L_GOLS_COF,   L_GOLS_CDC),
        (L_DE_COC,     L_DE_CDF),
        (L_SG_COC,     L_SG_CDF),
        (L_PDE_COC,    L_PDE_CDF),
    ]
    R_pairs = [
        (R_CHUTAG_COC, R_CHUTAG_CDF),
        (R_CHUTPM_COC, R_CHUTPM_CDF),
        (R_GOLS_COC,   R_GOLS_CDF),
        (R_DE_COF,     R_DE_CDC),
        (R_SG_COF,     R_SG_CDC),
        (R_PDE_COF,    R_PDE_CDC),
    ]

    # Destaques (verde claro) – mantém como estava (se existir regra mais exata depois, ajustamos)
    HIGHLIGHT = (170, 230, 205, 255)

    def maybe_highlight(v):
        # regra provisória: se valor não vazio e diferente de "0"
        if v == "" or v == "0" or v == "0.0":
            return False
        return True

    y_data0 = y0 + h1 + h2 + h3

    # pré-gerar círculos
    circle_d = int(row_h * 0.78)  # círculo um pouco menor que a altura da linha
    circle_img = make_circle_token(circle_d)

    log_lines = []
    def paste_match_shield(team_name, center_xy, circle_d):
        nonlocal img
        fpath, teamcanon, how = best_shield_file(team_name, escudos_dir)
        if fpath is None:
            # placeholder: só círculo branco (sem inventar escudo)
            log_lines.append(f"ESCUDO_NAO_ENCONTRADO | recebido='{team_name}' | canon='{teamcanon}'")
            token = circle_img.copy()
            # coloca um "?" discreto
            dd = ImageDraw.Draw(token)
            ff = load_font(int(46*scale), bold=True)
            dd.text((circle_d//2, circle_d//2), "?", font=ff, fill=(80,80,80,255), anchor="mm")
            px = int(center_xy[0] - circle_d/2)
            py = int(center_xy[1] - circle_d/2)
            img.alpha_composite(token, (px,py))
            return

        shield = open_shield(fpath)
        shield_fit = fit_shield_inside_circle(shield, circle_d)

        token = circle_img.copy()
        # centraliza dentro do círculo
        sx, sy = shield_fit.size
        ox = (circle_d - sx)//2
        oy = (circle_d - sy)//2
        token.alpha_composite(shield_fit, (ox,oy))

        px = int(center_xy[0] - circle_d/2)
        py = int(center_xy[1] - circle_d/2)
        img.alpha_composite(token, (px,py))

    # desenha linhas
    for i, row in df.iterrows():
        y_top = y_data0 + i*row_h
        y_bot = y_top + row_h

        # fundo branco linha inteira (lado esquerdo + direito)
        rect(left_x0,  y_top, left_x1,  y_bot, WHITE, w=GRID)
        rect(right_x0, y_top, right_x1, y_bot, WHITE, w=GRID)

        # mando: faixa verde
        rect(mando_x0, y_top, mando_x1, y_bot, GREEN_HDR, w=GRID)

        # ====== desenha valores (SEM linha dividindo subcolunas) ======
        def draw_side_values(side_x0, widths, pairs):
            cx = side_x0
            for ww,(cA,cB) in zip(widths, pairs):
                # borda do bloco principal
                rect(cx, y_top, cx+ww, y_bot, WHITE, w=GRID)

                # duas posições internas (sem linha)
                half = ww//2
                vA = get_val(row, cA)
                vB = get_val(row, cB)

                # highlight por célula (provisório)
                if maybe_highlight(vA):
                    rect(cx, y_top, cx+half, y_bot, HIGHLIGHT, outline=(0,0,0,0), w=0)
                if maybe_highlight(vB):
                    rect(cx+half, y_top, cx+ww, y_bot, HIGHLIGHT, outline=(0,0,0,0), w=0)

                # textos
                draw.text((cx + half//2, y_top + row_h//2), vA, font=f_cell, fill=BLACK, anchor="mm")
                draw.text((cx + half + (ww-half)//2, y_top + row_h//2), vB, font=f_cell, fill=BLACK, anchor="mm")

                # borda entre blocos principais (garante)
                draw.line((cx+ww, y_top, cx+ww, y_bot), fill=(0,0,0,255), width=GRID)

                cx += ww

        draw_side_values(left_x0, left_ws, L_pairs)
        draw_side_values(right_x0, right_ws, R_pairs)

        # ====== confronto no mando ======
        home = str(row[home_col]) if home_col in row.index else ""
        away = str(row[away_col]) if away_col in row.index else ""

        # centros (dois escudos e o X)
        cx_mid = (mando_x0 + mando_x1)//2
        y_mid = y_top + row_h//2

        # distância horizontal
        dx = int(mando_w * 0.22)
        left_c = (cx_mid - dx, y_mid)
        right_c = (cx_mid + dx, y_mid)

        paste_match_shield(home, left_c, circle_d)
        paste_match_shield(away, right_c, circle_d)

        # X
        draw.text((cx_mid, y_mid), "×", font=f_x, fill=WHITE, anchor="mm")

    # =========================
    # Log
    # =========================
    if out_log:
        out_log.parent.mkdir(parents=True, exist_ok=True)
        out_log.write_text("\n".join(log_lines), encoding="utf-8")

    # =========================
    # Export: downsample p/ qualidade (2x -> 1x)
    # =========================
    out_png.parent.mkdir(parents=True, exist_ok=True)
    final = img.resize((W//scale, H//scale), Image.Resampling.LANCZOS)
    final.save(out_png, format="PNG", optimize=True)

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--csv", required=True)
    ap.add_argument("--escudos_dir", required=True)
    ap.add_argument("--out_png", required=True)
    ap.add_argument("--out_log", default="")
    args = ap.parse_args()
    render(args)

if __name__ == "__main__":
    main()
PY

echo "OK: v14 criado em 03_CONFIG"


OK: v14 criado em 03_CONFIG


In [ ]:
%%bash
set -e

RUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"
SCRIPT="/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v14.py"

python3 -u "$SCRIPT" \
  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "$RUN/RENDER_GOLEIROS_v14.png" \
  --out_log "$RUN/LOG_RENDER_GOLEIROS_v14.txt"

echo "OK: gerou v14"


OK: gerou v14


In [ ]:
%%bash
set -e

cat > "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v15.py" << 'PY'
import re, unicodedata, difflib, argparse
from pathlib import Path

import pandas as pd
from PIL import Image, ImageDraw, ImageFont

# =========================
# Normalização / aliases
# =========================
def norm(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9\s\-_/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

ALIASES = {
    "atletico": "atletico mg",
    "atletico-mg": "atletico mg",
    "clube atletico mineiro": "atletico mg",
    "atletico mineiro": "atletico mg",

    "athletico-pr": "athletico pr",
    "athletico pr": "athletico pr",
    "athletico paranaense": "athletico pr",

    "red bull bragantino": "bragantino",
    "rb bragantino": "bragantino",
    "rbbragantino": "bragantino",
    "bragantino": "bragantino",

    "sao paulo": "sao paulo",
    "são paulo": "sao paulo",
    "gremio": "gremio",
    "grêmio": "gremio",
    "vitoria": "vitoria",
    "vitória": "vitoria",
    "vasco da gama": "vasco",
}

def canon_team(name: str) -> str:
    k = norm(name)
    return ALIASES.get(k, k)

def load_font(size: int, bold=False):
    candidates = []
    if bold:
        candidates += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
            "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf",
        ]
    else:
        candidates += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
            "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
        ]
    for p in candidates:
        try:
            return ImageFont.truetype(p, size=size)
        except:
            pass
    return ImageFont.load_default()

def draw_text_shadow(draw, xy, text, font, fill, shadow=(0,0,0,90), offset=(2,2), anchor="mm"):
    x,y = xy
    ox, oy = offset
    draw.text((x+ox, y+oy), text, font=font, fill=shadow, anchor=anchor)
    draw.text((x, y), text, font=font, fill=fill, anchor=anchor)

# =========================
# Escudos
# =========================
def list_shields(escudos_dir: Path):
    exts = {".png",".jpg",".jpeg",".webp"}
    return [p for p in escudos_dir.glob("*") if p.is_file() and p.suffix.lower() in exts]

def best_shield_file(team_name: str, escudos_dir: Path):
    team = canon_team(team_name)
    files = list_shields(escudos_dir)
    if not files:
        return None, team, None

    slug_map = {}
    for f in files:
        slug = norm(f.stem)
        slug_map[slug] = f

    if team in slug_map:
        return slug_map[team], team, "exact"

    # tentativa por "contém" (muito útil quando o nome do arquivo tem sufixos)
    for slug, fp in slug_map.items():
        if team in slug or slug in team:
            return fp, team, f"contains:{slug}"

    keys = list(slug_map.keys())
    close = difflib.get_close_matches(team, keys, n=1, cutoff=0.55)
    if close:
        return slug_map[close[0]], team, f"fuzzy:{close[0]}"

    return None, team, None

def open_shield(path: Path):
    return Image.open(path).convert("RGBA")

def fit_shield_inside_circle(shield_rgba: Image.Image, target_d: int, factor: float = 0.85):
    # mantém proporção SEM distorcer; factor reduz para não vazar (SPFC/FLA/BOT/Athletico etc)
    w,h = shield_rgba.size
    if w <= 0 or h <= 0:
        return shield_rgba
    max_side = int(target_d * factor)
    scale = min(max_side / w, max_side / h)
    nw, nh = max(1,int(w*scale)), max(1,int(h*scale))
    return shield_rgba.resize((nw,nh), Image.Resampling.LANCZOS)

def make_circle_token(diameter: int):
    # antialias forte (8x) para evitar pixelização no zoom
    up = 8
    D = diameter * up
    img = Image.new("RGBA", (D, D), (0,0,0,0))
    d = ImageDraw.Draw(img)
    d.ellipse((0,0,D-1,D-1), fill=(255,255,255,255))
    # downsample
    img = img.resize((diameter,diameter), Image.Resampling.LANCZOS)
    return img

# =========================
# CSV -> 24 colunas numéricas (ordem)
# =========================
def detect_home_away_cols(df: pd.DataFrame):
    cols_norm = {c: norm(c) for c in df.columns}
    home_col = None
    away_col = None

    for c,nc in cols_norm.items():
        if nc in ("casa","time_casa","mandante","home","clube_casa","clube mandante"):
            home_col = c
        if nc in ("fora","time_fora","visitante","away","clube_fora","clube visitante"):
            away_col = c

    if home_col is None:
        for c,nc in cols_norm.items():
            if "casa" in nc or "mandante" in nc:
                home_col = c; break
    if away_col is None:
        for c,nc in cols_norm.items():
            if "fora" in nc or "visitante" in nc:
                away_col = c; break

    if home_col is None or away_col is None:
        home_col = df.columns[0]
        away_col = df.columns[1]
    return home_col, away_col

def numeric_cols_in_order(df: pd.DataFrame, exclude):
    cols = []
    for c in df.columns:
        if c in exclude:
            continue
        s = pd.to_numeric(df[c], errors="coerce")
        # considera numérica se tiver pelo menos 1 valor numérico
        if s.notna().any():
            cols.append(c)
    return cols

def format_cell(v):
    if v is None:
        return ""
    if isinstance(v, float):
        # 1 casa quando necessário; remove .0
        s = f"{v:.1f}"
        if s.endswith(".0"):
            s = s[:-2]
        return s
    return str(v)

# =========================
# Render (layout do v13)
# =========================
def render(args):
    csv_path = Path(args.csv)
    escudos_dir = Path(args.escudos_dir)
    out_png = Path(args.out_png)
    out_log = Path(args.out_log) if args.out_log else None

    df = pd.read_csv(csv_path)

    # remove colunas “lixo” tipo POR_MANDO, se existirem
    drop_cols = [c for c in df.columns if norm(c) in ("por_mando","por mando")]
    if drop_cols:
        df = df.drop(columns=drop_cols)

    home_col, away_col = detect_home_away_cols(df)

    # pega 24 colunas numéricas na ordem do CSV (isso evita quebrar como no v14)
    num_cols = numeric_cols_in_order(df, exclude={home_col, away_col})
    if len(num_cols) < 24:
        # não quebra: só loga e preenche o que der
        pass
    # usa só as 24 primeiras
    num_cols = num_cols[:24]

    # mapeamento fixo por posição (24)
    # 0..11 = lado esquerdo (6 blocos x 2)
    # 12..23 = lado direito (6 blocos x 2)
    def get24(row, idx):
        if idx >= len(num_cols):
            return ""
        v = row[num_cols[idx]]
        if pd.isna(v):
            return ""
        return format_cell(float(v) if isinstance(v,(int,float)) else v)

    # =========================
    # CONFIG VISUAL (igual v13, com supersampling)
    # =========================
    scale = 3  # qualidade
    W0, H0 = 2600, 1450  # base (v13-like). Mantém proporção e evita “crescer tudo”
    W, H = W0*scale, H0*scale

    GREEN_BG = (18, 92, 70, 255)
    GREEN_HDR = (15, 92, 73, 255)
    BLUE = (160, 200, 230, 255)
    ORANGE = (230, 180, 140, 255)
    SUB_PINK = (240, 210, 220, 255)
    SUB_GRAY = (215, 220, 225, 255)
    WHITE = (255,255,255,255)
    BLACK = (0,0,0,255)

    BORDER = 3*scale
    GRID = 2*scale

    outer_margin = 35*scale
    inner_w = W - 2*outer_margin
    inner_h = H - 2*outer_margin

    mando_w = int(inner_w * 0.16)
    side_w = (inner_w - mando_w) // 2

    h1 = 92*scale   # AMEAÇAS/OPORTUNIDADES
    h2 = 62*scale   # CHUT.AG etc
    h3 = 46*scale   # COF/CDC etc
    data_h = inner_h - (h1+h2+h3)

    n_rows = len(df)
    row_h = data_h // max(1, n_rows)

    f_top = load_font(int(34*scale), bold=True)
    f_mid = load_font(int(24*scale), bold=True)
    f_sub = load_font(int(19*scale), bold=True)
    f_cell = load_font(int(22*scale), bold=True)
    f_x   = load_font(int(30*scale), bold=True)

    img = Image.new("RGBA", (W, H), GREEN_BG)
    draw = ImageDraw.Draw(img)

    x0 = outer_margin
    y0 = outer_margin
    x1 = x0 + inner_w
    y1 = y0 + inner_h

    draw.rectangle((x0,y0,x1,y1), outline=(0,0,0,255), width=BORDER)

    def rect(xa, ya, xb, yb, fill, w=GRID):
        draw.rectangle((xa,ya,xb,yb), fill=fill, outline=(0,0,0,255), width=w)

    left_x0 = x0
    left


bash: line 274: warning: here-document at line 3 delimited by end-of-file (wanted `PY')


In [ ]:
%%bash
set -e

cat > "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v15.py" << 'PY'
import re, unicodedata, difflib, argparse
from pathlib import Path

import pandas as pd
from PIL import Image, ImageDraw, ImageFont

# =========================
# Normalização / aliases
# =========================
def norm(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9\s\-_/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

ALIASES = {
    "atletico": "atletico mg",
    "atletico-mg": "atletico mg",
    "clube atletico mineiro": "atletico mg",
    "atletico mineiro": "atletico mg",

    "athletico-pr": "athletico pr",
    "athletico pr": "athletico pr",
    "athletico paranaense": "athletico pr",

    "red bull bragantino": "bragantino",
    "rb bragantino": "bragantino",
    "rbbragantino": "bragantino",
    "bragantino": "bragantino",

    "sao paulo": "sao paulo",
    "são paulo": "sao paulo",
    "gremio": "gremio",
    "grêmio": "gremio",
    "vitoria": "vitoria",
    "vitória": "vitoria",
    "vasco da gama": "vasco",
}

def canon_team(name: str) -> str:
    k = norm(name)
    return ALIASES.get(k, k)

def load_font(size: int, bold=False):
    candidates = []
    if bold:
        candidates += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",
            "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf",
        ]
    else:
        candidates += [
            "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
            "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
        ]
    for p in candidates:
        try:
            return ImageFont.truetype(p, size=size)
        except:
            pass
    return ImageFont.load_default()

def draw_text_shadow(draw, xy, text, font, fill, shadow=(0,0,0,90), offset=(2,2), anchor="mm"):
    x,y = xy
    ox, oy = offset
    draw.text((x+ox, y+oy), text, font=font, fill=shadow, anchor=anchor)
    draw.text((x, y), text, font=font, fill=fill, anchor=anchor)

# =========================
# Escudos
# =========================
def list_shields(escudos_dir: Path):
    exts = {".png",".jpg",".jpeg",".webp"}
    return [p for p in escudos_dir.glob("*") if p.is_file() and p.suffix.lower() in exts]

def best_shield_file(team_name: str, escudos_dir: Path):
    team = canon_team(team_name)
    files = list_shields(escudos_dir)
    if not files:
        return None, team, None

    slug_map = {}
    for f in files:
        slug = norm(f.stem)
        slug_map[slug] = f

    if team in slug_map:
        return slug_map[team], team, "exact"

    # tentativa por "contém" (muito útil quando o nome do arquivo tem sufixos)
    for slug, fp in slug_map.items():
        if team in slug or slug in team:
            return fp, team, f"contains:{slug}"

    keys = list(slug_map.keys())
    close = difflib.get_close_matches(team, keys, n=1, cutoff=0.55)
    if close:
        return slug_map[close[0]], team, f"fuzzy:{close[0]}"

    return None, team, None

def open_shield(path: Path):
    return Image.open(path).convert("RGBA")

def fit_shield_inside_circle(shield_rgba: Image.Image, target_d: int, factor: float = 0.85):
    # mantém proporção SEM distorcer; factor reduz para não vazar (SPFC/FLA/BOT/Athletico etc)
    w,h = shield_rgba.size
    if w <= 0 or h <= 0:
        return shield_rgba
    max_side = int(target_d * factor)
    scale = min(max_side / w, max_side / h)
    nw, nh = max(1,int(w*scale)), max(1,int(h*scale))
    return shield_rgba.resize((nw,nh), Image.Resampling.LANCZOS)

def make_circle_token(diameter: int):
    # antialias forte (8x) para evitar pixelização no zoom
    up = 8
    D = diameter * up
    img = Image.new("RGBA", (D, D), (0,0,0,0))
    d = ImageDraw.Draw(img)
    d.ellipse((0,0,D-1,D-1), fill=(255,255,255,255))
    # downsample
    img = img.resize((diameter,diameter), Image.Resampling.LANCZOS)
    return img

# =========================
# CSV -> 24 colunas numéricas (ordem)
# =========================
def detect_home_away_cols(df: pd.DataFrame):
    cols_norm = {c: norm(c) for c in df.columns}
    home_col = None
    away_col = None

    for c,nc in cols_norm.items():
        if nc in ("casa","time_casa","mandante","home","clube_casa","clube mandante"):
            home_col = c
        if nc in ("fora","time_fora","visitante","away","clube_fora","clube visitante"):
            away_col = c

    if home_col is None:
        for c,nc in cols_norm.items():
            if "casa" in nc or "mandante" in nc:
                home_col = c; break
    if away_col is None:
        for c,nc in cols_norm.items():
            if "fora" in nc or "visitante" in nc:
                away_col = c; break

    if home_col is None or away_col is None:
        home_col = df.columns[0]
        away_col = df.columns[1]
    return home_col, away_col

def numeric_cols_in_order(df: pd.DataFrame, exclude):
    cols = []
    for c in df.columns:
        if c in exclude:
            continue
        s = pd.to_numeric(df[c], errors="coerce")
        # considera numérica se tiver pelo menos 1 valor numérico
        if s.notna().any():
            cols.append(c)
    return cols

def format_cell(v):
    if v is None:
        return ""
    if isinstance(v, float):
        # 1 casa quando necessário; remove .0
        s = f"{v:.1f}"
        if s.endswith(".0"):
            s = s[:-2]
        return s
    return str(v)

# =========================
# Render (layout do v13)
# =========================
def render(args):
    csv_path = Path(args.csv)
    escudos_dir = Path(args.escudos_dir)
    out_png = Path(args.out_png)
    out_log = Path(args.out_log) if args.out_log else None

    df = pd.read_csv(csv_path)

    # remove colunas “lixo” tipo POR_MANDO, se existirem
    drop_cols = [c for c in df.columns if norm(c) in ("por_mando","por mando")]
    if drop_cols:
        df = df.drop(columns=drop_cols)

    home_col, away_col = detect_home_away_cols(df)

    # pega 24 colunas numéricas na ordem do CSV (isso evita quebrar como no v14)
    num_cols = numeric_cols_in_order(df, exclude={home_col, away_col})
    if len(num_cols) < 24:
        # não quebra: só loga e preenche o que der
        pass
    # usa só as 24 primeiras
    num_cols = num_cols[:24]

    # mapeamento fixo por posição (24)
    # 0..11 = lado esquerdo (6 blocos x 2)
    # 12..23 = lado direito (6 blocos x 2)
    def get24(row, idx):
        if idx >= len(num_cols):
            return ""
        v = row[num_cols[idx]]
        if pd.isna(v):
            return ""
        return format_cell(float(v) if isinstance(v,(int,float)) else v)

    # =========================
    # CONFIG VISUAL (igual v13, com supersampling)
    # =========================
    scale = 3  # qualidade
    W0, H0 = 2600, 1450  # base (v13-like). Mantém proporção e evita “crescer tudo”
    W, H = W0*scale, H0*scale

    GREEN_BG = (18, 92, 70, 255)
    GREEN_HDR = (15, 92, 73, 255)
    BLUE = (160, 200, 230, 255)
    ORANGE = (230, 180, 140, 255)
    SUB_PINK = (240, 210, 220, 255)
    SUB_GRAY = (215, 220, 225, 255)
    WHITE = (255,255,255,255)
    BLACK = (0,0,0,255)

    BORDER = 3*scale
    GRID = 2*scale

    outer_margin = 35*scale
    inner_w = W - 2*outer_margin
    inner_h = H - 2*outer_margin

    mando_w = int(inner_w * 0.16)
    side_w = (inner_w - mando_w) // 2

    h1 = 92*scale   # AMEAÇAS/OPORTUNIDADES
    h2 = 62*scale   # CHUT.AG etc
    h3 = 46*scale   # COF/CDC etc
    data_h = inner_h - (h1+h2+h3)

    n_rows = len(df)
    row_h = data_h // max(1, n_rows)

    f_top = load_font(int(34*scale), bold=True)
    f_mid = load_font(int(24*scale), bold=True)
    f_sub = load_font(int(19*scale), bold=True)
    f_cell = load_font(int(22*scale), bold=True)
    f_x   = load_font(int(30*scale), bold=True)

    img = Image.new("RGBA", (W, H), GREEN_BG)
    draw = ImageDraw.Draw(img)

    x0 = outer_margin
    y0 = outer_margin
    x1 = x0 + inner_w
    y1 = y0 + inner_h

    draw.rectangle((x0,y0,x1,y1), outline=(0,0,0,255), width=BORDER)

    def rect(xa, ya, xb, yb, fill, w=GRID):
        draw.rectangle((xa,ya,xb,yb), fill=fill, outline=(0,0,0,255), width=w)

    left_x0 = x0
    left_x1 = left_x0 + side_w
    mando_x0 = left_x1
    mando_x1 = mando_x0 + mando_w
    right_x0 = mando_x1
    right_x1 = x1

    def draw_top_blocks(side_x0, side_x1):
        wside = side_x1 - side_x0
        wl = int(wside * 0.48)
        ax0 = side_x0
        ax1 = ax0 + wl
        ox0 = ax1
        ox1 = side_x1

        rect(ax0, y0, ax1, y0+h1, BLUE)
        rect(ox0, y0, ox1, y0+h1, ORANGE)

        # sombra pequena (em 3x fica nítida)
        draw_text_shadow(draw, ((ax0+ax1)//2, y0+h1//2), "AMEAÇAS", f_top, WHITE,
                         shadow=(0,0,0,70), offset=(2*scale,2*scale))
        draw_text_shadow(draw, ((ox0+ox1)//2, y0+h1//2), "OPORTUNIDADES", f_top, WHITE,
                         shadow=(0,0,0,70), offset=(2*scale,2*scale))

    draw_top_blocks(left_x0, left_x1)
    draw_top_blocks(right_x0, right_x1)

    # MANDO topo
    rect(mando_x0, y0, mando_x1, y0+h1, GREEN_HDR)
    draw_text_shadow(draw, ((mando_x0+mando_x1)//2, y0+h2//2), "MANDO", f_mid, WHITE,
                     shadow=(0,0,0,70), offset=(2*scale,2*scale))

    # Linha CHUT.AG etc
    headers = ["CHUT.AG","CHUT.PM","GOLS","DE","SG","% DE"]
    weights = [1.15, 1.15, 1.05, 1.0, 1.0, 1.20]
    wsum = sum(weights)

    def split_widths(total_w):
        widths = [int(total_w*(w/wsum)) for w in weights]
        widths[-1] += total_w - sum(widths)
        return widths

    left_ws = split_widths(side_w)
    right_ws = split_widths(side_w)

    def draw_mid_row(side_x0, widths):
        cx = side_x0
        for lab, ww in zip(headers, widths):
            rect(cx, y0+h1, cx+ww, y0+h1+h2, GREEN_HDR)
            draw_text_shadow(draw, (cx+ww//2, y0+h1+h2//2), lab, f_mid, WHITE,
                             shadow=(0,0,0,70), offset=(1*scale,1*scale))
            cx += ww

    draw_mid_row(left_x0, left_ws)
    draw_mid_row(right_x0, right_ws)

    rect(mando_x0, y0+h1, mando_x1, y0+h1+h2, GREEN_HDR)
    draw_text_shadow(draw, ((mando_x0+mando_x1)//2, y0+h1+h2//2), "CASA • FORA", f_mid, WHITE,
                     shadow=(0,0,0,70), offset=(1*scale,1*scale))

    # Subheaders: agora com divisão e cores CO/ CD
    # (exatamente como você pediu no seu último pedido)
    def draw_subheaders(side_x0, widths, side="L"):
        cx = side_x0
        if side == "L":
            pairs = [("COF","CDC"),("COF","CDC"),("COF","CDC"),("COC","CDF"),("COC","CDF"),("COC","CDF")]
        else:
            pairs = [("COC","CDF"),("COC","CDF"),("COC","CDF"),("COF","CDC"),("COF","CDC"),("COF","CDC")]

        for ww,(a,b) in zip(widths, pairs):
            half = ww//2
            fa = SUB_PINK if a.startswith("CO") else SUB_GRAY
            fb = SUB_PINK if b.startswith("CO") else SUB_GRAY

            rect(cx, y0+h1+h2, cx+half, y0+h1+h2+h3, fa)
            rect(cx+half, y0+h1+h2, cx+ww, y0+h1+h2+h3, fb)

            # centralizado
            draw.text((cx+half//2, y0+h1+h2+h3//2), a, font=f_sub, fill=BLACK, anchor="mm")
            draw.text((cx+half+(ww-half)//2, y0+h1+h2+h3//2), b, font=f_sub, fill=BLACK, anchor="mm")

            # divisão apenas nessa linha
            draw.line((cx+half, y0+h1+h2, cx+half, y0+h1+h2+h3), fill=(0,0,0,255), width=GRID)

            cx += ww

    draw_subheaders(left_x0, left_ws, "L")
    draw_subheaders(right_x0, right_ws, "R")
    rect(mando_x0, y0+h1+h2, mando_x1, y0+h1+h2+h3, GREEN_HDR)

    # Dados
    y_data0 = y0 + h1 + h2 + h3
    circle_d = int(row_h * 0.78)
    circle_img = make_circle_token(circle_d)

    log_lines = []
    def paste_shield(team_name, center_xy):
        nonlocal img
        fpath, teamcanon, how = best_shield_file(team_name, escudos_dir)
        token = circle_img.copy()

        if fpath is None:
            log_lines.append(f"ESCUDO_NAO_ENCONTRADO | recebido='{team_name}' | canon='{teamcanon}'")
            dd = ImageDraw.Draw(token)
            ff = load_font(int(18*scale), bold=True)
            dd.text((circle_d//2, circle_d//2), "?", font=ff, fill=(80,80,80,255), anchor="mm")
        else:
            shield = open_shield(fpath)
            shield_fit = fit_shield_inside_circle(shield, circle_d, factor=0.85)
            sx, sy = shield_fit.size
            ox = (circle_d - sx)//2
            oy = (circle_d - sy)//2
            token.alpha_composite(shield_fit, (ox,oy))

        px = int(center_xy[0] - circle_d/2)
        py = int(center_xy[1] - circle_d/2)
        img.alpha_composite(token, (px,py))

    # sem linha interna nas subcolunas no corpo (mantém seu padrão que estava bom)
    def draw_row_values(side_x0, widths, start_idx):
        cx = side_x0
        idx = start_idx
        for ww in widths:
            rect(cx, y_top, cx+ww, y_bot, WHITE)
            half = ww//2
            vA = get24(row, idx); vB = get24(row, idx+1)
            draw.text((cx+half//2, y_top+row_h//2), vA, font=f_cell, fill=BLACK, anchor="mm")
            draw.text((cx+half+(ww-half)//2, y_top+row_h//2), vB, font=f_cell, fill=BLACK, anchor="mm")
            # borda de separação entre blocos principais
            draw.line((cx+ww, y_top, cx+ww, y_bot), fill=(0,0,0,255), width=GRID)
            cx += ww
            idx += 2

    for i, row in df.iterrows():
        y_top = y_data0 + i*row_h
        y_bot = y_top + row_h

        # fundos
        rect(left_x0,  y_top, left_x1,  y_bot, WHITE)
        rect(right_x0, y_top, right_x1, y_bot, WHITE)
        rect(mando_x0, y_top, mando_x1, y_bot, GREEN_HDR)

        # valores
        draw_row_values(left_x0, left_ws, 0)
        draw_row_values(right_x0, right_ws, 12)

        # confronto
        home = str(row[home_col]) if home_col in row.index else ""
        away = str(row[away_col]) if away_col in row.index else ""
        cx_mid = (mando_x0+mando_x1)//2
        y_mid = y_top + row_h//2
        dx = int(mando_w*0.22)
        paste_shield(home, (cx_mid-dx, y_mid))
        paste_shield(away, (cx_mid+dx, y_mid))
        draw.text((cx_mid, y_mid), "×", font=f_x, fill=WHITE, anchor="mm")

    # log
    if out_log:
        out_log.parent.mkdir(parents=True, exist_ok=True)
        out_log.write_text("\n".join(log_lines), encoding="utf-8")

    out_png.parent.mkdir(parents=True, exist_ok=True)

    # downsample -> alta nitidez
    final = img.resize((W0, H0), Image.Resampling.LANCZOS)
    final.save(out_png, format="PNG", optimize=True)

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--csv", required=True)
    ap.add_argument("--escudos_dir", required=True)
    ap.add_argument("--out_png", required=True)
    ap.add_argument("--out_log", default="")
    args = ap.parse_args()
    render(args)

if __name__ == "__main__":
    main()
PY

echo "OK: v15 criado em 03_CONFIG"


OK: v15 criado em 03_CONFIG


In [ ]:
%%bash
set -e

RUN="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"
SCRIPT="/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v15.py"

python3 -u "$SCRIPT" \
  --csv "$RUN/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "$RUN/RENDER_GOLEIROS_v15.png" \
  --out_log "$RUN/LOG_RENDER_GOLEIROS_v15.txt"

echo "OK: gerou v15"


OK: gerou v15


In [ ]:
python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt"


SyntaxError: invalid syntax (ipython-input-2796012396.py, line 1)

In [ ]:
python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt"


SyntaxError: invalid syntax (ipython-input-2796012396.py, line 1)

In [ ]:
python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt"


SyntaxError: invalid syntax (ipython-input-2796012396.py, line 1)

In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_1.py" --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png" --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_1.py': [Errno 2] No such file or directory


In [ ]:
!ls -1 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG" | grep RENDER_GOLEIROS


ls: cannot access '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG': No such file or directory


In [ ]:
!ls -1 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG" | grep RENDER_GOLEIROS


ls: cannot access '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/TCC TABELAS 2026"


01_INPUT  02_OUTPUT  03_CONFIG	04_ESCUDOS  README_TCC.md


In [ ]:
!ls "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG" | grep v13


RENDER_GOLEIROS_TCC_v13_1.py
RENDER_GOLEIROS_TCC_v13.py


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt


In [ ]:
%%bash
set -e

python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt


In [ ]:
%%bash
set -e

python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_1.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_1.txt


In [ ]:
%%bash
set -e

# AJUSTE AQUI (confira no ls do passo 1)
CFG_DIR="/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG -"

OUT_FILE="$CFG_DIR/RENDER_GOLEIROS_TCC_v13_1.py"

cat > "$OUT_FILE" <<'PY'
# -*- coding: utf-8 -*-
import re
import unicodedata
import argparse
from pathlib import Path
import difflib

import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


def norm(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9\s\-_/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


ALIASES = {
    "atletico": "atletico mg",
    "atletico-mg": "atletico mg",
    "clube atletico mineiro": "atletico mg",
    "atletico mineiro": "atletico mg",

    "athletico-pr": "athletico pr",
    "athletico pr": "athletico pr",
    "athletico paranaense": "athletico pr",

    "red bull bragantino": "bragantino",
    "rb bragantino": "bragantino",
    "bragantino": "bragantino",

    "vasco da gama": "vasco",
    "grêmio": "gremio",
    "vitoria": "vitoria",
}


COLORS = {
    "bg": "#0d5b48",
    "bg2": "#0b4a3b",
    "grid": "#1a1a1a",
    "header_ameacas": "#9ec5e3",
    "header_oport": "#e6b08b",
    "header_row": "#0d5b48",
    "sub_co": "#e9d3da",   # rosado (conquistado)
    "sub_ce": "#cfd3d6",   # cinza  (cedido)
    "white": "#ffffff",
    "text": "#111111",
}


def guess_team_key(name: str, keys):
    n = norm(name)
    if n in ALIASES:
        n = ALIASES[n]
    if n in keys:
        return n
    # tenta aproximação
    cand = difflib.get_close_matches(n, list(keys), n=1, cutoff=0.55)
    return cand[0] if cand else None


def load_crest(path: Path, size_px: int):
    # carrega escudo e mantém proporção, encaixando num quadrado size_px
    im = Image.open(path).convert("RGBA")
    w, h = im.size
    scale = min(size_px / w, size_px / h)
    nw, nh = max(1, int(w * scale)), max(1, int(h * scale))
    im = im.resize((nw, nh), Image.LANCZOS)
    return im


def draw_table(df, escudos_dir: Path, out_png: Path):
    # layout base
    n_rows = len(df)
    dpi = 200
    fig_w = 16
    fig_h = 9.1

    fig = plt.figure(figsize=(fig_w, fig_h), dpi=dpi)
    ax = plt.axes([0, 0, 1, 1])
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis("off")

    # áreas
    margin = 0.02
    top = 1 - margin
    left = margin
    right = 1 - margin
    bottom = margin

    center_w = 0.165
    x_center0 = 0.5 - center_w / 2
    x_center1 = 0.5 + center_w / 2

    # blocos laterais
    xL0, xL1 = left, x_center0
    xR0, xR1 = x_center1, right

    # alturas
    h_title = 0.09
    h_header = 0.055
    h_sub = 0.045
    y_title0 = top - h_title
    y_header0 = y_title0 - h_header
    y_sub0 = y_header0 - h_sub

    # corpo
    y_body0 = y_sub0
    body_h = y_body0 - bottom
    row_h = body_h / n_rows

    # fundo geral
    ax.add_patch(Rectangle((left, bottom), right - left, top - bottom, facecolor=COLORS["bg"], edgecolor=COLORS["grid"], linewidth=2))

    # títulos topo (ameaças / oportunidades em cada lado)
    def title_pair(x0, x1):
        mid = (x0 + x1) / 2
        w = (x1 - x0)
        ax.add_patch(Rectangle((x0, y_title0), w * 0.5, h_title, facecolor=COLORS["header_ameacas"], edgecolor=COLORS["grid"], linewidth=2))
        ax.add_patch(Rectangle((x0 + w * 0.5, y_title0), w * 0.5, h_title, facecolor=COLORS["header_oport"], edgecolor=COLORS["grid"], linewidth=2))
        ax.text(x0 + w * 0.25, y_title0 + h_title / 2, "AMEAÇAS", ha="center", va="center", color="white", fontsize=26, fontweight="bold")
        ax.text(x0 + w * 0.75, y_title0 + h_title / 2, "OPORTUNIDADES", ha="center", va="center", color="white", fontsize=26, fontweight="bold")

    title_pair(xL0, xL1)
    title_pair(xR0, xR1)

    # centro topo (MANDO)
    ax.add_patch(Rectangle((x_center0, y_title0), center_w, h_title, facecolor=COLORS["bg2"], edgecolor=COLORS["grid"], linewidth=2))
    ax.text((x_center0 + x_center1) / 2, y_title0 + h_title / 2, "MANDO", ha="center", va="center", color="white", fontsize=22, fontweight="bold")

    # header row (CHUT.AG etc) — (mantido como no v13)
    headers = ["CHUT.AG", "CHUT.PM", "GOLS", "DE", "SG", "% DE"]
    # larguras por métrica (cada métrica tem 2 subcolunas)
    metric_ws = [0.18, 0.18, 0.16, 0.16, 0.16, 0.16]
    metric_ws = [w / sum(metric_ws) for w in metric_ws]

    def draw_metric_header(x0, x1):
        w = x1 - x0
        x = x0
        for lab, frac in zip(headers, metric_ws):
            mw = w * frac
            ax.add_patch(Rectangle((x, y_header0), mw, h_header, facecolor=COLORS["header_row"], edgecolor=COLORS["grid"], linewidth=2))
            ax.text(x + mw / 2, y_header0 + h_header / 2, lab, ha="center", va="center", color="white", fontsize=14, fontweight="bold")
            x += mw

    draw_metric_header(xL0, xL1)
    draw_metric_header(xR0, xR1)

    # centro header (CASA • FORA) — EXATAMENTE como você quer (FUNDO BRANCO, TEXTO PRETO)
    ax.add_patch(Rectangle((x_center0, y_header0), center_w, h_header, facecolor=COLORS["bg2"], edgecolor=COLORS["grid"], linewidth=2))
    ax.text((x_center0 + x_center1) / 2, y_header0 + h_header / 2, "CASA • FORA", ha="center", va="center", color="white", fontsize=15, fontweight="bold")

    # subheaders (COF/CDC etc) com cores por conquistado/cedido e borda entre eles
    subpairs_left = [("COF", "CDC"), ("COF", "CDC"), ("COF", "CDC"), ("COC", "CDF"), ("COC", "CDF"), ("COC", "CDF")]
    subpairs_right = [("COC", "CDF"), ("COC", "CDF"), ("COC", "CDF"), ("COF", "CDC"), ("COF", "CDC"), ("COF", "CDC")]

    def is_conquistado(s):  # CO*
        return s.startswith("CO")

    def draw_subheaders(x0, x1, pairs):
        w = x1 - x0
        x = x0
        for (a, b), frac in zip(pairs, metric_ws):
            mw = w * frac
            sw = mw / 2
            # A
            ax.add_patch(Rectangle((x, y_sub0), sw, h_sub,
                                   facecolor=COLORS["sub_co"] if is_conquistado(a) else COLORS["sub_ce"],
                                   edgecolor=COLORS["grid"], linewidth=1.8))
            ax.text(x + sw / 2, y_sub0 + h_sub / 2, a, ha="center", va="center", color=COLORS["text"], fontsize=12, fontweight="bold")
            # B
            ax.add_patch(Rectangle((x + sw, y_sub0), sw, h_sub,
                                   facecolor=COLORS["sub_co"] if is_conquistado(b) else COLORS["sub_ce"],
                                   edgecolor=COLORS["grid"], linewidth=1.8))
            ax.text(x + sw + sw / 2, y_sub0 + h_sub / 2, b, ha="center", va="center", color=COLORS["text"], fontsize=12, fontweight="bold")
            x += mw

    draw_subheaders(xL0, xL1, subpairs_left)
    draw_subheaders(xR0, xR1, subpairs_right)

    # subheader do centro: FUNDO BRANCO + TEXTO PRETO (A MUDANÇA QUE VOCÊ PEDIU)
    ax.add_patch(Rectangle((x_center0, y_sub0), center_w, h_sub,
                           facecolor=COLORS["white"], edgecolor=COLORS["grid"], linewidth=2))
    ax.text((x_center0 + x_center1) / 2, y_sub0 + h_sub / 2, "CASA • FORA",
            ha="center", va="center", color=COLORS["text"], fontsize=13, fontweight="bold")

    # grades e dados
    def draw_body_grid(x0, x1):
        w = x1 - x0
        x = x0
        # colunas por métrica (sem dividir COF/CDC etc aqui; o grid interno fica por célula)
        for frac in metric_ws:
            mw = w * frac
            # bordas da métrica
            ax.add_line(plt.Line2D([x, x], [bottom, y_body0], color=COLORS["grid"], linewidth=2))
            x += mw
        ax.add_line(plt.Line2D([x1, x1], [bottom, y_body0], color=COLORS["grid"], linewidth=2))

        # linhas horizontais
        for i in range(n_rows + 1):
            y = y_body0 - i * row_h
            ax.add_line(plt.Line2D([x0, x1], [y, y], color=COLORS["grid"], linewidth=1.3))

    draw_body_grid(xL0, xL1)
    draw_body_grid(xR0, xR1)

    # centro corpo fundo verde + linhas horizontais
    ax.add_patch(Rectangle((x_center0, bottom), center_w, y_body0 - bottom, facecolor=COLORS["bg2"], edgecolor=COLORS["grid"], linewidth=2))
    for i in range(n_rows + 1):
        y = y_body0 - i * row_h
        ax.add_line(plt.Line2D([x_center0, x_center1], [y, y], color=COLORS["grid"], linewidth=1.3))

    # coloca números (mantido simples; usa colunas do df)
    # Espera colunas nomeadas conforme sua pipeline atual.
    def put_side_values(x0, x1, cols_map, side="L"):
        w = x1 - x0
        for r in range(n_rows):
            y = y_body0 - r * row_h - row_h / 2
            x = x0
            for key, frac in zip(cols_map, metric_ws):
                mw = w * frac
                # subcolunas
                a, b = key
                sw = mw / 2
                va = df.iloc[r].get(a, "")
                vb = df.iloc[r].get(b, "")
                ax.text(x + sw / 2, y, f"{va}", ha="center", va="center", color=COLORS["text"], fontsize=12, fontweight="bold")
                ax.text(x + sw + sw / 2, y, f"{vb}", ha="center", va="center", color=COLORS["text"], fontsize=12, fontweight="bold")
                x += mw

    # ajuste estes nomes se seu CSV usa outros headers internos
    left_cols = [
        ("CHUT_AG_COF", "CHUT_AG_CDC"),
        ("CHUT_PM_COF", "CHUT_PM_CDC"),
        ("GOLS_COF", "GOLS_CDC"),
        ("DE_COC", "DE_CDF"),
        ("SG_COC", "SG_CDF"),
        ("PDE_COC", "PDE_CDF"),
    ]
    right_cols = [
        ("CHUT_AG_COC", "CHUT_AG_CDF"),
        ("CHUT_PM_COC", "CHUT_PM_CDF"),
        ("GOLS_COC", "GOLS_CDF"),
        ("DE_COF", "DE_CDC"),
        ("SG_COF", "SG_CDC"),
        ("PDE_COF", "PDE_CDC"),
    ]
    put_side_values(xL0, xL1, left_cols, "L")
    put_side_values(xR0, xR1, right_cols, "R")

    # escudos no centro
    size_badge = int(row_h * fig_h * dpi * 0.65)  # proporcional
    if size_badge < 26:
        size_badge = 26

    for r in range(n_rows):
        y_mid = y_body0 - r * row_h - row_h / 2
        # times
        t_home = str(df.iloc[r].get("TIME_CASA", "")).strip()
        t_away = str(df.iloc[r].get("TIME_FORA", "")).strip()

        def draw_badge(team, cx):
            if not team:
                return
            key = guess_team_key(team, [p.stem.lower() for p in escudos_dir.glob("*.png")])
            if not key:
                return
            # tenta achar arquivo
            cand = list(escudos_dir.glob(f"{key}.*"))
            if not cand:
                cand = list(escudos_dir.glob(f"{key}.png"))
            if not cand:
                return
            im = load_crest(cand[0], size_badge)
            # converte para array e plota
            import numpy as np
            arr = np.array(im)
            extent = [cx - 0.035, cx + 0.035, y_mid - row_h * 0.32, y_mid + row_h * 0.32]
            ax.imshow(arr, extent=extent, aspect="auto", zorder=5)

        # positions
        cx1 = x_center0 + center_w * 0.33
        cx2 = x_center0 + center_w * 0.67
        draw_badge(t_home, cx1)
        draw_badge(t_away, cx2)
        ax.text((cx1 + cx2) / 2, y_mid, "x", ha="center", va="center", color="white", fontsize=16, fontweight="bold", zorder=6)

    fig.savefig(out_png, dpi=dpi)
    plt.close(fig)


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--csv", required=True)
    ap.add_argument("--escudos_dir", required=True)
    ap.add_argument("--out_png", required=True)
    args = ap.parse_args()

    df = pd.read_csv(args.csv)
    draw_table(df, Path(args.escudos_dir), Path(args.out_png))


if __name__ == "__main__":
    main()
PY

echo "OK: escreveu $OUT_FILE"
ls -lh "$OUT_FILE"


bash: line 8: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG -/RENDER_GOLEIROS_TCC_v13_1.py: No such file or directory


CalledProcessError: Command 'b'set -e\n\n# AJUSTE AQUI (confira no ls do passo 1)\nCFG_DIR="/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG -"\n\nOUT_FILE="$CFG_DIR/RENDER_GOLEIROS_TCC_v13_1.py"\n\ncat > "$OUT_FILE" <<\'PY\'\n# -*- coding: utf-8 -*-\nimport re\nimport unicodedata\nimport argparse\nfrom pathlib import Path\nimport difflib\n\nimport pandas as pd\nfrom PIL import Image\nimport matplotlib.pyplot as plt\nfrom matplotlib.patches import Rectangle\n\n\ndef norm(s: str) -> str:\n    if s is None:\n        return ""\n    s = str(s).strip().lower()\n    s = unicodedata.normalize("NFKD", s)\n    s = "".join(ch for ch in s if not unicodedata.combining(ch))\n    s = re.sub(r"[^a-z0-9\\s\\-_/]", " ", s)\n    s = re.sub(r"\\s+", " ", s).strip()\n    return s\n\n\nALIASES = {\n    "atletico": "atletico mg",\n    "atletico-mg": "atletico mg",\n    "clube atletico mineiro": "atletico mg",\n    "atletico mineiro": "atletico mg",\n\n    "athletico-pr": "athletico pr",\n    "athletico pr"

In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_2.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_2.txt"


Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py", line 426, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py", line 422, in main
    draw_table(df, Path(args.escudos_dir), Path(args.out_png), Path(args.out_log))
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py", line 345, in draw_table
    ha="center", va="center", fontsize=fs_center_sub, fontweight="bold", color=COLORS["text"])
                                                                               ~~~~~~^^^^^^^^
KeyError: 'text'


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_2.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_2.txt"


Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py", line 426, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py", line 422, in main
    draw_table(df, Path(args.escudos_dir), Path(args.out_png), Path(args.out_log))
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py", line 345, in draw_table
    ha="center", va="center", fontsize=fs_center_sub, fontweight="bold", color=COLORS["text"])
                                                                               ~~~~~~^^^^^^^^
KeyError: 'text'


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_2.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_2.txt"


Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py", line 426, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py", line 422, in main
    draw_table(df, Path(args.escudos_dir), Path(args.out_png), Path(args.out_log))
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_2.py", line 345, in draw_table
    ha="center", va="center", fontsize=fs_center_sub, fontweight="bold", color=COLORS["text"])
                                                                               ~~~~~~^^^^^^^^
KeyError: 'text'


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_3.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_3.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_3.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_3.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_3.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_GOLEIROS_TCC_v13_4.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_GOLEIROS_R01.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_4.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_4.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_GOLEIROS_v13_4.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_GOLEIROS_v13_4.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/TABELA_ZAGUEIROS_RXX.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/RENDER_ZAGUEIROS.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/LOG_RENDER_ZAGUEIROS.txt"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py': [Errno 2] No such file or directory


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/TABELA_ZAGUEIROS_RXX.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/RENDER_ZAGUEIROS.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/LOG_RENDER_ZAGUEIROS.txt"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py': [Errno 2] No such file or directory


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/TABELA_ZAGUEIROS_RXX.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/RENDER_ZAGUEIROS.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/LOG_RENDER_ZAGUEIROS.txt"


python3: can't open file '/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py': [Errno 2] No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py" \
  --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/TABELA_ZAGUEIROS_RXX.csv" \
  --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS" \
  --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/RENDER_ZAGUEIROS.png" \
  --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/SUA_RUN_AQUI/LOG_RENDER_ZAGUEIROS.txt"


Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py", line 553, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py", line 549, in main
    render(Path(args.csv), Path(args.escudos_dir), Path(args.out_png), out_log)
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py", line 238, in render
    df = pd.read_csv(csv_path)
         ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1620, in 

In [ ]:
%%bash
set -e

BASE="/content/drive/MyDrive/TCC TABELAS 2026"
CFG_DIR="$BASE/03_CONFIG"
OUT_BASE="$BASE/02_OUTPUT"
ESCUDOS_DIR="$BASE/04_ESCUDOS"

echo "BASE:       $BASE"
echo "CFG_DIR:    $CFG_DIR"
echo "OUT_BASE:   $OUT_BASE"
echo "ESCUDOS:    $ESCUDOS_DIR"
echo

# 1) Confere pastas essenciais
test -d "$CFG_DIR" || { echo "ERRO: Pasta não existe: $CFG_DIR"; exit 1; }
test -d "$OUT_BASE" || { echo "ERRO: Pasta não existe: $OUT_BASE"; exit 1; }
test -d "$ESCUDOS_DIR" || { echo "ERRO: Pasta não existe: $ESCUDOS_DIR"; exit 1; }

# 2) Encontra a pasta RUN mais recente
RUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* 2>/dev/null | head -n 1)"
test -n "$RUN_DIR" || { echo "ERRO: Nenhuma pasta RUN_* encontrada em $OUT_BASE"; exit 1; }

echo "RUN detectado: $RUN_DIR"

# 3) Encontra o CSV de zagueiros dentro do RUN
CSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv 2>/dev/null | head -n 1)"
test -n "$CSV_FILE" || { echo "ERRO: Não achei TABELA_ZAGUEIROS_*.csv em $RUN_DIR"; ls -lh "$RUN_DIR"; exit 1; }

echo "CSV detectado: $CSV_FILE"

# 4) Encontra o script mais recente do renderer de zagueiros no 03_CONFIG
SCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py 2>/dev/null | head -n 1)"
test -n "$SCRIPT_FILE" || { echo "ERRO: Não achei RENDER_ZAGUEIROS_TCC_v*.py em $CFG_DIR"; ls -lh "$CFG_DIR"; exit 1; }

echo "SCRIPT detectado: $SCRIPT_FILE"
echo

# 5) Define saídas (dentro do próprio RUN)
OUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"
OUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"

# 6) Executa
python3 "$SCRIPT_FILE" \
  --csv "$CSV_FILE" \
  --escudos_dir "$ESCUDOS_DIR" \
  --out_png "$OUT_PNG" \
  --out_log "$OUT_LOG"

echo
echo "OK: gerado -> $OUT_PNG"
echo "OK: log   -> $OUT_LOG"
ls -lh "$OUT_PNG" "$OUT_LOG"


BASE:       /content/drive/MyDrive/TCC TABELAS 2026
CFG_DIR:    /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG
OUT_BASE:   /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT
ESCUDOS:    /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS

RUN detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
CSV detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv
SCRIPT detectado: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py



Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py", line 553, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py", line 549, in main
    render(Path(args.csv), Path(args.escudos_dir), Path(args.out_png), out_log)
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py", line 324, in render
    ax.add_patch(Rectangle((x0, y0), inner_w, inner_h, facecolor=GREEN_BG, edgecolor=GRID_COLOR, linewidth=LW_OUT))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/patches.py", line 772, in __init__
    super().__init__(**kwargs)
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/patches.py", line 84, in __init__
    self.set_edgecolor(edgecolor)
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/patches.py"

CalledProcessError: Command 'b'set -e\n\nBASE="/content/drive/MyDrive/TCC TABELAS 2026"\nCFG_DIR="$BASE/03_CONFIG"\nOUT_BASE="$BASE/02_OUTPUT"\nESCUDOS_DIR="$BASE/04_ESCUDOS"\n\necho "BASE:       $BASE"\necho "CFG_DIR:    $CFG_DIR"\necho "OUT_BASE:   $OUT_BASE"\necho "ESCUDOS:    $ESCUDOS_DIR"\necho\n\n# 1) Confere pastas essenciais\ntest -d "$CFG_DIR" || { echo "ERRO: Pasta n\xc3\xa3o existe: $CFG_DIR"; exit 1; }\ntest -d "$OUT_BASE" || { echo "ERRO: Pasta n\xc3\xa3o existe: $OUT_BASE"; exit 1; }\ntest -d "$ESCUDOS_DIR" || { echo "ERRO: Pasta n\xc3\xa3o existe: $ESCUDOS_DIR"; exit 1; }\n\n# 2) Encontra a pasta RUN mais recente\nRUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* 2>/dev/null | head -n 1)"\ntest -n "$RUN_DIR" || { echo "ERRO: Nenhuma pasta RUN_* encontrada em $OUT_BASE"; exit 1; }\n\necho "RUN detectado: $RUN_DIR"\n\n# 3) Encontra o CSV de zagueiros dentro do RUN\nCSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv 2>/dev/null | head -n 1)"\ntest -n "$CSV_FILE" || { echo "ERRO: N\xc3\x

In [ ]:
%%bash
set -e

BASE="/content/drive/MyDrive/TCC TABELAS 2026"
CFG_DIR="$BASE/03_CONFIG"
OUT_BASE="$BASE/02_OUTPUT"
ESCUDOS_DIR="$BASE/04_ESCUDOS"

echo "BASE:       $BASE"
echo "CFG_DIR:    $CFG_DIR"
echo "OUT_BASE:   $OUT_BASE"
echo "ESCUDOS:    $ESCUDOS_DIR"
echo

# 1) Confere pastas essenciais
test -d "$CFG_DIR" || { echo "ERRO: Pasta não existe: $CFG_DIR"; exit 1; }
test -d "$OUT_BASE" || { echo "ERRO: Pasta não existe: $OUT_BASE"; exit 1; }
test -d "$ESCUDOS_DIR" || { echo "ERRO: Pasta não existe: $ESCUDOS_DIR"; exit 1; }

# 2) Encontra a pasta RUN mais recente
RUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* 2>/dev/null | head -n 1)"
test -n "$RUN_DIR" || { echo "ERRO: Nenhuma pasta RUN_* encontrada em $OUT_BASE"; exit 1; }

echo "RUN detectado: $RUN_DIR"

# 3) Encontra o CSV de zagueiros dentro do RUN
CSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv 2>/dev/null | head -n 1)"
test -n "$CSV_FILE" || { echo "ERRO: Não achei TABELA_ZAGUEIROS_*.csv em $RUN_DIR"; ls -lh "$RUN_DIR"; exit 1; }

echo "CSV detectado: $CSV_FILE"

# 4) Encontra o script mais recente do renderer de zagueiros no 03_CONFIG
SCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py 2>/dev/null | head -n 1)"
test -n "$SCRIPT_FILE" || { echo "ERRO: Não achei RENDER_ZAGUEIROS_TCC_v*.py em $CFG_DIR"; ls -lh "$CFG_DIR"; exit 1; }

echo "SCRIPT detectado: $SCRIPT_FILE"
echo

# 5) Define saídas (dentro do próprio RUN)
OUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"
OUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"

# 6) Executa
python3 "$SCRIPT_FILE" \
  --csv "$CSV_FILE" \
  --escudos_dir "$ESCUDOS_DIR" \
  --out_png "$OUT_PNG" \
  --out_log "$OUT_LOG"

echo
echo "OK: gerado -> $OUT_PNG"
echo "OK: log   -> $OUT_LOG"
ls -lh "$OUT_PNG" "$OUT_LOG"


BASE:       /content/drive/MyDrive/TCC TABELAS 2026
CFG_DIR:    /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG
OUT_BASE:   /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT
ESCUDOS:    /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS

RUN detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
CSV detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv
SCRIPT detectado: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py



Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py", line 553, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py", line 549, in main
    render(Path(args.csv), Path(args.escudos_dir), Path(args.out_png), out_log)
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v1.py", line 324, in render
    ax.add_patch(Rectangle((x0, y0), inner_w, inner_h, facecolor=GREEN_BG, edgecolor=GRID_COLOR, linewidth=LW_OUT))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/patches.py", line 772, in __init__
    super().__init__(**kwargs)
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/patches.py", line 84, in __init__
    self.set_edgecolor(edgecolor)
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/patches.py"

CalledProcessError: Command 'b'set -e\n\nBASE="/content/drive/MyDrive/TCC TABELAS 2026"\nCFG_DIR="$BASE/03_CONFIG"\nOUT_BASE="$BASE/02_OUTPUT"\nESCUDOS_DIR="$BASE/04_ESCUDOS"\n\necho "BASE:       $BASE"\necho "CFG_DIR:    $CFG_DIR"\necho "OUT_BASE:   $OUT_BASE"\necho "ESCUDOS:    $ESCUDOS_DIR"\necho\n\n# 1) Confere pastas essenciais\ntest -d "$CFG_DIR" || { echo "ERRO: Pasta n\xc3\xa3o existe: $CFG_DIR"; exit 1; }\ntest -d "$OUT_BASE" || { echo "ERRO: Pasta n\xc3\xa3o existe: $OUT_BASE"; exit 1; }\ntest -d "$ESCUDOS_DIR" || { echo "ERRO: Pasta n\xc3\xa3o existe: $ESCUDOS_DIR"; exit 1; }\n\n# 2) Encontra a pasta RUN mais recente\nRUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* 2>/dev/null | head -n 1)"\ntest -n "$RUN_DIR" || { echo "ERRO: Nenhuma pasta RUN_* encontrada em $OUT_BASE"; exit 1; }\n\necho "RUN detectado: $RUN_DIR"\n\n# 3) Encontra o CSV de zagueiros dentro do RUN\nCSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv 2>/dev/null | head -n 1)"\ntest -n "$CSV_FILE" || { echo "ERRO: N\xc3\x

In [ ]:
%%bash
set -e

BASE="/content/drive/MyDrive/TCC TABELAS 2026"
CFG_DIR="$BASE/03_CONFIG"
OUT_BASE="$BASE/02_OUTPUT"
ESCUDOS_DIR="$BASE/04_ESCUDOS"

echo "BASE:       $BASE"
echo "CFG_DIR:    $CFG_DIR"
echo "OUT_BASE:   $OUT_BASE"
echo "ESCUDOS:    $ESCUDOS_DIR"
echo

# 1) Confere pastas essenciais
test -d "$CFG_DIR" || { echo "ERRO: Pasta não existe: $CFG_DIR"; exit 1; }
test -d "$OUT_BASE" || { echo "ERRO: Pasta não existe: $OUT_BASE"; exit 1; }
test -d "$ESCUDOS_DIR" || { echo "ERRO: Pasta não existe: $ESCUDOS_DIR"; exit 1; }

# 2) Encontra a pasta RUN mais recente
RUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* 2>/dev/null | head -n 1)"
test -n "$RUN_DIR" || { echo "ERRO: Nenhuma pasta RUN_* encontrada em $OUT_BASE"; exit 1; }

echo "RUN detectado: $RUN_DIR"

# 3) Encontra o CSV de zagueiros dentro do RUN
CSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv 2>/dev/null | head -n 1)"
test -n "$CSV_FILE" || { echo "ERRO: Não achei TABELA_ZAGUEIROS_*.csv em $RUN_DIR"; ls -lh "$RUN_DIR"; exit 1; }

echo "CSV detectado: $CSV_FILE"

# 4) Encontra o script mais recente do renderer de zagueiros no 03_CONFIG
SCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py 2>/dev/null | head -n 1)"
test -n "$SCRIPT_FILE" || { echo "ERRO: Não achei RENDER_ZAGUEIROS_TCC_v*.py em $CFG_DIR"; ls -lh "$CFG_DIR"; exit 1; }

echo "SCRIPT detectado: $SCRIPT_FILE"
echo

# 5) Define saídas (dentro do próprio RUN)
OUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"
OUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"

# 6) Executa
python3 "$SCRIPT_FILE" \
  --csv "$CSV_FILE" \
  --escudos_dir "$ESCUDOS_DIR" \
  --out_png "$OUT_PNG" \
  --out_log "$OUT_LOG"

echo
echo "OK: gerado -> $OUT_PNG"
echo "OK: log   -> $OUT_LOG"
ls -lh "$OUT_PNG" "$OUT_LOG"


BASE:       /content/drive/MyDrive/TCC TABELAS 2026
CFG_DIR:    /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG
OUT_BASE:   /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT
ESCUDOS:    /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS

RUN detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
CSV detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv
SCRIPT detectado: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v2.py


OK: gerado -> /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS.png
OK: log   -> /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_ZAGUEIROS.txt
-rw------- 1 root root  317 Jan  5 16:06 /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_ZAGUEIROS.txt
-rw------

In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v3.py"


OK: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS_v3.png


In [ ]:
%%bash
set -e

CFG_DIR="/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG"
OUT_FILE="$CFG_DIR/RENDER_ZAGUEIROS_TCC_v4.py"

cat > "$OUT_FILE" << 'PYCODE'
# ============================================================
# RENDER_ZAGUEIROS_TCC_v4.py
# Renderer de ZAGUEIROS
# PADRÃO VISUAL IDÊNTICO AO GOLEIROS v13_4
# ============================================================

import argparse
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from PIL import Image

# ---------------- CONFIG VISUAL GLOBAL (IGUAL GOLEIROS) ----------------
COLORS = {
    "bg": "#0d5b48",
    "grid": "#1a1a1a",
    "header": "#0d5b48",
    "sub_header_a": "#efd9de",   # rosa
    "sub_header_b": "#e3e8eb",   # cinza
    "title_text": "#0d5b48",
    "text": "#000000",
    "white": "#ffffff",
}

FONT_MAIN = "DejaVu Sans"

CELL_W = 1.0
CELL_H = 1.0
HEADER_H = 1.2
TITLE_H = 1.4
MANDO_W = 2.2

DPI = 300

# ---------------- UTIL ----------------
def draw_cell(ax, x, y, w, h, fc, ec=COLORS["grid"], lw=1):
    ax.add_patch(Rectangle((x, y), w, h, facecolor=fc, edgecolor=ec, linewidth=lw))

def draw_text(ax, x, y, w, h, text, size=10, weight="bold", color=COLORS["text"]):
    ax.text(x + w/2, y + h/2, text, ha="center", va="center",
            fontsize=size, fontweight=weight, color=color, family=FONT_MAIN)

def draw_badge(ax, cx, cy, r, img_path):
    circ = Circle((cx, cy), r, facecolor=COLORS["white"], edgecolor=COLORS["grid"], linewidth=1)
    ax.add_patch(circ)
    if img_path.exists():
        img = Image.open(img_path).convert("RGBA")
        ax.imshow(img, extent=(cx-r*0.9, cx+r*0.9, cy-r*0.9, cy+r*0.9), zorder=10)

# ---------------- MAIN ----------------
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--csv", required=True)
    parser.add_argument("--escudos_dir", required=True)
    parser.add_argument("--out_png", required=True)
    parser.add_argument("--out_log", required=True)
    args = parser.parse_args()

    df = pd.read_csv(args.csv)

    cols_left = ["SG", "DESARME", "CHUTES", "PONTOS", "MÉDIA BÁSICA"]
    cols_right = ["SG", "DESARME", "CHUTES", "PONTOS", "MÉDIA BÁSICA"]

    n_rows = len(df)
    total_w = (len(cols_left)*2 + len(cols_right)*2)*CELL_W + MANDO_W
    total_h = TITLE_H + HEADER_H*2 + n_rows*CELL_H

    fig, ax = plt.subplots(figsize=(total_w*0.6, total_h*0.6), dpi=DPI)
    ax.set_xlim(0, total_w)
    ax.set_ylim(0, total_h)
    ax.axis("off")

    # Fundo
    ax.add_patch(Rectangle((0, 0), total_w, total_h, facecolor=COLORS["bg"]))

    y = total_h - TITLE_H
    draw_cell(ax, 0, y, total_w, TITLE_H, COLORS["white"])
    draw_text(ax, 0, y, total_w, TITLE_H, "ANÁLISE ESTATÍSTICA DOS ZAGUEIROS", size=18, color=COLORS["title_text"])

    y -= HEADER_H
    x = 0
    for c in cols_left:
        draw_cell(ax, x, y, CELL_W*2, HEADER_H, COLORS["header"])
        draw_text(ax, x, y, CELL_W*2, HEADER_H, c, size=11, color=COLORS["white"])
        x += CELL_W*2

    draw_cell(ax, x, y, MANDO_W, HEADER_H, COLORS["header"])
    draw_text(ax, x, y, MANDO_W, HEADER_H, "MANDO", size=11, color=COLORS["white"])
    x += MANDO_W

    for c in cols_right:
        draw_cell(ax, x, y, CELL_W*2, HEADER_H, COLORS["header"])
        draw_text(ax, x, y, CELL_W*2, HEADER_H, c, size=11, color=COLORS["white"])
        x += CELL_W*2

    y -= HEADER_H
    x = 0
    for _ in range(len(cols_left)*2):
        draw_cell(ax, x, y, CELL_W, HEADER_H, COLORS["sub_header_a"] if int(x/CELL_W)%2==0 else COLORS["sub_header_b"])
        draw_text(ax, x, y, CELL_W, HEADER_H, "COC" if int(x/CELL_W)%2==0 else "CDF", size=9)
        x += CELL_W

    draw_cell(ax, x, y, MANDO_W, HEADER_H, COLORS["white"])
    draw_text(ax, x, y, MANDO_W, HEADER_H, "CASA • FORA", size=10)
    x += MANDO_W

    for _ in range(len(cols_right)*2):
        draw_cell(ax, x, y, CELL_W, HEADER_H, COLORS["sub_header_a"] if int(x/CELL_W)%2==0 else COLORS["sub_header_b"])
        draw_text(ax, x, y, CELL_W, HEADER_H, "COF" if int(x/CELL_W)%2==0 else "CDC", size=9)
        x += CELL_W

    y -= CELL_H
    esc_dir = Path(args.escudos_dir)

    for _, row in df.iterrows():
        x = 0
        for _ in range(len(cols_left)*2):
            draw_cell(ax, x, y, CELL_W, CELL_H, COLORS["white"])
            x += CELL_W

        draw_cell(ax, x, y, MANDO_W, CELL_H, COLORS["header"])
        cx1 = x + MANDO_W*0.3
        cx2 = x + MANDO_W*0.7
        cy = y + CELL_H/2
        draw_badge(ax, cx1, cy, 0.32, esc_dir / f"{row['CASA']}.png")
        draw_badge(ax, cx2, cy, 0.32, esc_dir / f"{row['FORA']}.png")
        ax.text(x + MANDO_W/2, cy, "X", ha="center", va="center", color="white", fontsize=12, fontweight="bold")
        x += MANDO_W

        for _ in range(len(cols_right)*2):
            draw_cell(ax, x, y, CELL_W, CELL_H, COLORS["white"])
            x += CELL_W

        y -= CELL_H

    plt.savefig(args.out_png, bbox_inches="tight")
    plt.close()

    with open(args.out_log, "w") as f:
        f.write("Render ZAGUEIROS v4 OK\n")

if __name__ == "__main__":
    main()
PYCODE

echo "OK — arquivo criado:"
ls -lh "$OUT_FILE"


OK — arquivo criado:
-rw------- 1 root root 4.9K Jan  5 17:47 /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v4.py


In [ ]:
%%bash
set -e

BASE="/content/drive/MyDrive/TCC TABELAS 2026"
CFG_DIR="$BASE/03_CONFIG"
OUT_BASE="$BASE/02_OUTPUT"
ESCUDOS_DIR="$BASE/04_ESCUDOS"

echo "BASE:       $BASE"
echo "CFG_DIR:    $CFG_DIR"
echo "OUT_BASE:   $OUT_BASE"
echo "ESCUDOS:    $ESCUDOS_DIR"
echo

# 1) Pasta RUN mais recente
RUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* | head -n 1)"
echo "RUN detectado: $RUN_DIR"

# 2) CSV de zagueiros da rodada
CSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv | head -n 1)"
echo "CSV detectado: $CSV_FILE"

# 3) Renderer MAIS RECENTE de zagueiros
SCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py | head -n 1)"
echo "SCRIPT detectado: $SCRIPT_FILE"
echo

# 4) Saídas
OUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"
OUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"

# 5) Executa
python3 "$SCRIPT_FILE" \
  --csv "$CSV_FILE" \
  --escudos_dir "$ESCUDOS_DIR" \
  --out_png "$OUT_PNG" \
  --out_log "$OUT_LOG"

echo
echo "OK: PNG gerado -> $OUT_PNG"
echo "OK: LOG gerado -> $OUT_LOG"
ls -lh "$OUT_PNG" "$OUT_LOG"


BASE:       /content/drive/MyDrive/TCC TABELAS 2026
CFG_DIR:    /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG
OUT_BASE:   /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT
ESCUDOS:    /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS

RUN detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
CSV detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv
SCRIPT detectado: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v4.py



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'CASA'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v4.py", line 144, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v4.py", line 126, in main
    draw_badge(ax, cx1, cy, 0.32, esc_dir / f"{row['CASA'

CalledProcessError: Command 'b'set -e\n\nBASE="/content/drive/MyDrive/TCC TABELAS 2026"\nCFG_DIR="$BASE/03_CONFIG"\nOUT_BASE="$BASE/02_OUTPUT"\nESCUDOS_DIR="$BASE/04_ESCUDOS"\n\necho "BASE:       $BASE"\necho "CFG_DIR:    $CFG_DIR"\necho "OUT_BASE:   $OUT_BASE"\necho "ESCUDOS:    $ESCUDOS_DIR"\necho\n\n# 1) Pasta RUN mais recente\nRUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* | head -n 1)"\necho "RUN detectado: $RUN_DIR"\n\n# 2) CSV de zagueiros da rodada\nCSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv | head -n 1)"\necho "CSV detectado: $CSV_FILE"\n\n# 3) Renderer MAIS RECENTE de zagueiros\nSCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py | head -n 1)"\necho "SCRIPT detectado: $SCRIPT_FILE"\necho\n\n# 4) Sa\xc3\xaddas\nOUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"\nOUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"\n\n# 5) Executa\npython3 "$SCRIPT_FILE" \\\n  --csv "$CSV_FILE" \\\n  --escudos_dir "$ESCUDOS_DIR" \\\n  --out_png "$OUT_PNG" \\\n  --out_log "$OUT_LOG"\n\necho\necho "OK: PNG ger

In [ ]:
%%bash
set -e

BASE="/content/drive/MyDrive/TCC TABELAS 2026"
CFG_DIR="$BASE/03_CONFIG"
OUT_BASE="$BASE/02_OUTPUT"
ESCUDOS_DIR="$BASE/04_ESCUDOS"

echo "BASE:       $BASE"
echo "CFG_DIR:    $CFG_DIR"
echo "OUT_BASE:   $OUT_BASE"
echo "ESCUDOS:    $ESCUDOS_DIR"
echo

# 1) Pasta RUN mais recente
RUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* | head -n 1)"
echo "RUN detectado: $RUN_DIR"

# 2) CSV de zagueiros da rodada
CSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv | head -n 1)"
echo "CSV detectado: $CSV_FILE"

# 3) Renderer MAIS RECENTE de zagueiros
SCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py | head -n 1)"
echo "SCRIPT detectado: $SCRIPT_FILE"
echo

# 4) Saídas
OUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"
OUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"

# 5) Executa
python3 "$SCRIPT_FILE" \
  --csv "$CSV_FILE" \
  --escudos_dir "$ESCUDOS_DIR" \
  --out_png "$OUT_PNG" \
  --out_log "$OUT_LOG"

echo
echo "OK: PNG gerado -> $OUT_PNG"
echo "OK: LOG gerado -> $OUT_LOG"
ls -lh "$OUT_PNG" "$OUT_LOG"


BASE:       /content/drive/MyDrive/TCC TABELAS 2026
CFG_DIR:    /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG
OUT_BASE:   /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT
ESCUDOS:    /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS

RUN detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
CSV detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv
SCRIPT detectado: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v5.py


OK: PNG gerado -> /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS.png
OK: LOG gerado -> /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_ZAGUEIROS.txt
-rw------- 1 root root  535 Jan  5 18:03 /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_ZAGUEIROS.txt


/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v5.py:191: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  pil = Image.fromarray(img8, mode="RGBA").resize((d, d), resample=Image.LANCZOS)


In [ ]:
%%bash
set -e

BASE="/content/drive/MyDrive/TCC TABELAS 2026"
CFG_DIR="$BASE/03_CONFIG"
OUT_BASE="$BASE/02_OUTPUT"
ESCUDOS_DIR="$BASE/04_ESCUDOS"

echo "BASE:       $BASE"
echo "CFG_DIR:    $CFG_DIR"
echo "OUT_BASE:   $OUT_BASE"
echo "ESCUDOS:    $ESCUDOS_DIR"
echo

RUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* | head -n 1)"
echo "RUN detectado: $RUN_DIR"

CSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv | head -n 1)"
echo "CSV detectado: $CSV_FILE"

SCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py | head -n 1)"
echo "SCRIPT detectado: $SCRIPT_FILE"
echo

OUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"
OUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"

python3 "$SCRIPT_FILE" \
  --csv "$CSV_FILE" \
  --escudos_dir "$ESCUDOS_DIR" \
  --out_png "$OUT_PNG" \
  --out_log "$OUT_LOG"

echo
echo "OK: PNG gerado -> $OUT_PNG"
echo "OK: LOG gerado -> $OUT_LOG"
ls -lh "$OUT_PNG" "$OUT_LOG"


BASE:       /content/drive/MyDrive/TCC TABELAS 2026
CFG_DIR:    /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG
OUT_BASE:   /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT
ESCUDOS:    /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS

RUN detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
CSV detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv
SCRIPT detectado: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v6.py



Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v6.py", line 496, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v6.py", line 492, in main
    draw_table(df, Path(args.escudos_dir), Path(args.out_png), Path(args.out_log))
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v6.py", line 378, in draw_table
    draw_group_label(x, col_w * 2, label)
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v6.py", line 371, in draw_group_label
    ax.add_patch(Rectangle((x, y0), w, header_h0, facecolor=COLORS['header_row'], edgecolor=COLORS['grid'], linewidth=3))
                                                            ~~~~~~^^^^^^^^^^^^^^
KeyError: 'header_row'


CalledProcessError: Command 'b'set -e\n\nBASE="/content/drive/MyDrive/TCC TABELAS 2026"\nCFG_DIR="$BASE/03_CONFIG"\nOUT_BASE="$BASE/02_OUTPUT"\nESCUDOS_DIR="$BASE/04_ESCUDOS"\n\necho "BASE:       $BASE"\necho "CFG_DIR:    $CFG_DIR"\necho "OUT_BASE:   $OUT_BASE"\necho "ESCUDOS:    $ESCUDOS_DIR"\necho\n\nRUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* | head -n 1)"\necho "RUN detectado: $RUN_DIR"\n\nCSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv | head -n 1)"\necho "CSV detectado: $CSV_FILE"\n\nSCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py | head -n 1)"\necho "SCRIPT detectado: $SCRIPT_FILE"\necho\n\nOUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"\nOUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"\n\npython3 "$SCRIPT_FILE" \\\n  --csv "$CSV_FILE" \\\n  --escudos_dir "$ESCUDOS_DIR" \\\n  --out_png "$OUT_PNG" \\\n  --out_log "$OUT_LOG"\n\necho\necho "OK: PNG gerado -> $OUT_PNG"\necho "OK: LOG gerado -> $OUT_LOG"\nls -lh "$OUT_PNG" "$OUT_LOG"\n'' returned non-zero exit status 1.

In [ ]:
%%bash
set -e

BASE="/content/drive/MyDrive/TCC TABELAS 2026"
CFG_DIR="$BASE/03_CONFIG"
OUT_BASE="$BASE/02_OUTPUT"
ESCUDOS_DIR="$BASE/04_ESCUDOS"

echo "BASE:       $BASE"
echo "CFG_DIR:    $CFG_DIR"
echo "OUT_BASE:   $OUT_BASE"
echo "ESCUDOS:    $ESCUDOS_DIR"
echo

RUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* | head -n 1)"
echo "RUN detectado: $RUN_DIR"

CSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv | head -n 1)"
echo "CSV detectado: $CSV_FILE"

SCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py | head -n 1)"
echo "SCRIPT detectado: $SCRIPT_FILE"
echo

OUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"
OUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"

python3 "$SCRIPT_FILE" \
  --csv "$CSV_FILE" \
  --escudos_dir "$ESCUDOS_DIR" \
  --out_png "$OUT_PNG" \
  --out_log "$OUT_LOG"

echo
echo "OK: PNG gerado -> $OUT_PNG"
echo "OK: LOG gerado -> $OUT_LOG"
ls -lh "$OUT_PNG" "$OUT_LOG"


BASE:       /content/drive/MyDrive/TCC TABELAS 2026
CFG_DIR:    /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG
OUT_BASE:   /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT
ESCUDOS:    /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS

RUN detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
CSV detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv
SCRIPT detectado: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v7.py



  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v7.py", line 165
    "header_row": "#165f4b",,
                            ^
SyntaxError: invalid syntax


CalledProcessError: Command 'b'set -e\n\nBASE="/content/drive/MyDrive/TCC TABELAS 2026"\nCFG_DIR="$BASE/03_CONFIG"\nOUT_BASE="$BASE/02_OUTPUT"\nESCUDOS_DIR="$BASE/04_ESCUDOS"\n\necho "BASE:       $BASE"\necho "CFG_DIR:    $CFG_DIR"\necho "OUT_BASE:   $OUT_BASE"\necho "ESCUDOS:    $ESCUDOS_DIR"\necho\n\nRUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* | head -n 1)"\necho "RUN detectado: $RUN_DIR"\n\nCSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv | head -n 1)"\necho "CSV detectado: $CSV_FILE"\n\nSCRIPT_FILE="$(ls -1t "$CFG_DIR"/RENDER_ZAGUEIROS_TCC_v*.py | head -n 1)"\necho "SCRIPT detectado: $SCRIPT_FILE"\necho\n\nOUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"\nOUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"\n\npython3 "$SCRIPT_FILE" \\\n  --csv "$CSV_FILE" \\\n  --escudos_dir "$ESCUDOS_DIR" \\\n  --out_png "$OUT_PNG" \\\n  --out_log "$OUT_LOG"\n\necho\necho "OK: PNG gerado -> $OUT_PNG"\necho "OK: LOG gerado -> $OUT_LOG"\nls -lh "$OUT_PNG" "$OUT_LOG"\n'' returned non-zero exit status 1.

In [ ]:
%%bash
set -e

BASE="/content/drive/MyDrive/TCC TABELAS 2026"
CFG_DIR="$BASE/03_CONFIG"
OUT_BASE="$BASE/02_OUTPUT"
ESCUDOS_DIR="$BASE/04_ESCUDOS"

echo "BASE:       $BASE"
echo "CFG_DIR:    $CFG_DIR"
echo "OUT_BASE:   $OUT_BASE"
echo "ESCUDOS:    $ESCUDOS_DIR"
echo

# 1) Confere pastas essenciais
test -d "$CFG_DIR" || { echo "ERRO: Pasta não existe: $CFG_DIR"; exit 1; }
test -d "$OUT_BASE" || { echo "ERRO: Pasta não existe: $OUT_BASE"; exit 1; }
test -d "$ESCUDOS_DIR" || { echo "ERRO: Pasta não existe: $ESCUDOS_DIR"; exit 1; }

# 2) Encontra a pasta RUN mais recente
RUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* 2>/dev/null | head -n 1)"
test -n "$RUN_DIR" || { echo "ERRO: Nenhuma pasta RUN_* encontrada em $OUT_BASE"; exit 1; }
echo "RUN detectado: $RUN_DIR"

# 3) Encontra o CSV de zagueiros dentro do RUN
CSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv 2>/dev/null | head -n 1)"
test -n "$CSV_FILE" || { echo "ERRO: Não achei TABELA_ZAGUEIROS_*.csv em $RUN_DIR"; ls -lh "$RUN_DIR"; exit 1; }
echo "CSV detectado: $CSV_FILE"

# 4) Usa ESPECIFICAMENTE o v8 (sem depender de “mais recente”)
SCRIPT_FILE="$CFG_DIR/RENDER_ZAGUEIROS_TCC_v8.py"
test -f "$SCRIPT_FILE" || { echo "ERRO: Não achei $SCRIPT_FILE"; ls -lh "$CFG_DIR"; exit 1; }
echo "SCRIPT: $SCRIPT_FILE"
echo

# 5) Saídas (dentro do próprio RUN)
OUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"
OUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"

# 6) Executa
python3 "$SCRIPT_FILE" \
  --csv "$CSV_FILE" \
  --escudos_dir "$ESCUDOS_DIR" \
  --out_png "$OUT_PNG" \
  --out_log "$OUT_LOG"

echo
echo "OK: gerado -> $OUT_PNG"
echo "OK: log   -> $OUT_LOG"
ls -lh "$OUT_PNG" "$OUT_LOG"


BASE:       /content/drive/MyDrive/TCC TABELAS 2026
CFG_DIR:    /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG
OUT_BASE:   /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT
ESCUDOS:    /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS

RUN detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
CSV detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv
SCRIPT: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v8.py


OK: gerado -> /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS.png
OK: log   -> /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_ZAGUEIROS.txt
-rw------- 1 root root  422 Jan  5 18:55 /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_ZAGUEIROS.txt
-rw------- 1 root r

In [ ]:
%%bash
set -e

BASE="/content/drive/MyDrive/TCC TABELAS 2026"
CFG_DIR="$BASE/03_CONFIG"
OUT_BASE="$BASE/02_OUTPUT"
ESCUDOS_DIR="$BASE/04_ESCUDOS"

echo "BASE:       $BASE"
echo "CFG_DIR:    $CFG_DIR"
echo "OUT_BASE:   $OUT_BASE"
echo "ESCUDOS:    $ESCUDOS_DIR"
echo

# 1) Confere pastas essenciais
test -d "$CFG_DIR" || { echo "ERRO: Pasta não existe: $CFG_DIR"; exit 1; }
test -d "$OUT_BASE" || { echo "ERRO: Pasta não existe: $OUT_BASE"; exit 1; }
test -d "$ESCUDOS_DIR" || { echo "ERRO: Pasta não existe: $ESCUDOS_DIR"; exit 1; }

# 2) Encontra a pasta RUN mais recente
RUN_DIR="$(ls -dt "$OUT_BASE"/RUN_* 2>/dev/null | head -n 1)"
test -n "$RUN_DIR" || { echo "ERRO: Nenhuma pasta RUN_* encontrada em $OUT_BASE"; exit 1; }
echo "RUN detectado: $RUN_DIR"

# 3) Encontra o CSV de zagueiros dentro do RUN
CSV_FILE="$(ls -1 "$RUN_DIR"/TABELA_ZAGUEIROS_*.csv 2>/dev/null | head -n 1)"
test -n "$CSV_FILE" || { echo "ERRO: Não achei TABELA_ZAGUEIROS_*.csv em $RUN_DIR"; ls -lh "$RUN_DIR"; exit 1; }
echo "CSV detectado: $CSV_FILE"

# 4) Usa ESPECIFICAMENTE o v8 (sem depender de “mais recente”)
SCRIPT_FILE="$CFG_DIR/RENDER_ZAGUEIROS_TCC_v8.py"
test -f "$SCRIPT_FILE" || { echo "ERRO: Não achei $SCRIPT_FILE"; ls -lh "$CFG_DIR"; exit 1; }
echo "SCRIPT: $SCRIPT_FILE"
echo

# 5) Saídas (dentro do próprio RUN)
OUT_PNG="$RUN_DIR/RENDER_ZAGUEIROS.png"
OUT_LOG="$RUN_DIR/LOG_RENDER_ZAGUEIROS.txt"

# 6) Executa
python3 "$SCRIPT_FILE" \
  --csv "$CSV_FILE" \
  --escudos_dir "$ESCUDOS_DIR" \
  --out_png "$OUT_PNG" \
  --out_log "$OUT_LOG"

echo
echo "OK: gerado -> $OUT_PNG"
echo "OK: log   -> $OUT_LOG"
ls -lh "$OUT_PNG" "$OUT_LOG"


BASE:       /content/drive/MyDrive/TCC TABELAS 2026
CFG_DIR:    /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG
OUT_BASE:   /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT
ESCUDOS:    /content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS

RUN detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847
CSV detectado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv
SCRIPT: /content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_v8.py


OK: gerado -> /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS.png
OK: log   -> /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_ZAGUEIROS.txt
-rw------- 1 root root  422 Jan  5 19:10 /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_RENDER_ZAGUEIROS.txt
-rw------- 1 root r

In [ ]:
# --- COMANDO PARA RENDERIZAR ZAGUEIROS ---

# Ajuste o caminho para o CSV da rodada que você quer renderizar
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_FINAL.py" \
    --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_R01_2026-01-01/TABELA_ZAGUEIROS_R01.csv" \
    --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS/" \
    --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_R01_2026-01-01/RENDER_ZAGUEIROS_FINAL.png" \
    --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_R01_2026-01-01/LOG_ZAGUEIROS.txt"

print("\nRenderização concluída!")

Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_FINAL.py", line 609, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_FINAL.py", line 604, in main
    df = pd.read_csv(args.csv)
         ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/

In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_FINAL.py" \
    --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv" \
    --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS/" \
    --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS_FINAL.png" \
    --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ZAGUEIROS.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS_FINAL.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ZAGUEIROS.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_FINAL.py" \
    --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv" \
    --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS/" \
    --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS_FINAL.png" \
    --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ZAGUEIROS.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS_FINAL.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ZAGUEIROS.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_FINAL.py" \
    --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv" \
    --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS/" \
    --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS_FINAL.png" \
    --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ZAGUEIROS.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS_FINAL.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ZAGUEIROS.txt


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/RENDER_ZAGUEIROS_TCC_FINAL.py" \
    --csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_ZAGUEIROS_R01.csv" \
    --escudos_dir "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS/" \
    --out_png "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS_FINAL.png" \
    --out_log "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ZAGUEIROS.txt"


PNG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/RENDER_ZAGUEIROS_FINAL.png
LOG gerado: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/LOG_ZAGUEIROS.txt


In [ ]:
RUN_DIR="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"

python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "SCOUTS_REORGANIZADO" \
  --fixtures_csv "${RUN_DIR}/TABELA_LATERAIS_R01.csv" \
  --n_jogos 5 \
  --out_csv "${RUN_DIR}/TABELA_LATERAIS_R01_CORRETA.csv"


SyntaxError: invalid syntax (ipython-input-145659504.py, line 3)

In [ ]:
!RUN_DIR="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"

!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "SCOUTS_REORGANIZADO" \
  --fixtures_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01.csv" \
  --n_jogos 5 \
  --out_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01_CORRETA.csv"



ERRO: Worksheet named 'SCOUTS_REORGANIZADO' not found


In [ ]:
!python3 - << 'EOF'
import pandas as pd

xls = pd.ExcelFile("/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx")
print(xls.sheet_names)
EOF


/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')
['Scouts', 'Por jogo']


NameError: name 'EOF' is not defined

In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --fixtures_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01.csv" \
  --n_jogos 5 \
  --out_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01_CORRETA.csv"



ERRO: Não consegui detectar a coluna obrigatória 'match_id'. Colunas disponíveis: ['ID', 'Time', 'Adversário', 'Posreal', 'Mando', 'Novo?', 'Nome', 'PosID', 'ClubeID', 'G', 'A', 'FT', 'SG', 'DP', 'GC', 'CV', 'DE', 'FD', 'FF', 'FS', 'DS', 'PP', 'CA', 'GS', 'FC', 'I', 'PS', 'PC', 'PI', 'Bás Tec', 'ID.1', 'Pts', 'Básica', 'Data'] ...


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --fixtures_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01.csv" \
  --n_jogos 5 \
  --out_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01_CORRETA.csv" \
  --col_match_id "ID.1" \
  --col_team "Time" \
  --col_opp "Adversário" \
  --col_is_home "Mando" \
  --col_posreal "Posreal" \
  --col_de "DE" \
  --col_g "G" \
  --col_a "A" \
  --col_basica "Básica"



ERRO: Não encontrei colunas suficientes para calcular SG. Preciso de (GF+GA) ou (GOLS_CASA+GOLS_FORA).


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --fixtures_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01.csv" \
  --n_jogos 5 \
  --out_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01_CORRETA.csv" \
  --col_match_id "ID" \
  --col_team "Time" \
  --col_opp "Adversário" \
  --col_is_home "Mando" \
  --col_posreal "Posreal" \
  --col_de "DE" \
  --col_g "G" \
  --col_a "A" \
  --col_basica "Básica"



ERRO: Não encontrei colunas suficientes para calcular SG. Preciso de (GF+GA) ou (GOLS_CASA+GOLS_FORA).


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --fixtures_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01.csv" \
  --n_jogos 5 \
  --out_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01_CORRETA.csv" \
  --col_match_id "ID.1" \
  --col_team "Time" \
  --col_opp "Adversário" \
  --col_is_home "Mando" \
  --col_posreal "Posreal" \
  --col_de "DE" \
  --col_g "G" \
  --col_a "A" \
  --col_basica "Básica"



ERRO: 'Namespace' object has no attribute 'col_sg_scout'


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --fixtures_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01.csv" \
  --n_jogos 5 \
  --out_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01_CORRETA.csv" \
  --col_match_id "ID.1" \
  --col_team "Time" \
  --col_opp "Adversário" \
  --col_is_home "Mando" \
  --col_posreal "Posreal" \
  --col_de "DE" \
  --col_g "G" \
  --col_a "A" \
  --col_basica "Básica" \
  --col_sg_scout "SG"


usage: GERAR_TABELA_LATERAIS_TCC_FINAL.py [-h] --excel EXCEL [--sheet SHEET]
                                          --fixtures_csv FIXTURES_CSV
                                          [--n_jogos N_JOGOS] --out_csv
                                          OUT_CSV
                                          [--col_match_id COL_MATCH_ID]
                                          [--col_team COL_TEAM]
                                          [--col_opp COL_OPP]
                                          [--col_is_home COL_IS_HOME]
                                          [--col_posreal COL_POSREAL]
                                          [--col_de COL_DE] [--col_g COL_G]
                                          [--col_a COL_A]
                                          [--col_basica COL_BASICA]
                                          [--col_gf COL_GF] [--col_ga COL_GA]
                                          [--col_gols_casa COL_GOLS_CASA]
                                       

In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --fixtures_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01.csv" \
  --n_jogos 5 \
  --out_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01_CORRETA.csv" \
  --col_match_id "ID.1" \
  --col_team "Time" \
  --col_opp "Adversário" \
  --col_is_home "Mando" \
  --col_posreal "Posreal" \
  --col_de "DE" \
  --col_g "G" \
  --col_a "A" \
  --col_basica "Básica"



ERRO: 'Namespace' object has no attribute 'col_sg_scout'


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --fixtures_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01.csv" \
  --n_jogos 5 \
  --out_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01_CORRETA.csv" \
  --col_match_id "ID.1" \
  --col_team "Time" \
  --col_opp "Adversário" \
  --col_is_home "Mando" \
  --col_posreal "Posreal" \
  --col_de "DE" \
  --col_g "G" \
  --col_a "A" \
  --col_basica "Básica"


CSV GERADO: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R01_CORRETA.csv


In [ ]:
import pandas as pd
import numpy as np

# Caminhos (ajuste se necessário)
EXCEL = "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx"
SHEET = "Por jogo"

CONFRONTOS_ORIG = "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/CONFRONTOS_2026_NORMALIZADO.csv"
CONFRONTOS_TESTE = "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/CONFRONTOS_2026_NORMALIZADO_TESTE.csv"

RODADA_FAKE = 39
SEED = 2026039  # mantém reprodutível

# 1) Times reais presentes na base 2025
df = pd.read_excel(EXCEL, sheet_name=SHEET)
times = sorted(df["Time"].dropna().astype(str).str.strip().unique().tolist())

print("Times encontrados na base:", len(times))
print(times)

if len(times) % 2 != 0:
    raise ValueError("Número ímpar de times detectado. Precisa ser par para montar confrontos.")

# 2) Gerar pareamentos aleatórios (mandante/visitante)
rng = np.random.default_rng(SEED)
rng.shuffle(times)
pares = list(zip(times[0::2], times[1::2]))  # (mandante, visitante)

fake = pd.DataFrame({
    "RODADA": [RODADA_FAKE]*len(pares),
    "MANDANTE": [a for a,b in pares],
    "VISITANTE": [b for a,b in pares],
})

print("\nRODADA FAKE GERADA:")
print(fake)

# 3) Carregar confrontos originais e salvar teste (com rodada 39 adicionada)
conf = pd.read_csv(CONFRONTOS_ORIG)

# Evita duplicar se você rodar duas vezes
conf = conf[conf["RODADA"] != RODADA_FAKE].copy()

conf_teste = pd.concat([fake, conf], ignore_index=True)

# Ordena só para ficar “bonito”
conf_teste = conf_teste.sort_values(["RODADA", "MANDANTE", "VISITANTE"]).reset_index(drop=True)

conf_teste.to_csv(CONFRONTOS_TESTE, index=False, encoding="utf-8")
print("\nSALVO:", CONFRONTOS_TESTE)


Times encontrados na base: 20
['Atlético-MG', 'Bahia', 'Botafogo', 'Ceará', 'Corinthians', 'Cruzeiro', 'Flamengo', 'Fluminense', 'Fortaleza', 'Grêmio', 'Internacional', 'Juventude', 'Mirassol', 'Palmeiras', 'Red Bull Bragantino', 'Santos', 'Sport', 'São Paulo', 'Vasco', 'Vitória']

RODADA FAKE GERADA:
   RODADA             MANDANTE    VISITANTE
0      39                Bahia        Ceará
1      39             Botafogo     Flamengo
2      39          Corinthians  Atlético-MG
3      39               Grêmio     Mirassol
4      39                Sport    Juventude
5      39            São Paulo     Cruzeiro
6      39              Vitória       Santos
7      39                Vasco   Fluminense
8      39        Internacional    Palmeiras
9      39  Red Bull Bragantino    Fortaleza

SALVO: /content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/CONFRONTOS_2026_NORMALIZADO_TESTE.csv


In [ ]:
import os, pandas as pd

ESC_DIR = "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS"
CONFRONTOS_TESTE = "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/CONFRONTOS_2026_NORMALIZADO_TESTE.csv"
RODADA_FAKE = 39

conf = pd.read_csv(CONFRONTOS_TESTE)
r = conf[conf["RODADA"] == RODADA_FAKE].copy()

times = sorted(set(r["MANDANTE"].astype(str).str.strip()) | set(r["VISITANTE"].astype(str).str.strip()))
arquivos = set([f.lower() for f in os.listdir(ESC_DIR)])

faltando = []
for t in times:
    nome = t.strip().lower()
    # ajuste se seu padrão de arquivo tiver sufixo .png sempre
    esperado = nome + ".png"
    if esperado not in arquivos:
        faltando.append(esperado)

print("Times na rodada fake:", len(times))
print("Escudos faltando:", len(faltando))
print(faltando)


Times na rodada fake: 20
Escudos faltando: 5
['atlético-mg.png', 'ceará.png', 'grêmio.png', 'são paulo.png', 'vitória.png']


In [ ]:
import os, unicodedata, re
import pandas as pd

ESC_DIR = "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS"
CONFRONTOS_TESTE = "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/CONFRONTOS_2026_NORMALIZADO_TESTE.csv"
RODADA_FAKE = 39

def key(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("-", " ")                 # hífen -> espaço (resolve Atlético-MG)
    s = re.sub(r"\s+", " ", s)              # colapsa espaços
    s = unicodedata.normalize("NFC", s)     # normaliza acentos
    return s

# indexa arquivos de escudo por chave normalizada
files = [f for f in os.listdir(ESC_DIR) if f.lower().endswith(".png")]
index = {key(os.path.splitext(f)[0]): f for f in files}

conf = pd.read_csv(CONFRONTOS_TESTE)
r = conf[conf["RODADA"] == RODADA_FAKE].copy()
times = sorted(set(r["MANDANTE"].astype(str)) | set(r["VISITANTE"].astype(str)))

faltando = []
presentes = []
for t in times:
    k = key(t)
    if k in index:
        presentes.append((t, index[k]))
    else:
        faltando.append((t, k))

print("Times na rodada:", len(times))
print("Escudos encontrados:", len(presentes))
print("Escudos faltando:", len(faltando))
print("\nFALTANDO (time_original -> chave_normalizada):")
for a,b in faltando:
    print(" -", repr(a), "->", repr(b))

print("\nEXEMPLOS ENCONTRADOS (time_original -> arquivo):")
for a,b in presentes[:10]:
    print(" -", a, "->", b)


Times na rodada: 20
Escudos encontrados: 19
Escudos faltando: 1

FALTANDO (time_original -> chave_normalizada):
 - 'Grêmio' -> 'grêmio'

EXEMPLOS ENCONTRADOS (time_original -> arquivo):
 - Atlético-MG -> atlético mg.png
 - Bahia -> bahia.png
 - Botafogo -> botafogo.png
 - Ceará -> ceará.png
 - Corinthians -> corinthians.png
 - Cruzeiro -> cruzeiro.png
 - Flamengo -> flamengo.png
 - Fluminense -> fluminense.png
 - Fortaleza -> fortaleza.png
 - Internacional -> internacional.png


In [ ]:
import unicodedata, re

def escudo_key(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("-", " ")
    s = re.sub(r"\s+", " ", s)
    # remove acentos
    s = "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )
    return s


In [ ]:
import os, unicodedata, re
import pandas as pd

ESC_DIR = "/content/drive/MyDrive/TCC TABELAS 2026/04_ESCUDOS"
CONFRONTOS_TESTE = "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/CONFRONTOS_2026_NORMALIZADO_TESTE.csv"
RODADA = 39

def escudo_key(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("-", " ")
    s = re.sub(r"\s+", " ", s)
    s = "".join(c for c in unicodedata.normalize("NFD", s)
                if unicodedata.category(c) != "Mn")
    return s

# Indexa escudos
files = [f for f in os.listdir(ESC_DIR) if f.lower().endswith(".png")]
escudos = {escudo_key(os.path.splitext(f)[0]): f for f in files}

conf = pd.read_csv(CONFRONTOS_TESTE)
r = conf[conf["RODADA"] == RODADA].copy()

times = sorted(set(r["MANDANTE"].astype(str)) | set(r["VISITANTE"].astype(str)))

faltando = []
for t in times:
    if escudo_key(t) not in escudos:
        faltando.append(t)

print("Times na rodada:", len(times))
print("Escudos encontrados:", len(times) - len(faltando))
print("Escudos faltando:", len(faltando))
print("Lista:", faltando)


Times na rodada: 20
Escudos encontrados: 20
Escudos faltando: 0
Lista: []


In [ ]:
!RUN_DIR="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"

!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --confrontos_csv "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/CONFRONTOS_2026_NORMALIZADO_TESTE.csv" \
  --piloto_csv "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/PILOTO_GERACAO_TABELAS.csv" \
  --out_csv "${RUN_DIR}/TABELA_LATERAIS_R39_FINAL.csv"


Traceback (most recent call last):
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py", line 301, in <module>
    main()
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py", line 209, in main
    rodada, n_jogos, filtro = read_piloto(args.piloto_csv)
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py", line 31, in read_piloto
    raise ValueError(f"ERRO: PILOTO sem coluna de rodada. Colunas: {list(p.columns)}")
ValueError: ERRO: PILOTO sem coluna de rodada. Colunas: ['CHAVE', 'VALOR']


In [ ]:
!RUN_DIR="/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847"

!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --confrontos_csv "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/CONFRONTOS_2026_NORMALIZADO_TESTE.csv" \
  --piloto_csv "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/PILOTO_GERACAO_TABELAS.csv" \
  --out_csv "${RUN_DIR}/TABELA_LATERAIS_R39_FINAL.csv"


OK: CSV LATERAIS gerado (rodada=39, n_jogos=5, filtro=MANDO)
SAÍDA: /TABELA_LATERAIS_R39_FINAL.csv


In [ ]:
import pandas as pd

CSV = "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R39_FINAL.csv"
df = pd.read_csv(CSV)

expected = [
    "MANDANTE","VISITANTE",
    "CASA_LE_COC_DE","CASA_LE_CDF_DE","CASA_LE_COC_PG","CASA_LE_CDF_PG","CASA_LE_COC_MB","CASA_LE_CDF_MB",
    "CASA_LD_COC_DE","CASA_LD_CDF_DE","CASA_LD_COC_PG","CASA_LD_CDF_PG","CASA_LD_COC_MB","CASA_LD_CDF_MB",
    "CASA_SG_COC","CASA_SG_CDF",
    "CASA_X_FORA",
    "FORA_SG_COF","FORA_SG_CDC",
    "FORA_LD_COF_MB","FORA_LD_CDC_MB","FORA_LD_COF_PG","FORA_LD_CDC_PG","FORA_LD_COF_DE","FORA_LD_CDC_DE",
    "FORA_LE_COF_MB","FORA_LE_CDC_MB","FORA_LE_COF_PG","FORA_LE_CDC_PG","FORA_LE_COF_DE","FORA_LE_CDC_DE"
]

cols = list(df.columns)
print("LINHAS:", len(df))
print("MISSING:", [c for c in expected if c not in cols])
print("EXTRA:", [c for c in cols if c not in expected])
print("ORDER_OK:", cols == expected)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R39_FINAL.csv'

In [ ]:
!ls -lah | grep TABELA_LATERAIS_R39_FINAL || true


In [ ]:
!find /content -maxdepth 4 -type f -name "TABELA_LATERAIS_R39_FINAL.csv" 2>/dev/null | head -n 20


In [ ]:
!find /content -type f -name "TABELA_LATERAIS_R39_FINAL.csv" 2>/dev/null | head -n 20


^C


In [ ]:
!ls -lah "/content/drive/MyDrive/TCC TABELAS 2026" | head


total 21K
drwx------ 2 root root 4.0K Jan  5 23:29 01_INPUT
drwx------ 3 root root 4.0K Jan  1 14:05 02_OUTPUT
drwx------ 2 root root 4.0K Jan  5 17:47 03_CONFIG
drwx------ 2 root root 4.0K Jan  4 14:51 04_ESCUDOS
-rw------- 1 root root 4.4K Jan  3 20:10 README_TCC.md


In [ ]:
!python3 "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/GERAR_TABELA_LATERAIS_TCC_FINAL.py" \
  --excel "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/Scouts_Reorganizado.xlsx" \
  --sheet "Por jogo" \
  --confrontos_csv "/content/drive/MyDrive/TCC TABELAS 2026/01_INPUT/CONFRONTOS_2026_NORMALIZADO_TESTE.csv" \
  --piloto_csv "/content/drive/MyDrive/TCC TABELAS 2026/03_CONFIG/PILOTO_GERACAO_TABELAS.csv" \
  --out_csv "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R39_FINAL.csv"

!ls -lah "/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R39_FINAL.csv"


OK: CSV LATERAIS gerado (rodada=39, n_jogos=5, filtro=MANDO)
SAÍDA: /content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R39_FINAL.csv
-rw------- 1 root root 1.6K Jan  5 23:53 '/content/drive/MyDrive/TCC TABELAS 2026/02_OUTPUT/RUN_2026_R01_POR_MANDO_N5_20260103_230847/TABELA_LATERAIS_R39_FINAL.csv'
